<a href='https://github.com/MadsLorentzen/geophysics_copenhagen'>
    <img src= 'https://user-images.githubusercontent.com/50207393/57354680-68ca3600-716c-11e9-8bf3-3054c7eec0ab.pdf' ,width=100, height= 100 ></a>
    
    
# TABLE OF CONTENT

## [Wavelets](#wavelets)
## [Blocky model](#blockymodel)
## [AVO modelling](#AVOmodelling)
## [Rock Physics (IF) model](#RPmodelling)
## [Interactive IF model](#RPmodelling_interactive)
## [Wedge Modelling](#wedge)


## [LIVE DEMO 1](#RPmodelling_interactive)
## [LIVE DEMO 2](#wedge_interactive)

In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from matplotlib import gridspec
import lasio as la
from ipywidgets import *
#import seaborn as sns
import scipy as sc
import bruges as br
import mlo_module as mlo
from my_plot import set_size
width = 433.62
import segyio
from shutil import copyfile
from scipy import interpolate
from scipy.interpolate import make_interp_spline


import svgutils.compose as sc
from IPython.display import SVG # /!\ note the 'SVG' function also in svgutils.compose

#x_smooth = np.linspace(x.min(), x.max(), 300)
#y_smooth = make_interp_spline(x, y)(x_smooth)
import urllib

%matplotlib inline


plt.style.use('default')
mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'


mpl.rcParams['axes.labelsize' ] = 8
mpl.rcParams['font.size'      ] = 8
mpl.rcParams['legend.fontsize'] = 6
mpl.rcParams['xtick.labelsize'] = 6
mpl.rcParams['ytick.labelsize'] = 6
    
    
path_data ='//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/'
path_las = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/well logs/'
path_wvlt = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/wavelets/'
fig_path = 'M:/LOCRETA_WORKPACKAGE_D/Valdemar_sprint_well_data_Mads_L/MLO documents/Figures/'
path_hrs = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/HRS project/Locreta_proj_Mads_localtest.prj/seismic.dir/'
#showfig = True
savefig = True


In [ ]:
import os.path as path
import matplotlib as mpl
print("Your style sheets are located at: {}".format(path.join(mpl.__path__[0], 'mpl-data', 'stylelib')))
#las=la.read(path_las+"BO-1X_logs_HRS.las")
#las.to_csv(path_las+"BO-1X_logs_HRS.csv")
##
#las=la.read(path_las+"BO-2X_logs_HRS_27ms.las")
#las.to_csv(path_las+"BO-2X_logs_HRS_27ms.csv")
#
#las=la.read(path_las+"BO-3X_logs_HRS.las")
#las.to_csv(path_las+"BO-3X_logs_HRS.csv")

# Extract well log data

In [ ]:
bo2 = pd.read_csv(path_las+'BO-2X_logs_HRS_27ms.csv')
bo2 = bo2.fillna(0)
info_units = bo2[:1]
bo2 = bo2.drop([0]) #Getting rid of first row with units as strings
bo2= bo2.astype(float) #replace strings with floats 
bo2.DT = bo2.DT
bo2.DTDS = bo2.DTDS **-1 *10**6 *.3048
bo2.ZDEN = bo2.ZDEN * 1000.0
bo2.loc[:, 'fm'] = 0 #formation index
bo2 = bo2.replace(np.inf, 0)
bo2.DPTM = bo2.DPTM/1000. #ms to s 
#bo2.shape

# Smoothing and despiking the logs

In [ ]:
#def rolling_window(a, window):
#        shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
#        strides = a.strides + (a.strides[-1],)
#        rolled = np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
#        
#        return rolled
#    
#    
#window = 13 # the length of filter is 13 samples or ~ 2 metres
#
## Smooth density
#rho_sm = np.median(rolling_window(bo2['ZDEN'].values,window), -1) # Smooths the logs using a median filter
#bo2['rho_sm'] = np.pad(rho_sm, np.int(window/2), mode='edge') # "Pads" the log together such that we get original log size
#
## Smooth P-sonic
#dt_sm = np.median(rolling_window(bo2['DT'].values,window), -1) # Smooths the logs using a median filter
#bo2['vp_sm'] = np.pad(dt_sm, np.int(window/2), mode='edge') # "Pads" the log together such that we get original log size
#
## Smoothh S-sonic
#dts_sm = np.median(rolling_window(bo2['DTDS'].values,window), -1) # Smooths the logs using a median filter
#bo2['vs_sm'] = np.pad(dts_sm, np.int(window/2), mode='edge') # "Pads" the log together such that we get original log size
#
## Despike function
#def despike(curve, curve_sm, max_clip): 
#    spikes = np.where(curve - curve_sm > max_clip)[0]
#    spukes = np.where(curve_sm - curve > max_clip)[0]
#    out = np.copy(curve)
#    out[spikes] = curve_sm[spikes] + max_clip  # Clip at the max allowed diff
#    out[spukes] = curve_sm[spukes] - max_clip  # Clip at the min allowed diff
#    return out
#
#bo2['RHOZ_edt'] = despike(bo2['ZDEN'].values,bo2['rho_sm'].values,max_clip=100) 
#bo2['DT_edt'] = despike(bo2['DT'].values,bo2['vp_sm'].values,max_clip=5) 
#bo2['DTS_edt'] = despike(bo2['DTDS'].values,bo2['vs_sm'].values,max_clip=5)

# Loading formation picks from CRETSYS

In [ ]:
bo2_fms = pd.read_excel(path_las+'CRETSYS_FINAL_PICKS_02-11_17_QC_ET.xlsx', index_col=0,header=None)  
bo2_fms =bo2_fms.drop(columns=[1,3,5,6])
bo2_fms = bo2_fms.loc['BO-2X']
bo2_fms = bo2_fms.rename(columns={1:"well", 2: "fm", 4: "depth"})
bo2_fms["fm name"] = ['Ekofisk', 'ch21', 'U. Tor', 'ch17', 'Lower Tor', 'ch13', 'Hod D', 
                      'Hod C', 'Hod B', 'Hod A', 'Her./blod.' , 'Hidra', 'Rødby',
                      'Sola', 'Tuxen', 'Valhall', 'BCU']

bo2_fms["fm name"].astype('str')
np.array(bo2_fms["fm name"])
bo2_fms = bo2_fms.reset_index()
bo2_fms


# Function to insert row in the dataframe 
def Insert_row_(row_number, df, row_value): 
    # Slice the upper half of the dataframe 
    df1 = df[0:row_number] 
   
    # Store the result of lower half of the dataframe 
    df2 = df[row_number:] 
   
    # Inser the row in the upper half dataframe 
    df1.loc[row_number]=row_value 
   
    # Concat the two dataframes 
    df_result = pd.concat([df1, df2]) 
   
    # Reassign the index labels 
    df_result.index = [*range(df_result.shape[0])] 
   
    # Return the updated dataframe 
    return df_result 
#row_value = ['BO-2X', 0, 7865*.3048 , 'L. Tuxen'] #munk marl interpretation from BO-2X Final Well Report (p.35)
#bo2_fms = Insert_row_(15,bo2_fms,row_value)

bo2_fms

<a id='the_destination'></a>
# Define facies


In [ ]:
bo2_fms = bo2_fms.drop_duplicates(subset='depth',keep='last') # dropping formations with the same picked depth (0 thickness)
bo2_fms = bo2_fms.set_index(0)
#bo2_fms = bo2_fms.reset_index(drop=True, inplace=True)
#bo2_fms[bo2_fms.loc['fm name'] == 'ch17'] = 'Tor Upper'
#bo2_fms['fm name'].loc[bo2_fms['fm name'] == 'ch17'] = 'Tor Upper'
#resSec['F'].loc[(resSec['DEPTH'] > intf[0]) & (resSec['DEPTH'] <= intf[1])] = 0;
bo2_fms['fm name'].loc[bo2_fms['fm name'] == 'ch17'] = 'U. Tor'

In [ ]:
bo2_fms['fm name'].loc[bo2_fms['fm name'] == 'ch17'] = 'Upper Tor'

In [ ]:
idx = np.zeros(np.array(bo2_fms['depth']).size)

for i in range (np.array(bo2_fms['depth']).size):
    idx[i] = abs(bo2['DEPTH'] - np.array(bo2_fms['depth'])[i]).idxmin()

bo2_fms['twt'] = np.array(bo2['DPTM'][idx]*1000.) #np.array([2020.78 , 2026.65, 2043.57, 2067.46, 2094.93, 2126.65, 2160.67, 2187.45, 2224.53, 2267.65, 2363.65, 2373.43])

#bo2_fms['twt'] = np.array([2096,2103,2119,2146,2184,2212,2226,2239,2285,2379])#np.array([2099,2106,2122,2149,2187,2214,2229,2242, 2258,2287,2382])-3 # np.array([2097,2104,2120,2147,2185,2212,2227,2240, 2258,2285,2387])

#np.array([2099,2106,2122,2149,2187,2214,2229,2242, 2258,2287,2382])


bo2_fms

In [ ]:
# Define elastic logs
bo2['Vp'] = bo2['DT'] #bo2['DT_edt']
bo2['Vs'] = bo2['DTDS'] #bo2['DTS_edt']
bo2['AI'] = bo2['Vp'] /1000. * bo2['ZDEN']#bo2['RHOZ_edt']
bo2['PS'] = bo2['Vp'] / bo2['Vs']


intf = np.array(bo2_fms["depth"])
Fstr = np.array(bo2_fms["fm name"])

# Crop reservoir interval
resSec = bo2[(bo2['DEPTH'] > bo2_fms.loc[bo2_fms['fm name'] == 'U. Tor']['depth'][0]) &
             (bo2['DEPTH'] < bo2_fms.loc[bo2_fms['fm name'] == 'BCU']['depth'][0])]

lc = bo2[(bo2['DEPTH'] > bo2_fms.loc[bo2_fms['fm name'] == 'Hidra']['depth'][0]) &
             (bo2['DEPTH'] < bo2_fms.loc[bo2_fms['fm name'] == 'BCU']['depth'][0])]


#resSec = bo2[(bo2['DEPTH'] >= float(top_hidra)) & (bo2['DEPTH'] < float(top_valhall)+30)]
# Subdivide reservoir into ("0") Upper shale zone 1, ("1") Upper shale zone 2, ("2") clean reservoir zone, and ("3") shaly reservoir zone
resSec['F'] = np.zeros(len(resSec))
resSec['F_label'] = ["" for x in range(len(resSec))]

#Removing zeros in the bottom of the Valdemar formation:
resSec = resSec.loc[resSec['AI'] != 0]
lc = lc.loc[lc['AI'] != 0]

#resSec['F'].loc[(resSec['DEPTH'] < intf[0])] = 0
for i in range (len(intf)-1):
    if i == 0:
        resSec['F'].loc[(resSec['DEPTH'] > intf[0]) & (resSec['DEPTH'] <= intf[1])] = 0;
        resSec['F'].loc[(resSec['DEPTH'] >= intf[0]) & (resSec['DEPTH'] <= intf[1])] = 1;
        resSec['F_label'].loc[(resSec['DEPTH'] <= intf[0])] = Fstr[0];
        resSec['F_label'].loc[(resSec['DEPTH'] >= intf[0]) & (resSec['DEPTH'] <= intf[1])] = Fstr[0];
    else:
        resSec['F'].loc[(resSec['DEPTH'] >= intf[i]) & (resSec['DEPTH'] <= intf[i+1])] = i+1;
        resSec['F_label'].loc[(resSec['DEPTH'] >= intf[i]) & (resSec['DEPTH'] <= intf[i+1])] = Fstr[i];

dz = []
for m in range(1,len(Fstr)+1):
    dz.append(resSec[resSec['F']==m]['DEPTH'].diff().sum())

fig = plt.figure(figsize=set_size(width, subplots=(5,2)))
#fig, ax = plt.subplots(10, 8, figsize=set_size(width, subplots=(10, 8)))
#fig.tight_layout(pad=2)
fig.tight_layout
gs1 = gridspec.GridSpec(nrows=12, ncols=6, figure=fig,hspace =0 ,wspace=0.25)#,width_ratios=np.array([.5,.5,.5,.5,.5,.5,.5]))
ax0 = fig.add_subplot(gs1[0:7, 0])
ax1 = fig.add_subplot(gs1[0:7, 2])
ax2 = fig.add_subplot(gs1[0:7, 3])
ax3 = fig.add_subplot(gs1[0:7, 4])
ax4 = fig.add_subplot(gs1[0:7, 5])

#ax5 = fig.add_subplot(gs1[8:11, 2:6])

ax6 = fig.add_subplot(gs1[0:7, 1])

# Porosity
x1 = lc['PHIE']*(1-lc['SW_CRETSYS'])
x2 = lc['PHIE']*lc['SW_CRETSYS']


ax0.set_ylim(np.min(lc['DEPTH']),np.max(lc['DEPTH']))
ax0.invert_yaxis()
for pos, spine in ax0.axes.spines.items():
    spine.set_visible(False)
ax0.set_xticks([])
ax0.set_yticks([])

ax6.set_ylim(np.min(lc['DEPTH']),np.max(lc['DEPTH']))
ax6.invert_yaxis()
for pos, spine in ax0.axes.spines.items():
    spine.set_visible(False)
ax6.set_xticks([])
#ax6.set_yticks([])


x_offset = 0.2
#ax0.annotate('',xy=(x_offset,bo2_fms.loc[bo2_fms['fm name'] == 'Sola']['depth'][0]),xytext=(x_offset, np.min(lc['DEPTH'])),
#            arrowprops=dict(facecolor='black',arrowstyle='<->', linewidth=1),)

ax0.annotate('',xy=(x_offset-0.1,np.max(lc['DEPTH'])),xytext=(x_offset-0.1, bo2_fms.loc[bo2_fms['fm name'] == 'Sola']['depth'][0]),
            arrowprops=dict(facecolor='black',arrowstyle='<->', linewidth=1),)

#ax0.text(x = x_offset-0.2 , y = (np.min(lc['DEPTH']) + bo2_fms.loc[bo2_fms['fm name'] == 'Sola']['depth'][0]) / 2 + 30,  s = 'Upper Chalk', weight='normal', rotation = -270)
ax0.text(x =x_offset-0.35, y =  (np.max(lc['DEPTH']) + bo2_fms.loc[bo2_fms['fm name'] == 'Sola']['depth'][0]) / 2 ,  s = 'LC', weight='normal', rotation = -270)

    
ax1.fill_betweenx(lc['DEPTH'],x2,0, alpha=0.3,color='b',label='Water')
ax1.fill_betweenx(lc['DEPTH'],x2,lc['PHIE'], alpha=0.3,color='r',label='Oil')
ax1.plot(lc['PHIE'],lc['DEPTH'],'k-', lw = 1,label='')
ax1.axis('tight')
ax1.set_ylim(np.min(lc['DEPTH']),np.max(lc['DEPTH']))
ax1.invert_yaxis()
ax1.set_xlim(0,0.5)
ax1.set_xlabel('POR \n (v/v)' )
#ax1.grid()
ax1.set_xticks(np.array([0, 0.5]))
ax1.legend(loc=0,fontsize=6)
ax1.set_yticklabels('')
#ax1.yaxis.set_label_coords(-0.43, .5)


# Shale volume
ax2.fill_between(lc['VSHALE'],lc['DEPTH'], color='k', step="pre", alpha=0.3)
ax2.plot(lc['VSHALE'],lc['DEPTH'],'k-', lw = 1)
ax2.axis('tight')
ax2.set_ylim(np.min(lc['DEPTH']),np.max(lc['DEPTH']))
ax2.invert_yaxis()
ax2.set_xlim(0,1)
ax2.set_xlabel('SHV \n (v/v)' )
#ax2.set_ylabel('Depth (m)' )
ax2.set_yticklabels('')
ax2.set_xticks(np.array([0, 0.5, 1]))
#ax2.grid(axis='x')

# AI
ax3.plot(lc['AI']/1000.,lc['DEPTH'],'k-', lw = 1)
ax3.axis('tight')
ax3.set_ylim(np.min(lc['DEPTH']),np.max(lc['DEPTH']))
ax3.invert_yaxis()
ax3.set_xlabel('AI \n' +r'$(km/s \times g/cm^3)$' )
ax3.set_yticklabels('')
#ax3.grid(axis='x')

# Vp/Vs
ax4.plot(lc['PS'],lc['DEPTH'],'k-', lw = 1)
ax4.axis('tight')
ax4.set_ylim(np.min(lc['DEPTH']),np.max(lc['DEPTH']))
ax4.invert_yaxis()
ax4.set_xlabel(r'$V_p/V_s$' )
ax4.set_yticklabels('')
#ax4.grid(axis='x')
ax4.set_xticks(np.array([1.6, 2, 2.4]))

for n in range(0,len(intf)-5):
    ax1.axhline(y = intf[n], lw = 1, linestyle = '-',
                   alpha = 0.5)    
    ax2.axhline(y = intf[n], lw = 1, linestyle = '-',
                   alpha = 0.5)
    ax3.axhline(y = intf[n], lw = 1, linestyle = '-',
                   alpha = 0.5)
    ax4.axhline(y = intf[n], lw = 1, linestyle = '-',
                   alpha = 0.5)
    ax6.axhline(y = intf[n], lw = 1, linestyle = '-',
                   alpha = 0.5)
    
ax1.axhline(intf[6], color = 'red',   lw=2,   alpha=0.8)
ax1.axhline(intf[7], color = 'green', lw=2,   alpha=0.8)
ax1.axhline(intf[8], color = 'orange',  lw=2,   alpha=0.8)
ax1.axhline(intf[9], color = 'orange',  lw=2,   alpha=0.8)
ax2.axhline(intf[6], color = 'red',   lw=2,   alpha=0.8)
ax2.axhline(intf[7], color = 'green', lw=2,   alpha=0.8)
ax2.axhline(intf[8], color = 'orange',  lw=2,   alpha=0.8)
ax2.axhline(intf[9], color = 'orange',  lw=2,   alpha=0.8)
ax3.axhline(intf[6], color = 'red',   lw=2,   alpha=0.8)
ax3.axhline(intf[7], color = 'green', lw=2,   alpha=0.8)
ax3.axhline(intf[8], color = 'orange',  lw=2,   alpha=0.8)
ax3.axhline(intf[9], color = 'orange',  lw=2,   alpha=0.8)
ax4.axhline(intf[6], color = 'red',   lw=2,   alpha=0.8, label = 'Top Sola')
ax4.axhline(intf[7], color = 'green', lw=2,   alpha=0.8, label = 'Top U. Tuxen')
ax4.axhline(intf[8], color = 'orange',  lw=2,   alpha=0.8, label = 'Top L. Tuxen')
ax4.axhline(intf[9], color = 'orange',  lw=2,   alpha=0.8, label = 'Top Valhall')


ax6.set_ylabel('Depth (m)' )
ax6.axhline(intf[6], color = 'red',   lw=2,   alpha=0.8)#, label = 'Top Sola')
ax6.axhline(intf[7], color = 'green', lw=2,   alpha=0.8)#, label = 'Top U. Tuxen')
ax6.axhline(intf[8], color = 'orange',  lw=2,   alpha=0.8)#, label = 'Top L. Tuxen')
ax6.axhline(intf[9], color = 'orange',  lw=2,   alpha=0.8)#, label = 'Top Valhall')   
for i in range(5,len(Fstr)-1):
    ax6.text(x = ax6.set_xlim()[0]+0.5, y =intf[i]+8, s = Fstr[i]+'\n ({0} m)'.format(str(round(dz[i],1))),
            fontsize = 6,
            horizontalalignment = 'center',
            verticalalignment = 'center',
            #bbox = dict(facecolor = 'white', lw = 0.5),
            weight = 'light',
            rotation=0)

#groups = resSec['F_label'].unique();
#
#cmap_dummy = np.array(['magenta','red','green','orange']);
#k=0;
#import pylab;
#NUM_COLORS = 6
##
#cm = pylab.get_cmap('tab20b');
##for i in range(NUM_COLORS):
##    color = cm(1.*i/NUM_COLORS)  # color will now be an RGBA tuple
#
#
#marker_size = 30
#
##for i in range(len(groups)-4):
##    x, y = (lc[lc['F_label']==groups[i]]['AI']/1000.,lc[lc['F_label']==groups[i]]['PS']);
##    col = cm(1.*i/NUM_COLORS)  # color will now be an RGBA tuple
##    ax5.scatter(x, y, alpha=0.2, s=marker_size, label=groups[i], color=col,marker = '+');
#for i in range(5,len(groups)):
#    x, y = (resSec[resSec['F_label']==groups[i]]['AI']/1000.,resSec[resSec['F_label']==groups[i]]['PS']);
#    ax5.scatter(x, y, alpha=0.2, s=marker_size, label=groups[i], color = cmap_dummy[k]);
#    k=k+1
#        
#ax5.set_xlabel('AI '+ r'$(km/s \times g/cm^3)$' );
#ax5.set_ylabel(r'$V_p/V_s$' );
#ax5.grid();
#ax5.set_axisbelow(True);
#lg = ax5.legend(bbox_to_anchor=(1.0, 0.95),title='Facies',fancybox=True,shadow=True)



ax6.text(x = ax6.get_xlim()[0]-0.2, y = ax6.get_ylim()[1]-4,  s = 'a)', weight='normal')
ax1.text(x = ax1.get_xlim()[0]-0.12, y = ax1.get_ylim()[1]-4,  s = 'b)', weight='normal')
ax2.text(x = ax2.get_xlim()[0]-0.2, y =  ax2.get_ylim()[1]-4,  s = 'c)', weight='normal')
ax3.text(x = ax3.get_xlim()[0]-2.5, y =  ax3.get_ylim()[1]-4,  s = 'd)', weight='normal')
ax4.text(x = ax4.get_xlim()[0]-0.17, y = ax4.get_ylim()[1]-4  ,  s = 'e)', weight='normal')
#ax5.text(x = ax5.get_xlim()[0]-0.3, y = ax5.get_ylim()[1]+0.02  ,  s = 'f)', weight='normal')



ax4.text(x = ax4.get_xlim()[1]+0.1, y = intf[6]+2,  s = 'Top Sola', weight='normal', color = 'red')
ax4.text(x = ax4.get_xlim()[1]+0.1, y = intf[7]+2,  s = 'Top Tuxen', weight='normal', color = 'green')
ax4.text(x = ax4.get_xlim()[1]+0.1, y = intf[8]+2,  s = 'Top Valhall', weight='normal', color = 'orange')
#ax4.text(x = ax4.get_xlim()[1]+0.1, y = intf[9]+2,  s = 'Top Valhall', weight='normal', color = 'orange')





if savefig == True:
    plt.savefig(fig_path+'welllogs.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')
plt.show()

In [ ]:
bo1_ai = np.array([5582.30720693, 5588.24303572, 5873.12783714, 5939.91648741,
       5987.77015222, 6017.92999612, 6068.37891318, 6063.28098483,
       6098.07278592, 6134.32343252, 6132.8378457 , 6067.04020607,
       6100.20382922, 6066.49654012, 6007.31210325, 5920.87801529,
       5806.14591594, 5796.77963256, 5750.75035058, 5697.84312991,
       5691.22972733, 5480.06406964, 5512.38143417, 5505.37717689,
       5363.70585416, 5359.9453568 , 5344.31513386, 5287.89979794,
       5135.0544926 , 5054.83418907, 5113.73025004, 5204.80426911,
       5187.89514648, 5051.9294092 , 5016.05251657, 5048.22598802,
       5105.23851859, 5214.75387804, 5330.73434722, 5409.1781871 ,
       5573.94878583, 5746.59089606, 5862.87220272, 5874.28953516,
       5804.03549314, 5730.70818356, 5688.58273707, 5725.00263298,
       5785.23061421, 5856.59612173, 5918.51296355, 5967.41765908,
       6017.38946011, 6050.24219425, 6022.70212157, 5922.67449908,
       5820.56838055, 5780.59677407, 5790.95289084, 5822.48075315,
       5886.70195175, 5826.89476565, 5745.29352249, 5722.25480092,
       5712.33274512, 5620.18144844, 5616.3970761 , 5579.73783195,
       5370.76701269, 5190.36655045, 5158.40968969, 5196.56713538,
       5253.56481988, 5269.54195546, 5270.37445697, 5230.14528361,
       5239.61118159, 5257.59638776, 5246.47395763, 5262.8240586 ,
       5298.82375515, 5362.79050818, 5351.35978007, 5213.48810286,
       5194.0509991 , 5229.28641656, 5299.77490294, 5361.22319331,
       5328.94122199, 5254.89307391, 5266.96549842, 5256.5093804 ,
       5268.35975898, 5290.33465717, 5298.88045091, 5345.15871894,
       5278.3722685 , 5290.03690535, 5381.43281373, 5450.97900008,
       5547.0619656 , 5698.86784108, 5804.36418673, 5898.84629219,
       5868.34792625, 6031.30872174, 6226.01124555, 6240.27219786,
       6203.71138844, 6174.58815327, 6245.26193051, 6279.09969782,
       6275.81206762, 6256.36025565, 6131.46539938, 5996.40688394,
       5901.80858085, 5719.96822006, 5160.95592905, 4748.57748331,
       4704.24163627, 4701.12572385, 4692.58997949, 4674.07444037,
       4691.4971617 , 4651.60807819, 4673.20592078, 4732.85239736,
       4679.02801612, 4725.18393554, 4827.32904119, 4964.26271074,
       5070.28283121, 5222.33328402, 5443.68940219, 5640.2785328 ,
       5838.33913563, 5989.88417643, 5976.62636875, 5909.76899946,
       5861.30165281, 5871.02592949, 5902.91936491, 5948.97759487,
       5965.70141838, 5972.52229925, 5947.07343524, 5892.11646529,
       5836.61233785, 5692.68359025, 5591.68734398, 5523.82608773,
       5510.60490534, 5524.25231454, 5540.34040674, 5580.49891174,
       5625.57498211, 5653.46384911, 5655.95256328, 5603.65119611,
       5550.85819056, 5540.78653531, 5546.2814612 , 5568.33211753,
       5548.93664246, 5506.53644879, 5502.67095587, 5458.89103871,
       5450.64448497, 5393.93124233, 5351.78530785, 5384.31216483,
       5471.62791818, 5572.7350059 , 5605.59870374, 5547.80094982,
       5489.12413846, 5487.36932127, 5460.49615572, 5374.68129313,
       5284.66224418, 5326.08363865, 5360.09010559, 5399.96349843,
       5509.19469258, 5556.34850339, 5584.78741536, 5568.90261768,
       5487.2491867 , 5455.78744172, 5534.31956536, 5634.84506541,
       5644.3167982 , 5660.0491868 , 5706.52389413, 5772.92761715,
       5917.95761557, 5953.15880549, 6043.07351242, 6143.45590166,
       6233.3706086 , 6284.36700955, 6290.27185597, 6182.21143979,
       6026.55277523, 5982.14015825, 6009.53237112, 6024.40976687,
       6076.41254411, 6119.14642463, 6063.02050678, 5973.93493594,
       5944.45399229, 5925.4087809 , 5866.2556402 , 5768.36724679,
       5750.87785384, 5827.62235427, 5894.18646179, 5930.99249771,
       6005.64464092, 6093.45985318, 6145.61844736, 6192.25001015,
       6248.10250343, 6270.53528474, 6244.95348546, 6198.49709197,
       6161.86625292, 6166.14565278, 6130.02778107, 6111.14250029,
       6035.24889096, 5915.47206242, 5892.59303583, 5909.25560027,
       5927.54889109, 5951.23534065, 5957.48563737, 5865.00072377,
       5819.82576221, 5838.76307815, 5866.95714319, 5928.38486811,
       5995.90923707, 6050.07066202, 6110.86235007, 6006.46606799,
       5913.62312903, 5956.37284386, 5998.59802231, 6028.25770605,
       6117.33338083, 6183.44374896, 6084.68482996, 5996.58658958,
       5958.22616588, 5885.49897299, 5806.91729141, 5800.48346176,
       5825.42009676, 5827.31760371, 5830.87904766, 5836.73809676,
       5825.10548327, 5782.4024489 , 5727.09430198, 5699.70635808,
       5756.11488846, 5802.51242998, 5835.06761105, 5929.91341415,
       6115.50358023, 6158.25288123, 6113.74042807, 6076.0094185 ,
       6017.40342034, 5979.63511041, 6010.63117166, 6002.03562527,
       5905.66131718, 5721.0252555 , 5569.28683466, 5549.65602275,
       5579.48465903, 5636.3375299 , 5694.21018321, 5753.61251017,
       5838.46484564, 6015.61805718, 6097.58506711, 6118.99830076,
       6163.65405497, 6319.19683173, 6445.68787066, 6425.39548554,
       6409.57910696, 6407.39069704, 6403.83453092, 6350.4920391 ,
       6248.69317103, 6107.67132647, 5927.89128608, 5917.65748252,
       5969.17207212, 5812.37852524, 5634.31657395, 5552.08593028,
       5557.48217288, 5640.45697142, 5767.78331969, 5773.79873772,
       5765.27689551, 5779.06222849, 5719.65997546, 5650.95554417,
       5713.17485652, 5780.05461868, 5877.92689762, 6025.06554221,
       6113.42311293, 6167.25913991, 6237.9801653 , 6404.52221068,
       6516.11423918, 6501.63858065, 6507.90955639, 6515.81643883,
       6505.1830452 , 6506.00099855, 6494.82230268, 6398.72168999,
       6221.41662404, 6146.81630304, 6120.97807237, 6105.0300621 ,
       6164.95926867, 6158.47365643, 6038.45781272, 5909.70088662,
       5853.58217638, 5817.39496056, 5696.08333068, 5604.84145152,
       5531.24729503, 5533.7335841 , 5578.73541628, 5632.19063129,
       5756.81520973])

bo1_vpvs = np.array([2.07209379, 2.08849422, 2.15968563, 2.14118501, 2.13449154,
       2.13102253, 2.1517267 , 2.15440775, 2.15548115, 2.14433443,
       2.13935036, 2.11666865, 2.11338811, 2.12405772, 2.14128834,
       2.14011388, 2.11713114, 2.1202122 , 2.14217828, 2.16958258,
       2.18276815, 2.10384114, 2.08941452, 2.03632745, 1.95806181,
       1.95581801, 1.96482295, 1.9842816 , 1.9429576 , 1.86600891,
       1.81978746, 1.79874461, 1.81487353, 1.84580491, 1.86500223,
       1.85033356, 1.83036204, 1.81508015, 1.80283991, 1.79923068,
       1.8315679 , 1.86612863, 1.89694181, 1.92959486, 1.96238135,
       2.00345453, 2.01995935, 2.00957989, 1.98819418, 1.97290369,
       1.9537669 , 1.94557328, 1.93851477, 1.92161495, 1.93137513,
       1.96467884, 2.00314219, 2.03000877, 2.03009022, 2.01845166,
       2.02326369, 2.01439522, 2.01874266, 2.02192068, 2.00138747,
       1.94833386, 1.94539406, 1.96095357, 1.93503635, 1.92193747,
       1.93751374, 1.92062783, 1.8903343 , 1.86430837, 1.8595877 ,
       1.85598954, 1.84961846, 1.84126412, 1.83898729, 1.83578916,
       1.82541768, 1.82969386, 1.82678444, 1.8065985 , 1.81863888,
       1.81365562, 1.81391863, 1.84258509, 1.89573229, 1.9450922 ,
       1.98221536, 1.99096286, 1.99701824, 1.98458014, 1.9817052 ,
       2.0420843 , 2.06638965, 2.06711931, 2.04700995, 1.99987768,
       1.95588001, 1.90143095, 1.85537401, 1.87647626, 1.88501225,
       1.9194298 , 1.95378805, 1.97931854, 2.00853139, 2.02717412,
       2.05724898, 2.06322351, 2.06305543, 2.06821387, 2.05692359,
       2.06242056, 2.1045804 , 2.15933214, 2.04518895, 1.89738433,
       1.8763054 , 1.86903948, 1.85384673, 1.8357326 , 1.81678107,
       1.77870288, 1.76887601, 1.7951424 , 1.79924331, 1.81318971,
       1.80104185, 1.77042142, 1.73217393, 1.73826303, 1.7983121 ,
       1.86296619, 1.90279532, 1.92686856, 1.94557961, 1.97703088,
       1.9945984 , 2.00242756, 1.99936885, 1.98801723, 1.97588252,
       1.96751392, 1.97466489, 1.9954098 , 2.01551886, 2.01071402,
       2.03065753, 2.05391695, 2.06704001, 2.0685069 , 2.04830068,
       2.01543156, 1.97946755, 1.95182635, 1.9454221 , 1.93883091,
       1.93280074, 1.93442187, 1.92791754, 1.91726054, 1.91872605,
       1.94172617, 1.96909638, 1.96342272, 1.97904275, 2.00926322,
       2.01735448, 1.99306532, 1.95088817, 1.91089908, 1.90094774,
       1.91971157, 1.94050026, 1.95426261, 1.96916028, 2.00869415,
       2.04661588, 2.06399115, 2.04970714, 2.0612173 , 2.09847181,
       2.08778627, 2.07660186, 2.0870574 , 2.11442419, 2.14742183,
       2.13219696, 2.09487763, 2.06466461, 2.05044569, 2.0411412 ,
       2.05038509, 2.10123473, 2.0855184 , 2.05235044, 2.01687907,
       1.98772526, 1.97068629, 1.97341399, 1.99361936, 2.02076452,
       2.0383323 , 2.02760465, 1.98729624, 1.95940635, 1.956249  ,
       1.95970786, 1.96239429, 1.97875029, 1.99508616, 2.02220219,
       2.05105326, 2.0538811 , 2.02683417, 1.99705011, 1.9747685 ,
       1.95783915, 1.94956489, 1.93775895, 1.92256536, 1.90815729,
       1.90234262, 1.91148837, 1.9227259 , 1.92006275, 1.91701323,
       1.9181217 , 1.94103122, 1.96957309, 1.99154522, 2.0033621 ,
       1.99612473, 1.98631746, 1.97546502, 1.97260637, 1.96050043,
       1.97091157, 1.99130528, 2.00756408, 2.01890607, 1.99385377,
       1.94830774, 1.94635101, 1.97105664, 1.9943173 , 1.98942898,
       1.97239174, 1.94780562, 1.91606837, 1.90197936, 1.88531782,
       1.89348988, 1.91999594, 1.95116463, 1.97147744, 1.97693702,
       1.97098006, 1.965703  , 1.97274276, 1.96883256, 1.96677103,
       1.98014906, 1.99545602, 1.98674169, 1.97000809, 1.9511371 ,
       1.93141997, 1.89508073, 1.85210231, 1.84382586, 1.887291  ,
       1.91528188, 1.93356273, 1.94386571, 1.94452663, 1.95500827,
       1.99030946, 2.02019396, 2.03220033, 2.03989773, 2.02487858,
       2.00034314, 1.97537841, 1.94974553, 1.93254441, 1.98319378,
       2.01292118, 2.00712904, 2.00662526, 2.01003926, 1.99523387,
       1.98226762, 1.99771812, 2.003008  , 2.00881534, 2.02650293,
       2.04254076, 2.0446055 , 2.02139132, 2.00188899, 1.98469526,
       1.96666473, 1.98138862, 1.99239088, 1.97122259, 1.91876964,
       1.88576856, 1.87414416, 1.87139449, 1.87411555, 1.89324592,
       1.91894336, 1.94171283, 1.92057958, 1.9005559 , 1.90016641,
       1.89587387, 1.89521066, 1.89922494, 1.92673103, 1.94743336,
       1.95227936, 1.95204925, 1.95073509, 1.95256173, 1.95531037,
       1.9625633 , 1.97101722, 1.97337158, 1.97715134, 1.9757949 ,
       1.96995663, 1.96449982, 1.96171223, 1.97656656, 1.96607247,
       1.92321671, 1.89265293, 1.87722701, 1.90252015, 1.92947924,
       1.931875  , 1.91396082, 1.88468439, 1.85020329])

In [ ]:
fig = plt.figure(figsize=(8,8))
#fig, ax = plt.subplots(10, 8, figsize=set_size(width, subplots=(10, 8)))
#fig.tight_layout(pad=2)
fig.tight_layout
#gs1 = gridspec.GridSpec(nrows=12, ncols=6, figure=fig,hspace =0 ,wspace=0.25)#,width_ratios=np.array([.5,.5,.5,.5,.5,.5,.5]))
#ax5 = fig.add_subplot(gs1[8:11, 2:6])

groups = resSec['F_label'].unique();

cmap_dummy = np.array(['magenta','red','green','orange']);
k=0;
import pylab;
NUM_COLORS = 6
#
cm = pylab.get_cmap('tab20b');
#for i in range(NUM_COLORS):
#    color = cm(1.*i/NUM_COLORS)  # color will now be an RGBA tuple


marker_size = 130

#for i in range(len(groups)-4):
#    x, y = (lc[lc['F_label']==groups[i]]['AI']/1000.,lc[lc['F_label']==groups[i]]['PS']);
#    col = cm(1.*i/NUM_COLORS)  # color will now be an RGBA tuple
#    ax5.scatter(x, y, alpha=0.2, s=marker_size, label=groups[i], color=col,marker = '+');
for i in range(5,len(groups)):
    x, y = (resSec[resSec['F_label']==groups[i]]['AI']/1000.,resSec[resSec['F_label']==groups[i]]['PS']);
    plt.scatter(x, y, alpha=0.2, s=marker_size, label=groups[i], color = cmap_dummy[k]);
    k=k+1
    
plt.scatter(bo1_ai/1000. , bo1_vpvs, s=marker_size ,color = 'k',  marker = '+')

plt.xlabel('AI '+ r'$(km/s \times g/cm^3)$' );
plt.ylabel(r'$V_p/V_s$' );
plt.grid();
#plt.axisbelow(True);
lg = plt.legend(bbox_to_anchor=(1.0, 0.95),title='Facies',fancybox=True,shadow=True)


In [ ]:
len(groups)

# Blocky model input

In [ ]:
###########################################################
#       DEFINE MODELING PARAMETERS HERE

vp_mod = resSec.groupby('F').mean()['Vp'].values  # P-wave velocity (m/s)
vs_mod = resSec.groupby('F').mean()['Vs'].values   # S-wave velocity (m/s)
rho_mod = resSec.groupby('F').mean()['ZDEN'].values#['RHOZ_edt'].values   # Density
lit_mod = resSec.groupby('F').mean()['VSHALE'].values   # Shale
phi_mod = resSec.groupby('F').mean()['PHIE'].values   # Density
sw_mod = resSec.groupby('F').mean()['SW_CRETSYS'].values

thickness = np.round(dz,1)[1:-1] # vertical thickness of layer 2 in metres

#   Angle range for incident rays
theta1_min = 0.0    # best to leave this set to zero
theta1_max = 40.0
theta1_step= 1.0


# TWT-Depth Curve (from BO-2X checkshot)

In [ ]:
resSec['DPTM'].min()

tmin = resSec['DPTM'].min()
tmax = resSec['DPTM'].max()

md_min = resSec['DEPTH'].min()   #BO2X_checkshot['MD'].loc[BO2X_checkshot['MD'] >= resSec['DEPTH'].min()].min()
md_max = resSec['DEPTH'].max()

#   Plotting Display Parameters
#min_plot_time = tmin
#max_plot_time = tmax
excursion = 2

resSec['vp_mod']  = 0
resSec['vp_mod']  = 0
resSec['vs_mod']  = 0
resSec['rho_mod'] = 0
resSec['lit_mod'] = 0
resSec['phi_mod'] = 0
resSec['ai_mod'] = 0
resSec['sw_mod'] = 0

for i in range (len(intf)-1):
    if i == 0:
        resSec['vp_mod'].loc[(resSec['DEPTH']  > intf[0]) &  (resSec['DEPTH'] <= intf[1])] =  vp_mod[0]
        resSec['vs_mod'].loc[(resSec['DEPTH']  > intf[0]) &  (resSec['DEPTH'] <= intf[1])] =  vs_mod[0]
        resSec['rho_mod'].loc[(resSec['DEPTH'] > intf[0]) &  (resSec['DEPTH'] <= intf[1])] = rho_mod[0]
        resSec['lit_mod'].loc[(resSec['DEPTH'] > intf[0]) &  (resSec['DEPTH'] <= intf[1])] = lit_mod[0]
        resSec['phi_mod'].loc[(resSec['DEPTH'] > intf[0]) &  (resSec['DEPTH'] <= intf[1])] = phi_mod[0]
        resSec['sw_mod'].loc[(resSec['DEPTH'] > intf[0]) &  (resSec['DEPTH'] <= intf[1])] = sw_mod[0] 
        
    else:
        resSec['vp_mod'].loc[(resSec['DEPTH']  >= intf[i]) &  (resSec['DEPTH'] <= intf[i+1])] =  vp_mod[i]
        resSec['vs_mod'].loc[(resSec['DEPTH']  >= intf[i]) &  (resSec['DEPTH'] <= intf[i+1])] =  vs_mod[i]
        resSec['rho_mod'].loc[(resSec['DEPTH'] >= intf[i]) &  (resSec['DEPTH'] <= intf[i+1])] = rho_mod[i]
        resSec['lit_mod'].loc[(resSec['DEPTH'] >= intf[i]) &  (resSec['DEPTH'] <= intf[i+1])] = lit_mod[i]
        resSec['phi_mod'].loc[(resSec['DEPTH'] >= intf[i]) &  (resSec['DEPTH'] <= intf[i+1])] = phi_mod[i]
        resSec['sw_mod'].loc[(resSec['DEPTH'] >= intf[i]) &  (resSec['DEPTH'] <= intf[i+1])] = sw_mod[i]

        
resSec['ai_mod'] = resSec['vp_mod']/1000. * resSec['rho_mod']       

Z = np.array(resSec['ai_mod'])
RC = (Z[1:] - Z[:-1]) / (Z[1:] + Z[:-1])
RC_dummy = np.zeros(RC.size+1)
RC_dummy[:-1] = RC

resSec['RC'] = RC_dummy


def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx


tops_idx = np.zeros([intf.size])
twt_idx = np.zeros([intf.size])
tops_twt = bo2_fms['twt']/1000.#  np.array([2083,2090,2106,2133,2171,2198,2213,2226, 2244,2272,2405])/1000.

for i in range(intf.size):
    twt_idx[i] = find_nearest(np.array(resSec['DPTM']), tops_twt[i])
    tops_idx[i] = find_nearest(np.array(resSec['DEPTH']), intf[i])
    
twt_idx = np.array(twt_idx, dtype = 'int')
tops_idx = np.array(tops_idx, dtype = 'int')

#tops_twt = np.array(resSec['DPTM'])[twt_idx]
dummy_twt = np.zeros(intf.size)
dummy_twt[0] = tmin
dummy_twt[-1] = tmax
twt_idx = np.nonzero(np.array(resSec['RC']))
tops_twt = np.array(resSec['DPTM'])[twt_idx]
dummy_twt[1:-1] = tops_twt
tops_twt = dummy_twt
t_plot_twt = np.array(resSec['DPTM'])
t_plot_lc = t_plot_twt[twt_idx[0][4]:]
lc_idx = twt_idx[0][4]



#fig = plt.figure(figsize=set_size(width, subplots=(2, 2)))
#
#plt.subplot(151)
#plt.plot(resSec['ai_mod'] , resSec['DEPTH'])
#plt.ylim(md_max,md_min)
#for n in range(0,len(intf)):
#    plt.axhline(y = intf[n], color = 'b', lw = 2, linestyle = '-',
#                   alpha = 0.3)  
#for i in range(len(Fstr)-1):
#    plt.text(x = 2.35, y =intf[i]+10, s = Fstr[i]+'\n ({0} m)'.format(str(round(dz[i],1))),
#        fontsize = '10',
#        horizontalalignment = 'center',
#        verticalalignment = 'center',
#        bbox = dict(facecolor = 'white',alpha = 0.5, lw = 0.5),
#        weight = 'light',
#        rotation=0)
#plt.title('AI vs depth')
#plt.ylabel('MD [m]')    
#    
#plt.subplot(152)
#plt.plot(resSec['DPTM'],resSec['DEPTH'])
##plt.plot(t_plot_twt1/1000.,checkshot_depthrange[:-1])
#plt.ylim(md_max,md_min);
#plt.xlabel('TWT [s]')
#plt.title('TWT-Depth Curve')
##np.arange()
#for n in range(0,len(intf)):
#    plt.axhline(y = intf[n], color = 'b', lw = 2, linestyle = '-',
#                   alpha = 0.3)    
#    
#plt.subplot(153)
#plt.plot(resSec['ai_mod'] ,resSec['DPTM'])
#plt.ylim(tmax, tmin)
#plt.title('AI vs TWT')
#plt.ylabel('TWT [s]')
#for i in range (tops_twt.size):
#    plt.axhline(tops_twt[i], color='green', lw=1, alpha = 0.3)
#
#plt.subplot(154)
#plt.title('RC vs TWT')
#plt.plot(resSec['RC'] ,resSec['DPTM'])
##plt.yticklabels('')
#plt.ylim(tmax, tmin)
#for i in range (tops_twt.size):
#    plt.axhline(tops_twt[i], color='green', lw=1, alpha = 0.3)
#plt.tight_layout(pad=.1)


<a id='wavelets'></a>

# Wavelet Investigation

In [ ]:
t_plot_lc.shape

In [ ]:
 
# Wavelets (time domain)
wavelet_str = 'BO-2X_wavelets.txt'
#amplitude_str = 'wavelet_CP_stat107_freq_amp.txt'
#for i in range (len(wavelet_str)):
data = pd.read_csv(path_wvlt+wavelet_str, sep ='\t',encoding='utf-8',skiprows=0)
data = np.asarray(data, dtype=float)
wavelet_seismic = data[:,1:6:2]
wavelet_well = data[:,7:12:2]
#wavelet_rw = data[:,19:24:2]
#k = np.array([1,2,3,0]) #re-arrange array to order the angles (only for seismic wavelet)
wavelet_seismic

In [ ]:
wavelet_seismic = data[:,1:6:2]
wavelet_well = data[:,7:12:2]
# Wavelets (time domain)
#wavelet_str = 'seis_rw_fw_time_amp.txt'
#amplitude_str = 'wavelet_CP_stat107_freq_amp.txt'
#for i in range (len(wavelet_str)):
#data = pd.read_csv(path_wvlt+wavelet_str, sep ='\t',encoding='utf-8')
#data = np.asarray(data, dtype=float)

#dummy= np.zeros([data.shape[0]+1,data.shape[1]])
#dummy[1:,:] = data
#dummy[0,:] = np.array([-64.0000, -12.5062, -64.0000, -12.3317, -64.0000 , -14.2073, -64.0000,-14.0446,-64.0000,-0.0000,-64.0000,0.0002,-64.0000,0.0002,-64.0000,-0.0001,-64.0000,-0.0001,-64.0000,0.0000,-64.0000,0.0000,-64.0000,-0.0002])

#data=dummy

#wavelet_seismic = data[:,1:8:2]
#wavelet_well = data[:,9:17:2]
#wavelet_well = data[:,17:24:2]
k = np.array([0,1,2]) #re-arrange array to order the angles

# Wavelets (amplitude-frequency domain)
freq_amp_str='wavelets_freq_amp.txt'
data_amp = pd.read_csv(path_wvlt+freq_amp_str)
data_amp = np.asarray(data_amp, dtype=float)
data_amp = np.squeeze(data_amp)
data_dummy = np.zeros(data_amp.size + 1)
data_dummy[1:] = data_amp
data_amp = data_dummy
data_amp = np.reshape(data_amp,[16,65])
seis_amp = data_amp[1:8:2,:]
freq = data_amp[0,:]
well_amp = data_amp[9::2,:]
#well_freq = data_amp[8::2,:]

# Interpolations
dx=1280
dummy_well = np.zeros([dx,wavelet_well.shape[1]]) 
dummy_seismic = np.zeros([dx,wavelet_well.shape[1]]) 

dummy_seis_amp = np.zeros([dx,data_amp.shape[1]])
dummy_well_amp = np.zeros([dx,data_amp.shape[1]])


x =         np.linspace(data[0,0],data[-1,0],num=np.int(data.shape[0]))
xnew =      np.linspace(data[0,0],data[-1,0],num=np.int(dx))
wavelet_t = np.linspace(data[0,0],data[-1,0],num=xnew.size)


#x = np.linspace(-,0.1,num=data.shape[0])
#xnew = np.linspace(-0.1,0.1,num=dx)

xnew_freq = np.linspace(freq.min(),freq.max(),num=dx)


for i in range(wavelet_well.shape[1]):
    # Wavelets (time domain)
    y = wavelet_well[:,i] 
    f = interpolate.splrep(x, y,s=0)
    dummy_well[:,i] = interpolate.splev(xnew, f,der=0) #use interpolation function returned by `interp1d`
    
    y = wavelet_seismic[:,i] 
    f = interpolate.splrep(x, y,s=0)
    dummy_seismic[:,k[i]] = interpolate.splev(xnew, f,der=0) #use interpolation function returned by `interp1d`
    
    # Wavelets (amplitude-frequency domain)
    y = seis_amp[i,:] 
    f = interpolate.splrep(freq, y,s=0)
    dummy_seis_amp[:,k[i]] = interpolate.splev(xnew_freq, f,der=0) #use interpolation function returned by `interp1d`
    
    y = well_amp[i,:] 
    f = interpolate.splrep(freq, y,s=0)
    dummy_well_amp[:,i] = interpolate.splev(xnew_freq, f,der=0) #use interpolation function returned by `interp1d`
    
    
    
wavelet_well = dummy_well
wavelet_seismic = dummy_seismic


seis_amp = dummy_seis_amp
well_amp = dummy_well_amp

wavelet_well =  wavelet_well/np.abs(wavelet_well).max()
wavelet_seismic =  wavelet_seismic/np.abs(wavelet_seismic).max()

seis_amp = seis_amp/np.abs(seis_amp).max()
well_amp = well_amp/np.abs(well_amp).max()

#fig = plt.figure(figsize=(14, 10))


fig = plt.figure(figsize=set_size(width, subplots=(4,5)))

fig.tight_layout
gs1 = gridspec.GridSpec(nrows=7, ncols=6, figure=fig,hspace =1 ,wspace=3)
ax0 = fig.add_subplot(gs1[0:3, 0:3])
ax1 = fig.add_subplot(gs1[0:3, 3:6])
ax2 = fig.add_subplot(gs1[4:7, 0:3])
ax3 = fig.add_subplot(gs1[4:7, 3:6])




wavelet_well.shape
for i in range(wavelet_well.shape[1]):
    ax0.plot(xnew,wavelet_well[:,i],lw=1)
    ax2.plot(xnew,wavelet_seismic[:,i],lw=1)
    
    ax1.plot(xnew_freq,well_amp[:,i], lw=1)
    ax3.plot(xnew_freq,seis_amp[:,i],lw=1)
ax1.legend( ['10$^\circ$','20$^\circ$','30$^\circ$'])
ax0.grid(), ax1.grid(), ax2.grid(), ax3.grid();
ax1.set_ylim(0,1.1), ax3.set_ylim(0,1.1);
ax1.set_xlim(0,xnew_freq.max()), ax3.set_xlim(0,xnew_freq.max());
ax0.set_xlim(data[0,0],data[-1,0]), ax2.set_xlim(data[0,0],data[-1,0])

#ax0.set_xticks()

#ax0.set_title('Well Wavelet')
#ax2.set_title('Seismic Wavelet')

ax0.set_ylabel('Amplitude')
ax0.set_xlabel('Time [ms]')
ax2.set_xlabel('Time [ms]')
ax2.set_ylabel('Amplitude')

ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel('Amplitude')
ax3.set_xlabel('Frequency [Hz]')
ax3.set_ylabel('Amplitude');


plt.figtext(0.5,0.95, "Well Wavelets", ha="center", va="top", color="k")
plt.figtext(0.5,0.48, "Seismic Wavelets", ha="center", va="top", color="k")
#plt.subplots_adjust(hspace = 0.4 )

if savefig == True:
    plt.savefig(fig_path+'wavelets_well_stat.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')
plt.show()


In [ ]:

fig = plt.figure(figsize=set_size(width, subplots=(4,5)))

fig.tight_layout
gs1 = gridspec.GridSpec(nrows=7, ncols=6, figure=fig,hspace =1 ,wspace=3)
ax0 = fig.add_subplot(gs1[0:3, 0:3])
#ax1 = 
ax2 = fig.add_subplot(gs1[0:3, 3:6])
#ax3 = fig.add_subplot(gs1[4:7, 3:6])




wavelet_well.shape
for i in range(wavelet_well.shape[1]):
    ax0.plot(xnew,wavelet_well[:,i],lw=2,alpha=0.5)
    ax2.plot(xnew,wavelet_seismic[:,i],lw=2,alpha=0.5)
    
    #ax1.plot(xnew_freq,well_amp[:,i], lw=1)
    #ax3.plot(xnew_freq,seis_amp[:,i],lw=1)
ax2.legend( ['10$^\circ$','20$^\circ$','30$^\circ$'],bbox_to_anchor=(1, 0., 0.5, 1))
ax0.grid(), ax1.grid(), ax2.grid(), ax3.grid();
ax0.set_ylim(-1,.5), ax2.set_ylim(-1,.5);
#ax1.set_xlim(0,xnew_freq.max()), ax3.set_xlim(0,xnew_freq.max());
ax0.set_xlim(data[0,0],data[-1,0]), ax2.set_xlim(data[0,0],data[-1,0])

#ax0.set_xticks()

#ax0.set_title('Well Wavelet')
#ax2.set_title('Seismic Wavelet')

ax0.set_ylabel('Amplitude')
ax0.set_xlabel('Time [ms]')
ax2.set_xlabel('Time [ms]')
#ax2.set_ylabel('Amplitude')

#ax1.set_xlabel('Frequency [Hz]')
#ax1.set_ylabel('Amplitude')
#ax3.set_xlabel('Frequency [Hz]')
#ax3.set_ylabel('Amplitude');


ax0.set_title("Well Wavelets", ha="center", va="top", color="k")
ax2.set_title("Seismic Wavelets", ha="center", va="top", color="k")
#plt.subplots_adjust(hspace = 0.4 )

if savefig == True:
    plt.savefig(fig_path+'wavelets_well_stat.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')
plt.show()


## Comparison with Total's wavelets

In [ ]:
#
#
#
#wavelet_total1 = pd.read_csv(path_wavelets+'W6_A_FIN_wavelet_0_0_deg_awos.wvl',skiprows=0)
#wavelet_total1 = np.squeeze(np.array(wavelet_total1))
##print('shape=',wavelet_total1.shape)
#
#wavelet_total2 = pd.read_csv(path_wavelets+'MultiWellV5_ExtendedJude1_used4inversion_wavelet_0_0_deg_awos.wvl',skiprows=0)
#wavelet_total2 = np.squeeze(np.array(wavelet_total2))
#
#print(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2]*20)
#x =         np.linspace(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],num=np.int(wavelet_total1[0]))
#xnew =      np.linspace(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],num=np.int(wavelet_total1[1]*wavelet_total1[2]*20))
#wavelet_t = np.linspace(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],num=xnew.size)
#y = wavelet_total1[3:]  # 0 = 6-12, 1 = 12-18, 2=18-24, 3=24-30 degrees 
#
#
#f = interpolate.splrep(x, y,s=0)
#wavelet_total1 = interpolate.splev(xnew, f,der=0) #use interpolation function returned by `interp1d`
#
#
##x = np.linspace(-0.1,0.1,num=wavelet_total2.shape[0])
##xnew = np.linspace(-0.1,0.1,num=2000)
#y = wavelet_total2[3:]  # 0 = 6-12, 1 = 12-18, 2=18-24, 3=24-30 degrees 
#f = interpolate.splrep(x, y,s=0)
#wavelet_total2 = interpolate.splev(xnew, f,der=0) #use interpolation function returned by `interp1d`
#
#
#wavelet_total1 =  wavelet_total1/np.abs(wavelet_total1).max()
#wavelet_total2 =  wavelet_total2/np.abs(wavelet_total2).max()
#plt.figure()
#plt.title('Total Wavelets')
#plt.plot(wavelet_t,wavelet_total1,lw=3, label = 'W6 A FIN wavelet')
#plt.plot(wavelet_t,wavelet_total2,lw=3, label = 'MultiWellV5 ExtendedJude1')
#plt.grid()
#plt.ylabel('Amplitude')
#plt.xlabel('time [ms]')
#plt.legend()
#if savefig == True:
#    plt.savefig(fig_path+'wavelets_total.pdf',format='pdf', bbox_inches='tight')
#    print('FIGURE SAVED')
#plt.show()

In [ ]:
path_wavelets_total = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/wavelets/'
wavelet_total1 = pd.read_csv(path_wavelets_total+'W6_A_FIN_wavelet_0_0_deg_awos.wvl',skiprows=0)
wavelet_total1 = np.squeeze(np.array(wavelet_total1))
#print('shape=',wavelet_total1.shape)

wavelet_total2 = pd.read_csv(path_wavelets_total+'MultiWellV5_ExtendedJude1_used4inversion_wavelet_0_0_deg_awos.wvl',skiprows=0)
wavelet_total2 = np.squeeze(np.array(wavelet_total2))


wavelet_kenneth = pd.read_csv(path_wvlt+'BO-2X_wavelets.txt',skiprows=0, header=None, sep='\t')
wavelet_kenneth = np.array(wavelet_kenneth)[:,1]


print(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2]*20)
x =         np.linspace(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],num=np.int(wavelet_total1[0]))
xnew =      np.linspace(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],num=np.int(wavelet_total1[1]*wavelet_total1[2]*20))
wavelet_t = np.linspace(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],num=xnew.size)
y = wavelet_total1[3:]  # 0 = 6-12, 1 = 12-18, 2=18-24, 3=24-30 degrees 


f = interpolate.splrep(x, y,s=0)
wavelet_total1 = interpolate.splev(xnew, f,der=0) #use interpolation function returned by `interp1d`


#x = np.linspace(-0.1,0.1,num=wavelet_total2.shape[0])
#xnew = np.linspace(-0.1,0.1,num=2000)
y = wavelet_total2[3:]  # 0 = 6-12, 1 = 12-18, 2=18-24, 3=24-30 degrees 
f = interpolate.splrep(x, y,s=0)
wavelet_total2 = interpolate.splev(xnew, f,der=0) #use interpolation function returned by `interp1d`


y = wavelet_kenneth
f = interpolate.splrep(x, y,s=0)
wavelet_kenneth = interpolate.splev(xnew, f,der=0) #use interpolation function returned by `interp1d`


wavelet_total1 =  wavelet_total1/np.abs(wavelet_total1).max()
wavelet_total2 =  wavelet_total2/np.abs(wavelet_total2).max()
wavelet_kenneth = wavelet_kenneth/np.abs(wavelet_kenneth).max()

plt.figure()
plt.title('Wavelets')
plt.plot(wavelet_t,wavelet_total1,lw=3, label = 'W6 A FIN wavelet')
plt.plot(wavelet_t,wavelet_total2,lw=3, label = 'MultiWellV5 ExtendedJude1')
plt.plot(wavelet_t,wavelet_kenneth,lw=3, label = 'BO-2X (HRS)')
plt.grid()
plt.ylabel('Amplitude')
plt.xlabel('Time [ms]')
plt.legend(loc=3)
if savefig == True:
    plt.savefig(fig_path+'wavelets_total_HRS.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')
plt.show()



<a id='blockymodel'></a>
# Blocky Model

In [ ]:
#   Some handy constants
nlayers = len(vp_mod)
#nint = nlayers - 1
nangles = 35
dt =  0.0001 #np.diff(t_plot_twt)[0]#


#   Calculate reflectivities from model parameters

rc_zoep_pp = []
theta1 = []
dummy_rc = np.zeros([9,4,4])

for i in range(0,nangles):
    theta1_buf = i*theta1_step + theta1_min
    for i in range (vp_mod.size-1):
        dummy_rc[i,:,:] = mlo.rc_zoep(vp_mod[i], vs_mod[i], rho_mod[i], vp_mod[i+1], vs_mod[i+1], rho_mod[i+1], theta1_buf)
   
    theta1.append(theta1_buf)
    rc_zoep_pp.append((dummy_rc[0,0,0],dummy_rc[1,0,0],dummy_rc[2,0,0],
                      dummy_rc[3,0,0],dummy_rc[4,0,0],dummy_rc[5,0,0],
                      dummy_rc[6,0,0],dummy_rc[7,0,0],dummy_rc[8,0,0]))
    
rc_zoep_pp = np.array([rc_zoep_pp])

#rc_zoep_pp = np.array([rc_zoep_pp])
stack1 = np.squeeze(  np.sum(rc_zoep_pp[:,5:15,:],axis=1) / rc_zoep_pp[:,5:15,:].shape[2])
stack2 = np.squeeze(np.sum(rc_zoep_pp[:,15:25,:],axis=1) / rc_zoep_pp[:,15:25,:].shape[2])
stack3 = np.squeeze(np.sum(rc_zoep_pp[:,25:35,:],axis=1) / rc_zoep_pp[:,25:35,:].shape[2])
#stack4 = np.squeeze(np.sum(rc_zoep_pp[:,24:30,:],axis=1) / rc_zoep_pp[:,24:30,:].shape[2])



rc_zoep_pp = np.array([stack1,stack2,stack3])


    
syn_zoep_pp_well = [] 
syn_zoep_pp_seis = []
angle_dummy = np.array([5,15,25])
j=0
for angle in range(5, nangles,10):
        nlayers = len(rc_zoep_pp[j])
        nint = nlayers
        nsamp = len(t_plot_twt)
        rc = list(np.zeros(nsamp,dtype='float'))
        lyr = 0

        for i in range(0, nsamp):
            for k in range(0,tops_twt.size-1):
                if t_plot_twt[i] == tops_twt[k]:
                    rc[i] = rc_zoep_pp[j][k-1]
                if lyr > nint:
                    break
        #k=0
        #   Convolve wavelet with reflectivities
        
        syn_buf = np.convolve(rc, wavelet_well[:,j], mode='same')
        syn_buf = list(syn_buf)
        syn_zoep_pp_well.append(syn_buf)
        
        syn_buf_seis = np.convolve(rc, wavelet_seismic[:,j], mode='same')
        syn_buf_seis = list(syn_buf_seis)
        syn_zoep_pp_seis.append(syn_buf_seis)
        j=j+1
        #print("Calculated angle %i" % (angle))

#    Convert data arrays from lists/tuples to numpy arrays    
syn_zoep_pp_well = np.array(syn_zoep_pp_well)
syn_zoep_pp_seis = np.array(syn_zoep_pp_seis)
rc_zoep_pp = np.array(rc_zoep_pp)

#   Copy convolved top/base reflectivity values to Lists for easier plotting
[ntrc, nsamp] = syn_zoep_pp_well.shape
line1 = []
line2 = []
line3 = []
line4 = []
line5 = []
line6 = []
line7 = []
line8 = []
line9 = []
line10 = []

Yzoep = np.array(rc_zoep_pp[:,0])
Yzoep = Yzoep.reshape((ntrc, 1))

#Yconv = np.array(line1)
#Yconv = Yconv.reshape((ntrc, 1))
#
#ones = np.ones(ntrc)
#ones = ones.reshape((ntrc,1))

sintheta2 = np.sin(np.radians(np.arange(0, ntrc)))**2
sintheta2 = sintheta2.reshape((ntrc, 1))


tops_lines = np.zeros([tops_idx.size , syn_zoep_pp_well.shape[0]])    


for i in range(0, ntrc):
    line1.append(syn_zoep_pp_well[i,np.int(tops_idx[0])])
    line2.append(syn_zoep_pp_well[i,np.int(tops_idx[1])])
    line3.append(syn_zoep_pp_well[i,np.int(tops_idx[2])])
    line4.append(syn_zoep_pp_well[i,np.int(tops_idx[3])])
    line5.append(syn_zoep_pp_well[i,np.int(tops_idx[4])])
    line6.append(syn_zoep_pp_well[i,np.int(tops_idx[5])])
    line7.append(syn_zoep_pp_well[i,np.int(tops_idx[6])])
    line8.append(syn_zoep_pp_well[i,np.int(tops_idx[7])])
    line9.append(syn_zoep_pp_well[i,np.int(tops_idx[8])])     
    line10.append(syn_zoep_pp_well[i,np.int(tops_idx[9])])
    
line1=np.squeeze(np.array([line1]))
line2=np.squeeze(np.array([line2]))
line3=np.squeeze(np.array([line3]))
line4=np.squeeze(np.array([line4]))
line5=np.squeeze(np.array([line5]))
line6=np.squeeze(np.array([line6]))
line7=np.squeeze(np.array([line7]))
line8=np.squeeze(np.array([line8]))
line9=np.squeeze(np.array([line9]))
line10=np.squeeze(np.array([line10]))

In [ ]:
resSec['phi_mod'][lc_idx:]

In [ ]:
#fig, ax  = plt.subplots(1, 6, figsize=set_size(width, subplots=(3,2)))
fig = plt.figure(figsize=set_size(width, subplots=(5,2)))
#fig, ax = plt.subplots(10, 8, figsize=set_size(width, subplots=(10, 8)))
#fig.tight_layout(pad=2)
fig.tight_layout
gs1 = gridspec.GridSpec(nrows=12, ncols=6, figure=fig,hspace =0 ,wspace=0.25)#,width_ratios=np.array([.5,.5,.5,.5,.5,.5,.5]))
ax0 = fig.add_subplot(gs1[0:7, 0])
ax1 = fig.add_subplot(gs1[0:7, 1])
ax2 = fig.add_subplot(gs1[0:7, 2])
ax3 = fig.add_subplot(gs1[0:7, 3])
ax4 = fig.add_subplot(gs1[0:7, 4])

#ax5 = fig.add_subplot(gs1[8:12, 1:6])

#ax6 = fig.add_subplot(gs1[0:7, 1])
#
#ax6.set_ylim(t_plot_lc.min(),t_plot_lc.max())
#ax6.invert_yaxis()
#for pos, spine in ax0.axes.spines.items():
#    spine.set_visible(False)
#ax6.set_xticks([])
#ax6.set_ylabel(r'TWT [s]')
#ax6.axhline(tops_twt[6], color = 'red',     lw=2,   alpha=0.8)#, label = 'Top Sola')
#ax6.axhline(tops_twt[7], color = 'green',   lw=2,   alpha=0.8)#, label = 'Top U. Tuxen')
#ax6.axhline(tops_twt[8], color = 'orange',  lw=2,   alpha=0.8)#, label = 'Top L. Tuxen')
##ax6.axhline(tops_twt[9], color = 'blue',  lw=1,   alpha=0.3)#, label = 'Top Valhall')   
#    
#    #thicnkess in ms
#ds = np.round(np.diff(tops_twt)*1000.,1)
#
#for i in range(5,len(Fstr)-1):
#    if i == 0:
#        ax6.text(x = .5, y =tops_twt[i]+0.004, s = Fstr[i]+'\n ({0} ms)'.format(str(ds[i])),
#            fontsize = 5,
#            horizontalalignment = 'center',
#            verticalalignment = 'center',
#            #bbox = dict(facecolor = 'white', lw = 0.5),
#            weight = 'light',
#            rotation=0)
#    else:
#        ax6.text(x = .5, y =tops_twt[i]+0.007, s = Fstr[i]+'\n ({0} ms)'.format(str(ds[i])),
#            fontsize = 6,
#            horizontalalignment = 'center',
#            verticalalignment = 'center',
#            #bbox = dict(facecolor = 'white', lw = 0.5),
#            weight = 'light',
#            rotation=0)
  
#ax8 = fig.add_subplot(1,14,5)

ax1.plot(resSec['phi_mod'][lc_idx:] ,t_plot_lc,'k' ,lw=1)
ax1.set_ylim(t_plot_lc.max(), t_plot_lc.min())
ax1.set_xlabel('POR ' + r'$(v/v)$')
#ax1.xaxis.set_label_position('bottom')

#ax1.yaxis.set_label_coords(-0.37, .5)


x1 = resSec['phi_mod'][lc_idx:]*(1-resSec['sw_mod'][lc_idx:])
x2 = resSec['phi_mod'][lc_idx:]*resSec['sw_mod'][lc_idx:]


ax1.fill_betweenx(t_plot_lc,x2,0, alpha=0.3,color='b',label='Water')
ax1.fill_betweenx(t_plot_lc,x2,resSec['phi_mod'][lc_idx:], alpha=0.3,color='r',label='Oil')
#ax1.legend(loc=0)
ax1.set_xlim(0,0.5)
ax1.set_ylabel(r'TWT [s]')
ax1.legend(loc=0,fontsize=6)


#ax2 = ax1.twiny()  # instantiate a second axes that shares the same y-axis
ax2.plot(resSec['lit_mod'][lc_idx:], t_plot_lc,'k',lw=1)
ax2.set_ylim(t_plot_lc.max(), t_plot_lc.min())
ax2.set_xlabel(r'$SHV \ (v/v)$')
ax2.set_xlim(0,0.3)
ax2.set_yticklabels('')
ax2.fill_betweenx(t_plot_lc,0,resSec['lit_mod'][lc_idx:], color='k', alpha=0.3)



#ax2.set_title('AI')
ax3.plot(resSec['ai_mod'][lc_idx:]/1000 ,t_plot_lc,'k' ,lw=1)
ax3.set_xlabel('AI \n'+r' $(km/s \times g/cm^3)$',  )
ax3.set_ylim(t_plot_lc.max(), t_plot_lc.min())
ax3.set_yticklabels('')


#ax3.set_title('Vp/Vs')
ax4.plot(resSec['vp_mod'][lc_idx:]/resSec['vs_mod'][lc_idx:], t_plot_lc,'k' ,lw=1)
ax4.set_ylim(t_plot_lc.max(), t_plot_lc.min())
ax4.set_xlabel(r'$V_p/V_s$')
ax4.set_xticks(np.array([1.7,2]));
ax4.set_xticklabels(['1.7','2.0']);
ax4.set_yticklabels('')


for i in range (tops_twt.size-4):
    ax1.axhline(tops_twt[i],   lw=1, alpha = 0.3) 
    ax2.axhline(tops_twt[i],   lw=1, alpha = 0.3)  
    ax3.axhline(tops_twt[i],   lw=1, alpha = 0.3) 
    ax4.axhline(tops_twt[i],   lw=1, alpha = 0.3)  
    ax6.axhline(tops_twt[i],   lw=1, alpha = 0.3)                 

                
ax1.axhline(tops_twt[6], color = 'red', lw=2,   alpha=0.8)
ax1.axhline(tops_twt[7], color = 'green'  , lw=2,   alpha=0.8)
ax1.axhline(tops_twt[8], color = 'orange',     lw=2,   alpha=0.8)
#ax1.axhline(tops_twt[9], color = 'orange',  lw=2,   alpha=0.8)
ax2.axhline(tops_twt[6], color = 'red', lw=2,   alpha=0.8)
ax2.axhline(tops_twt[7], color = 'green',   lw=2,   alpha=0.8)
ax2.axhline(tops_twt[8], color = 'orange',     lw=2,   alpha=0.8)
#ax2.axhline(tops_twt[9], color = 'orange',  lw=2,   alpha=0.8)
ax3.axhline(tops_twt[6], color = 'red', lw=2,   alpha=0.8)
ax3.axhline(tops_twt[7], color = 'green',   lw=2,   alpha=0.8)
ax3.axhline(tops_twt[8], color = 'orange',     lw=2,   alpha=0.8)
#ax3.axhline(tops_twt[9], color = 'blue',  lw=2,   alpha=0.8)
ax4.axhline(tops_twt[6], color = 'red', lw=2,   alpha=0.8)
ax4.axhline(tops_twt[7], color = 'green',   lw=2,   alpha=0.8)
ax4.axhline(tops_twt[8], color = 'orange',     lw=2,   alpha=0.8)
#ax4.axhline(tops_twt[9], color = 'blue',  lw=2,   alpha=0.8)


ax0.set_ylim(np.min(t_plot_lc),np.max(t_plot_lc))
ax0.invert_yaxis()
for pos, spine in ax0.axes.spines.items():
    spine.set_visible(False)
ax0.set_xticks([])
ax0.set_yticks([])

x_offset = 0.2
#ax0.annotate('',xy=(x_offset,tops_twt[6]),xytext=(x_offset, np.min(t_plot_lc)),
#            arrowprops=dict(facecolor='black',arrowstyle='<->', linewidth=1),)

#ax0.annotate('',xy=(x_offset,np.max(resSec['DPTM'])),xytext=(x_offset, tops_twt[6]),
#            arrowprops=dict(facecolor='black',arrowstyle='<->', linewidth=1),)
#
##ax0.text(x = x_offset-0.2 , y = (np.min(t_plot_lc) + tops_twt[6]) / 2 + 0.015,  s = 'Upper Chalk', weight='normal', rotation = -270)
#ax0.text(x =x_offset-0.3, y =  (np.max(resSec['DPTM']) + tops_twt[6]) / 2 ,  s = 'LC', weight='normal', rotation = -270)
                       
ax6.text(x = ax6.get_xlim()[0]-0.2, y = ax6.get_ylim()[1]-0.0025,  s = 'a)', weight='normal')
ax1.text(x = ax1.get_xlim()[0]-0.1, y = ax1.get_ylim()[1]-0.0025,  s = 'b)', weight='normal')
ax2.text(x = ax2.get_xlim()[0]-0.04, y = ax2.get_ylim()[1]-0.0025,  s = 'c)', weight='normal')
ax3.text(x = ax3.get_xlim()[0]-1.2, y =  ax3.get_ylim()[1]-0.0025,  s = 'd)', weight='normal')
ax4.text(x = ax4.get_xlim()[0]-0.07, y =  ax3.get_ylim()[1]-0.0025,  s = 'e)', weight='normal')

#ax4.text(x = ax4.get_xlim()[1]+0.05, y = tops_twt[6]+0.001,  s = 'Top Sola', weight='normal', color = 'red')
#ax4.text(x = ax4.get_xlim()[1]+0.05, y = tops_twt[7]+0.001,  s = 'Top Tuxen', weight='normal', color = 'green')
#ax4.text(x = ax4.get_xlim()[1]+0.05, y = tops_twt[8]+0.001,  s = 'Top Valhall', weight='normal', color = 'orange')                                                

if savefig == True:
    plt.savefig(fig_path+'BO2X_block.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')
syn_zoep_cdp = syn_zoep_pp_well

In [ ]:
syn_zoep_cdp_seis = syn_zoep_pp_seis#np.zeros([len(offsets),syn_zoep_pp_seis.shape[1]])
syn_zoep_cdp_well = syn_zoep_pp_well #np.zeros([len(offsets),syn_zoep_pp_well.shape[1]])
print(t_plot_twt.shape , syn_zoep_cdp_seis.shape)
path_hrs


In [ ]:
il_bo2 = 8366
xl_bo2 = 7926
file = 'valdemar2016_5-35deg.sgy'

with segyio.open(path_hrs + file, iline=189) as f: # iline=189 , xline=189
    offsets = f.offsets
    gather_bo2 = np.asarray([np.copy(x) for x in f.iline[il_bo2,:]])

gather_bo2 = gather_bo2[:,np.argwhere(f.xlines==xl_bo2)[0][0],:]




# Trimming traces to reservoir zone:
time = np.arange(1.5,3.004,0.004)
t_start = tmin
index_start = (np.abs(time-t_start)).argmin()
t_stop = tmax
index_stop = (np.abs(time-t_stop)).argmin()

#print(gather_bo2.shape, time.shape)

gather_bo2 = gather_bo2[:,index_start:index_stop].T

print(gather_bo2.shape, index_start, index_stop, tmin, tmax)
#gather_exp = gather_exp[:,index_start:index_stop]

In [ ]:
#wavelet_total2 = pd.read_csv(path_wavelets+'MultiWellV5_ExtendedJude1_used4inversion_wavelet_0_0_deg_awos.wvl',skiprows=0)
#wavelet_total2 = np.squeeze(np.array(wavelet_total2))

#print(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2]*20)
x =         np.linspace(0,1,gather_bo2.shape[0])
xnew =      np.linspace(0,1,t_plot_twt.shape[0])#linspace(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],num=np.int(wavelet_total1[1]*wavelet_total1[2]*20))
#wavelet_t = np.linspace(-wavelet_total1[1]*wavelet_total1[2],wavelet_total1[1]*wavelet_total1[2],num=xnew.size)

gather_sm_bo2 = np.zeros([xnew.shape[0], gather_bo2.shape[1]])

for i in range (gather_sm_bo2.shape[1]):
    y = gather_bo2[:,i]#wavelet_total1[3:]  # 0 = 6-12, 1 = 12-18, 2=18-24, 3=24-30 degrees 
    f = interpolate.splrep(x, y,s=0)
    gather_sm_bo2[:,i] = interpolate.splev(xnew, f,der=0) #use interpolation function returned by `interp1d`
#gather_sm_bo2.shape

In [ ]:
#time = np.linspace(t_start,t_stop,gather_bo2.shape[0])
#time_sm_corr = np.linspace(time.min(), time.max(), syn_zoep_cdp_seis.shape[1])
#time_sm = np.linspace(tmin, tmax, len(time)*10)
#
#
#gather_sm_bo2 = np.zeros([gather_bo2.shape[1],syn_zoep_cdp_seis.shape[1]])
#
#
#print()
#
#for i in range(0,gather_bo2.shape[0]):
#        gather_sm_bo2[:,i] = make_interp_spline(time,gather_bo2[i,:])(time_sm_corr)
       
########

import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

# Install click and bruges with pip.
import click
import bruges as bg


def make_synthetic(size=256, top=0.4, base=0.6, value=1, freq=25, phase=0):
    """Make a synthetic. Return the wavelet, the model, the RC, and the synthetic.
    """
    v = np.ones(size) - value

    v[int(top*size):int(base*size)] = value
    rc = np.diff(v)

    w = bg.filters.ricker(0.256, 0.001, freq)

    if phase != 0:
        w = bg.filters.rotate_phase(w, phase, degrees=True)

    syn = np.convolve(rc, w, mode='same')
    return w, v, rc, syn

def get_colour(cmap, frac):
    """
    Decide whether to make white or black labels.
    """
    cmap = get_cmap(cmap)
    return 'k' if (np.mean(cmap(frac)[:3]) > 0.5) else 'w'

<a id='AVOmodelling'></a>
# Synthetic vs Observed AVO Seismic

In [ ]:
fig = plt.figure(figsize=set_size(width, subplots=(5,2)))
fig.tight_layout
gs1 = gridspec.GridSpec(nrows=12, ncols=6, figure=fig,hspace =1 ,wspace=0.25)#,width_ratios=np.array([.5,.5,.5,.5,.5,.5,.5]))

ax0 = fig.add_subplot(gs1[0:7, 0])
ax0.set_ylim(np.min(t_plot_twt),np.max(t_plot_twt))
ax0.invert_yaxis()
for pos, spine in ax0.axes.spines.items():
    spine.set_visible(False)
ax0.set_xticks([])
ax0.set_yticks([])

x_offset = 0.2
ax0.annotate('',xy=(x_offset,tops_twt[6]),xytext=(x_offset, np.min(resSec['DPTM'])),
            arrowprops=dict(facecolor='black',arrowstyle='<->', linewidth=1),)

ax0.annotate('',xy=(x_offset,np.max(resSec['DPTM'])),xytext=(x_offset, tops_twt[6]),
            arrowprops=dict(facecolor='black',arrowstyle='<->', linewidth=1),)

ax0.text(x = x_offset-0.2 , y = (np.min(resSec['DPTM']) + tops_twt[6]) / 2 + 0.015,  s = 'Upper Chalk', weight='normal', rotation = -270)
ax0.text(x =x_offset-0.2, y =  (np.max(resSec['DPTM']) + tops_twt[6]) / 2 ,  s = 'LC', weight='normal', rotation = -270)




ax6 = fig.add_subplot(gs1[0:7, 1])
ax6.set_ylim(t_plot_lc.min(),t_plot_lc.max())
ax6.invert_yaxis()
for pos, spine in ax0.axes.spines.items():
    spine.set_visible(False)
ax6.set_xticks([])
ax6.set_ylabel(r'TWT [s]')
ax6.axhline(tops_twt[6], color = 'green', lw=1,   alpha=0.3)#, label = 'Top Sola')
ax6.axhline(tops_twt[7], color = 'red',   lw=1,   alpha=0.3)#, label = 'Top U. Tuxen')
#ax6.axhline(tops_twt[8], color = 'm',     lw=1,   alpha=0.3)#, label = 'Top L. Tuxen')
ax6.axhline(tops_twt[8], color = 'blue',  lw=1,   alpha=0.3)#, label = 'Top Valhall')   
    
    #thicnkess in ms
ds = np.round(np.diff(tops_twt)*1000.,1)

for i in range(5,len(Fstr)-1):
    if i == 0:
        ax6.text(x = .5, y =tops_twt[i]+0.004, s = Fstr[i]+'\n ({0} ms)'.format(str(ds[i])),
            fontsize = 10,
            horizontalalignment = 'center',
            verticalalignment = 'center',
            #bbox = dict(facecolor = 'white', lw = 0.5),
            weight = 'light',
            rotation=0)
    else:
        ax6.text(x = .5, y =tops_twt[i]+0.007, s = Fstr[i]+'\n ({0} ms)'.format(str(ds[i])),
            fontsize = 10,
            horizontalalignment = 'center',
            verticalalignment = 'center',
            #bbox = dict(facecolor = 'white', lw = 0.5),
            weight = 'light',
            rotation=0)

#ax0 = plt.subplot2grid((4,4), (0,0), colspan=1, rowspan=3) 
ax1 = fig.add_subplot(gs1[0:7, 2])
#ax1.set_title('RC (ZO)')
ax1.plot(rc[lc_idx:] ,t_plot_lc,'k' ,lw=1)
ax1.set_ylim(t_plot_lc.min(), t_plot_lc.max())
ax1.invert_yaxis()
ax1.set_yticklabels(['']);
ax1.set_xlim(-.2,.2)
ax1.axhline(tops_twt[6], color = 'green', lw=1,alpha = 0.3, label = 'Top Sola')
ax1.axhline(tops_twt[7], color = 'red',   lw=1,alpha = 0.3, label = 'Upper Tuxen')
#ax1.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
ax1.axhline(tops_twt[8], color = 'blue',  lw=1,alpha = 0.3, label = 'Top Valhall')


ax2 = fig.add_subplot(gs1[0:7, 3])


mlo.plot_vawig(ax2, gather_sm_bo2.T[:,lc_idx:]/1e3, t_plot_lc, 1)
ax2.set_ylim(t_plot_lc.min(),t_plot_lc.max())
ax2.axhline(tops_twt[6], color = 'green', lw=1,alpha = 0.3, label = 'Top Sola')
ax2.axhline(tops_twt[7], color = 'red',   lw=1,alpha = 0.3, label = 'Upper Tuxen')
#ax2.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
ax2.axhline(tops_twt[8], color = 'blue',  lw=1,alpha = 0.3, label = 'Top Valhall')
ax2.invert_yaxis()
#ax2.set_title('Observed CMP')
ax2.xaxis.tick_bottom()
ax2.set_yticklabels('')
ax2.set_xlim(-1,3)
ax2.set_xticks(np.array([0,1,2]));
#ax2.set_xticklabels(['']);


ax3 = fig.add_subplot(gs1[0:7, 4])

mlo.plot_vawig(ax3, syn_zoep_cdp_well[:,lc_idx:] , t_plot_lc, 1)
ax3.set_ylim(t_plot_lc.min(),t_plot_lc.max())
ax3.axhline(tops_twt[6], color = 'green', lw=1,alpha = 0.3, label = 'Top Sola')
ax3.axhline(tops_twt[7], color = 'red',   lw=1,alpha = 0.3, label = 'Upper Tuxen')
#ax3.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
ax3.axhline(tops_twt[8], color = 'blue',  lw=1,alpha = 0.3, label = 'Top Valhall')
ax3.invert_yaxis()
#ax3.set_title('Well')
ax3.xaxis.tick_bottom()
ax3.xaxis.set_label_position('bottom')
ax3.set_yticklabels('')
ax3.set_xlim(-1,3)
ax3.set_xticks(np.array([0,1,2]));
#ax3.set_xticklabels(['']);

ax4 = fig.add_subplot(gs1[0:7, 5])

mlo.plot_vawig(ax4, syn_zoep_cdp_seis[:,lc_idx:] , t_plot_lc, 1)

ax4.set_ylim(t_plot_lc.min(),t_plot_lc.max())
ax4.axhline(tops_twt[6], color = 'green', lw=1,alpha = 0.3, label = 'Top Sola')
ax4.axhline(tops_twt[7], color = 'red',   lw=1,alpha = 0.3, label = 'Upper Tuxen')
#ax4.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
ax4.axhline(tops_twt[8], color = 'blue',  lw=1,alpha = 0.3, label = 'Top Valhall')
ax4.invert_yaxis()
ax4.set_yticklabels('')
ax4.text(x = ax3.get_xlim()[1] +.5 , y = tops_twt[6]+0.002,  s = 'Top Sola', weight='normal', color = 'green')
ax4.text(x = ax3.get_xlim()[1] +.5 , y = tops_twt[7]+0.002,  s = 'Top U. Tuxen', weight='normal', color = 'red')
#ax4.text(x = ax3.get_xlim()[1] +.5 , y = tops_twt[8]+0.002,  s = 'Top L. Tuxen (MM)', weight='normal', color = 'm')
ax4.text(x = ax3.get_xlim()[1] +.5 , y = tops_twt[8]+0.002,  s = 'Top Valhall', weight='normal', color = 'blue')   
ax4.set_xlim(-1,3)
ax4.set_xticks(np.array([0,1,2]));
#ax4.set_xticklabels(['']);

#ax3.set_xlabel('Offset (deg)', fontsize = 14)
#ax4.set_title('Seismic' ,weight = 'bold')
ax4.xaxis.tick_bottom()
ax4.xaxis.set_label_position('bottom')
#ax3.legend(bbox_to_anchor=(.85, 0.6))


for i in range (tops_twt.size-4):
    ax1.axhline(tops_twt[i],   lw=1, alpha = 0.3)
    ax2.axhline(tops_twt[i],   lw=1, alpha = 0.3)
    ax3.axhline(tops_twt[i],   lw=1, alpha = 0.3)
    ax4.axhline(tops_twt[i],   lw=1, alpha = 0.3)
    ax6.axhline(tops_twt[i],   lw=1, alpha = 0.3)
ax6.text(x = ax6.get_xlim()[0]-0.2, y = ax6.get_ylim()[1]-0.0025,  s = 'a)', weight='normal')
offset_range = range(-1,2)#np.linspace(offsets.min(), offsets.max(),len(offsets),dtype='int')
scaling = 0.0001


ax5 = fig.add_subplot(gs1[7:9, 3])
ax6 = fig.add_subplot(gs1[7:9, 4])
ax7 = fig.add_subplot(gs1[7:9, 5])
#ax5 = plt.subplot2grid((4,4), (3,1))
#ax6 = plt.subplot2grid((4,4), (3,2))
#ax7 = plt.subplot2grid((4,4), (3,3))
marker_size = 10

#ax4 = fig.add_subplot(gs[-1,0])
ax5.plot(offsets[offset_range],scaling*gather_sm_bo2[(np.argmin(np.abs(t_plot_twt -(t_plot_twt[tops_idx[6]])))),offset_range],'g.',ms=marker_size, alpha = 0.3)  #twt_uppertuxen))),offset_range],'r.',ms=12)
ax5.plot(offsets[offset_range],scaling*gather_sm_bo2[(np.argmin(np.abs(t_plot_twt -(t_plot_twt[tops_idx[7]])))),offset_range],'r.',ms=marker_size, alpha = 0.3)  #twt_lowertuxen))),offset_range],'m.',ms=12)
ax5.plot(offsets[offset_range],scaling*gather_sm_bo2[(np.argmin(np.abs(t_plot_twt -(t_plot_twt[tops_idx[8]])))),offset_range],'m.',ms=marker_size, alpha = 0.3)  #twt_valhall))),offset_range],'b.',ms=12)
ax5.plot(offsets[offset_range],scaling*gather_sm_bo2[(np.argmin(np.abs(t_plot_twt -(t_plot_twt[tops_idx[9]])))),offset_range],'b.',ms=marker_size, alpha = 0.3)  #twt_sola))),offset_range],'g.',ms=12)
ax5.set_xlim(0,40)
ax5.set_xlabel('Offset')
ax5.set_ylabel('Amplitude')
#ax5.yaxis.set_label_coords(-0.2, .5)
ax5.axhline(y = 0, color='black', lw=1)
ax5.set_xticks(np.array([10,20,30]));
ax5.set_xticklabels(['10$^\circ$','$20^\circ$','30$^\circ$']);
ax5.set_ylim(-2,2)
ax5.grid()

#
scaling=5
#ax6.set_xlim(-1,4)
ax6.yaxis.set_label_position("right")
#ax6.set_xticks(np.array([-1,0,1,2,3]));
#ax6.set_xticklabels(['','6','12','18','24']);
ax6.set_xlabel('Offset')
ax6.set_yticklabels(['']);
ax6.plot(offsets[offset_range],scaling*syn_zoep_cdp_well.T[(np.argmin(np.abs(t_plot_twt - (t_plot_twt[tops_idx[6]])))),offset_range],'g.',ms=marker_size, alpha = 0.3)
ax6.plot(offsets[offset_range],scaling*syn_zoep_cdp_well.T[(np.argmin(np.abs(t_plot_twt - (t_plot_twt[tops_idx[7]])))),offset_range],'r.',ms=marker_size, alpha = 0.3)
#ax6.plot(offsets[offset_range],scaling*syn_zoep_cdp_well.T[(np.argmin(np.abs(t_plot_twt - (t_plot_twt[tops_idx[8]])))),offset_range],'m.',ms=marker_size, alpha = 0.3)
ax6.plot(offsets[offset_range],scaling*syn_zoep_cdp_well.T[(np.argmin(np.abs(t_plot_twt - (t_plot_twt[tops_idx[9]])))),offset_range],'b.',ms=marker_size, alpha = 0.3)
ax6.set_xlim(0,40)
ax6.set_xlabel('Offset')
ax6.axhline(y = 0, color='black', lw=1)
ax6.set_ylim(-2,2)
ax6.set_xticks(np.array([10,20,30]));
ax6.set_xticklabels(['10$^\circ$','$20^\circ$','30$^\circ$']);
ax6.grid()
ax6.yaxis.set_label_position("right")
ax6.set_yticklabels(['']);
#
#

#
scaling=6
ax7.plot(offsets[offset_range],scaling*syn_zoep_cdp_seis.T[(np.argmin(np.abs(t_plot_twt - (t_plot_twt[tops_idx[6]])))),offset_range],'g.',ms=marker_size, alpha = 0.3)
ax7.plot(offsets[offset_range],scaling*syn_zoep_cdp_seis.T[(np.argmin(np.abs(t_plot_twt - (t_plot_twt[tops_idx[7]])))),offset_range],'r.',ms=marker_size, alpha = 0.3)
#ax7.plot(offsets[offset_range],scaling*syn_zoep_cdp_seis.T[(np.argmin(np.abs(t_plot_twt - (t_plot_twt[tops_idx[8]])))),offset_range],'m.',ms=marker_size, alpha = 0.3)
ax7.plot(offsets[offset_range],scaling*syn_zoep_cdp_seis.T[(np.argmin(np.abs(t_plot_twt - (t_plot_twt[tops_idx[9]])))),offset_range],'b.',ms=marker_size, alpha = 0.3)
ax7.set_xlim(0,40)
ax7.set_xlabel('Offset')
ax7.axhline(y = 0, color='black', lw=1)
ax7.set_ylim(-2,2)
ax7.set_xticks(np.array([10,20,30]));
ax7.set_xticklabels(['10$^\circ$','$20^\circ$','30$^\circ$']);
ax7.set_yticklabels(['']);
ax7.grid()
ax7.yaxis.set_label_position("right")



#########

gs2 = gridspec.GridSpec(nrows=12, ncols=12, figure=fig,hspace =1 ,wspace=0.25)

ax8 = fig.add_subplot( gs2[7:9, 1:2])
ax9 = fig.add_subplot( gs2[7:9, 2:3])
ax10 = fig.add_subplot(gs2[7:9, 3:4])
ax11 = fig.add_subplot(gs2[7:9, 4:5])



size, top, base = 256, 0.4, 0.6
layer='hard'
polarity='reverse'
freq='low'
phase=180
style='ramp'
cmap=None
cbar = np.linspace(-1, 1, size).reshape(-1, 1)
freqs = {'vhi': 60, 'hi': 30, 'med': 15, 'lo': 7.5,
             'vhigh': 60, 'high': 30, 'medium': 15, 'low': 7.5,
             'mod': 15, 'mid': 15}
backgr = 'soft' if layer == 'hard' else 'hard'
value = 1 if layer == 'hard' else 0


_, v, _, syn = make_synthetic(size, top, base, value, freq=freqs[freq], phase=phase)


# Plot the earth model.
ax = ax8
cmap_ = 'Greys'
ax.imshow(v.reshape(-1, 1), aspect='auto', cmap=cmap_, vmin=-1.5, vmax=1.5)
ax.axhline(top*size, c='w', lw=4)
ax.axhline(base*size, c='w', lw=4)
ax.axvline(0.55, c='w', lw=1)  # Total hack to adjust RHS
ax.text(0, size/4.75, backgr, ha='center', va='center', color=get_colour(cmap_, (1-value)*256))
ax.text(0, size/2+0.75, layer, ha='center', va='center', color=get_colour(cmap_, (value)*256))

# Plot the impedance diagram.
ax = ax9
cmap_ = 'Greys'
ax.imshow(v.reshape(-1, 1), aspect='auto', cmap=cmap_, vmin=0, vmax=2)
ax.axvline(-0.5, c=(0.58, 0.58, 0.58), lw=10)
ax.text(0.45, 2*size/8, 'imp', ha='right', va='center', color='k')
ax.annotate("", xy=(0.5, size/8), xytext=(-0.2, size/8), arrowprops=dict(arrowstyle="->", connectionstyle="arc3"))

# Plot the waveform.
ax = ax10
y = np.arange(syn.size)
ax.plot(syn, y, 'k', lw=0.5)
ax.fill_betweenx(y, syn, 0, where=syn>0, color='k')
ax.invert_yaxis()
ax.text(0.65, size/8, '+', ha='center', va='center')
ax.text(-0.9, size/8, '–', ha='center', va='center')

# Plot the colourbar.
ax = ax11
cmap = 'gray'
frac = 1/8
top_col = get_colour(cmap, frac)
bot_col = get_colour(cmap, 7*frac)
ax.imshow(cbar, cmap=cmap, aspect='auto')
ax.text(0, frac*size, '+', ha='center', va='center', color=top_col)
ax.text(0, 7*frac*size, '–', ha='center', va='center', color=bot_col)

# Make final adjustments to subplots and figure.
ax8.set_axis_off()
ax9.set_axis_off()
ax10.set_axis_off()
ax11.set_axis_off()

plt.subplots_adjust(left=0.1)


ax1.text(x = ax1.get_xlim()[0]-0.1, y = ax1.get_ylim()[1]-0.0025,  s = 'b)', weight='normal')
ax2.text(x = ax2.get_xlim()[0]-1.4, y = ax2.get_ylim()[1]-0.0025,  s = 'c)', weight='normal')
ax3.text(x = ax3.get_xlim()[0]-1.4, y =  ax3.get_ylim()[1]-0.0025,  s = 'd)', weight='normal')
ax4.text(x = ax4.get_xlim()[0]-1.4, y =  ax3.get_ylim()[1]-0.0025,  s = 'e)', weight='normal')

#ax8.text(x = ax8.get_xlim()[0], y =  ax8.get_ylim()[1]-0.0025,  s = 'e)', weight='normal')

ax0.text(x = ax0.get_xlim()[0]+1.5, y = t_plot_twt.max() + 0.015 ,  s = 'Polarity', weight='normal', color = 'k')

if savefig == True:
    plt.savefig(fig_path+'prestack.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')

In [ ]:
#fig, ax  = plt.subplots(1, 6, figsize=set_size(width, subplots=(3,2)))
fig = plt.figure(figsize=set_size(width, subplots=(5,2)))
#fig, ax = plt.subplots(10, 8, figsize=set_size(width, subplots=(10, 8)))
#fig.tight_layout(pad=2)
fig.tight_layout
gs1 = gridspec.GridSpec(nrows=12, ncols=6, figure=fig,hspace =0 ,wspace=0.45)#,width_ratios=np.array([.5,.5,.5,.5,.5,.5,.5]))
#ax0 = fig.add_subplot(gs1[0:7, 0])
ax1 = fig.add_subplot(gs1[0:7, 0])
ax2 = fig.add_subplot(gs1[0:7, 1])
ax3 = fig.add_subplot(gs1[0:7, 2])
ax4 = fig.add_subplot(gs1[0:7, 3])

#ax5 = fig.add_subplot(gs1[8:12, 1:6])

#ax6 = fig.add_subplot(gs1[0:7, 1])
#
#ax6.set_ylim(t_plot_lc.min(),t_plot_lc.max())
#ax6.invert_yaxis()
#for pos, spine in ax0.axes.spines.items():
#    spine.set_visible(False)
#ax6.set_xticks([])
#ax6.set_ylabel(r'TWT [s]')
#ax6.axhline(tops_twt[6], color = 'red',     lw=2,   alpha=0.8)#, label = 'Top Sola')
#ax6.axhline(tops_twt[7], color = 'green',   lw=2,   alpha=0.8)#, label = 'Top U. Tuxen')
#ax6.axhline(tops_twt[8], color = 'orange',  lw=2,   alpha=0.8)#, label = 'Top L. Tuxen')
##ax6.axhline(tops_twt[9], color = 'blue',  lw=1,   alpha=0.3)#, label = 'Top Valhall')   
#    
#    #thicnkess in ms
#ds = np.round(np.diff(tops_twt)*1000.,1)
#
#for i in range(5,len(Fstr)-1):
#    if i == 0:
#        ax6.text(x = .5, y =tops_twt[i]+0.004, s = Fstr[i]+'\n ({0} ms)'.format(str(ds[i])),
#            fontsize = 5,
#            horizontalalignment = 'center',
#            verticalalignment = 'center',
#            #bbox = dict(facecolor = 'white', lw = 0.5),
#            weight = 'light',
#            rotation=0)
#    else:
#        ax6.text(x = .5, y =tops_twt[i]+0.007, s = Fstr[i]+'\n ({0} ms)'.format(str(ds[i])),
#            fontsize = 6,
#            horizontalalignment = 'center',
#            verticalalignment = 'center',
#            #bbox = dict(facecolor = 'white', lw = 0.5),
#            weight = 'light',
#            rotation=0)
  
#ax8 = fig.add_subplot(1,14,5)

ax1.plot(resSec['phi_mod'][lc_idx:] ,t_plot_lc,'k' ,lw=1)
ax1.set_ylim(t_plot_lc.max(), t_plot_lc.min())
ax1.set_xlabel('POR ' + r'$(v/v)$')
#ax1.xaxis.set_label_position('bottom')

#ax1.yaxis.set_label_coords(-0.37, .5)


x1 = resSec['phi_mod'][lc_idx:]*(1-resSec['sw_mod'][lc_idx:])
x2 = resSec['phi_mod'][lc_idx:]*resSec['sw_mod'][lc_idx:]


ax1.fill_betweenx(t_plot_lc,x2,0, alpha=0.3,color='b',label='Water')
ax1.fill_betweenx(t_plot_lc,x2,resSec['phi_mod'][lc_idx:], alpha=0.3,color='r',label='Oil')
#ax1.legend(loc=0)
ax1.set_xlim(0,0.5)
ax1.set_ylabel(r'TWT [s]')
ax1.legend(loc=0,fontsize=6)


#ax2 = ax1.twiny()  # instantiate a second axes that shares the same y-axis
ax2.plot(resSec['lit_mod'][lc_idx:], t_plot_lc,'k',lw=1)
ax2.set_ylim(t_plot_lc.max(), t_plot_lc.min())
ax2.set_xlabel(r'$SHV \ (v/v)$')
ax2.set_xlim(0,0.3)
ax2.set_yticklabels('')
ax2.fill_betweenx(t_plot_lc,0,resSec['lit_mod'][lc_idx:], color='k', alpha=0.3)



#ax2.set_title('AI')
ax3.plot(resSec['ai_mod'][lc_idx:]/1000 ,t_plot_lc,'k' ,lw=1)
ax3.set_xlabel('AI \n'+r' $(km/s \times g/cm^3)$',  )
ax3.set_ylim(t_plot_lc.max(), t_plot_lc.min())
ax3.set_yticklabels('')


#ax3.set_title('Vp/Vs')
ax4.plot(resSec['vp_mod'][lc_idx:]/resSec['vs_mod'][lc_idx:], t_plot_lc,'k' ,lw=1)
ax4.set_ylim(t_plot_lc.max(), t_plot_lc.min())
ax4.set_xlabel(r'$V_p/V_s$')
ax4.set_xticks(np.array([1.7,2]));
ax4.set_xticklabels(['1.7','2.0']);
ax4.set_yticklabels('')


for i in range (tops_twt.size-4):
    ax1.axhline(tops_twt[i],   lw=1, alpha = 0.3) 
    ax2.axhline(tops_twt[i],   lw=1, alpha = 0.3)  
    ax3.axhline(tops_twt[i],   lw=1, alpha = 0.3) 
    ax4.axhline(tops_twt[i],   lw=1, alpha = 0.3)  
    ax6.axhline(tops_twt[i],   lw=1, alpha = 0.3)                 

                
ax1.axhline(tops_twt[6], color = 'red', lw=2,   alpha=0.8)
ax1.axhline(tops_twt[7], color = 'green'  , lw=2,   alpha=0.8)
ax1.axhline(tops_twt[8], color = 'orange',     lw=2,   alpha=0.8)
#ax1.axhline(tops_twt[9], color = 'orange',  lw=2,   alpha=0.8)
ax2.axhline(tops_twt[6], color = 'red', lw=2,   alpha=0.8)
ax2.axhline(tops_twt[7], color = 'green',   lw=2,   alpha=0.8)
ax2.axhline(tops_twt[8], color = 'orange',     lw=2,   alpha=0.8)
#ax2.axhline(tops_twt[9], color = 'orange',  lw=2,   alpha=0.8)
ax3.axhline(tops_twt[6], color = 'red', lw=2,   alpha=0.8)
ax3.axhline(tops_twt[7], color = 'green',   lw=2,   alpha=0.8)
ax3.axhline(tops_twt[8], color = 'orange',     lw=2,   alpha=0.8)
#ax3.axhline(tops_twt[9], color = 'blue',  lw=2,   alpha=0.8)
ax4.axhline(tops_twt[6], color = 'red', lw=2,   alpha=0.8)
ax4.axhline(tops_twt[7], color = 'green',   lw=2,   alpha=0.8)
ax4.axhline(tops_twt[8], color = 'orange',     lw=2,   alpha=0.8)
#ax4.axhline(tops_twt[9], color = 'blue',  lw=2,   alpha=0.8)


ax5 = fig.add_subplot(gs1[0:7, 4])


mlo.plot_vawig(ax5, gather_sm_bo2.T[:,lc_idx:]/1e3, t_plot_lc, 1)
ax5.set_ylim(t_plot_lc.min(),t_plot_lc.max())
ax5.axhline(tops_twt[6], color = 'red',     lw=2,alpha = 0.8, label = 'Top Sola')
ax5.axhline(tops_twt[7], color = 'green',   lw=2,alpha = 0.8, label = 'Top Tuxen')
#ax4.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
ax5.axhline(tops_twt[8], color = 'Orange',  lw=2,alpha = 0.8, label = 'Top Valhall')
ax5.invert_yaxis()
#ax4.set_title('Observed CMP')
ax5.xaxis.tick_bottom()
ax5.set_yticklabels('')
ax5.set_xlim(-1,3)
ax5.set_xticks(np.array([0,1,2]));
ax5.set_xticklabels(['10','20','30'])
ax5.xaxis.set_label_position('bottom')
ax5.set_xlabel('Offset (deg)')

ax6 = fig.add_subplot(gs1[0:7, 5])

mlo.plot_vawig(ax6, syn_zoep_cdp_well[:,lc_idx:] , t_plot_lc, 1)
ax6.set_ylim(t_plot_lc.min(),t_plot_lc.max())
#ax5.axhline(tops_twt[6], color = 'green', lw=1,alpha = 0.3, label = 'Top Sola')
#ax5.axhline(tops_twt[7], color = 'red',   lw=1,alpha = 0.3, label = 'Upper Tuxen')
##ax3.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
#ax5.axhline(tops_twt[8], color = 'blue',  lw=1,alpha = 0.3, label = 'Top Valhall')
ax6.invert_yaxis()
#ax3.set_title('Well')
ax6.xaxis.tick_bottom()
ax6.xaxis.set_label_position('bottom')
ax6.set_yticklabels('')
ax6.set_xlim(-1,3)
ax6.set_xticks(np.array([0,1,2]));
ax6.set_xticklabels(['10','20','30'])
ax6.xaxis.set_label_position('bottom')
ax6.set_xlabel('Offset (deg)')


ax0.set_ylim(np.min(t_plot_lc),np.max(t_plot_lc))
ax0.invert_yaxis()
for pos, spine in ax0.axes.spines.items():
    spine.set_visible(False)
ax0.set_xticks([])
ax0.set_yticks([])

x_offset = 0.2
#ax0.annotate('',xy=(x_offset,tops_twt[6]),xytext=(x_offset, np.min(t_plot_lc)),
#            arrowprops=dict(facecolor='black',arrowstyle='<->', linewidth=1),)

#ax0.annotate('',xy=(x_offset,np.max(resSec['DPTM'])),xytext=(x_offset, tops_twt[6]),
#            arrowprops=dict(facecolor='black',arrowstyle='<->', linewidth=1),)
#
##ax0.text(x = x_offset-0.2 , y = (np.min(t_plot_lc) + tops_twt[6]) / 2 + 0.015,  s = 'Upper Chalk', weight='normal', rotation = -270)
#ax0.text(x =x_offset-0.3, y =  (np.max(resSec['DPTM']) + tops_twt[6]) / 2 ,  s = 'LC', weight='normal', rotation = -270)
                       
ax1.text(x = ax1.get_xlim()[0],   y = ax1.get_ylim()[1]-0.0025,  s = 'a)', weight='normal')
ax2.text(x = ax2.get_xlim()[0],   y = ax2.get_ylim()[1]-0.0025,  s = 'b)', weight='normal')
ax3.text(x = ax3.get_xlim()[0],  y = ax3.get_ylim()[1]-0.0025,  s = 'c)', weight='normal')
ax4.text(x = ax4.get_xlim()[0],  y =  ax4.get_ylim()[1]-0.0025,  s = 'd)', weight='normal')
ax5.text(x = ax5.get_xlim()[0], y =  ax5.get_ylim()[1]-0.0025,  s = 'e)', weight='normal')
ax6.text(x = ax6.get_xlim()[0], y =  ax6.get_ylim()[1]-0.0025,  s = 'f)', weight='normal')

#ax4.text(x = ax4.get_xlim()[1]+0.05, y = tops_twt[6]+0.001,  s = 'Top Sola', weight='normal', color = 'red')
#ax4.text(x = ax4.get_xlim()[1]+0.05, y = tops_twt[7]+0.001,  s = 'Top Tuxen', weight='normal', color = 'green')
#ax4.text(x = ax4.get_xlim()[1]+0.05, y = tops_twt[8]+0.001,  s = 'Top Valhall', weight='normal', color = 'orange')                                                

if savefig == True:
    plt.savefig(fig_path+'BO2X_block.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')
syn_zoep_cdp = syn_zoep_pp_well

<a id='RPmodelling'></a>
# Rock Physics Modelling

In [ ]:

resSec['TWT'] = t_plot_twt.copy()
Tuxen = resSec.loc[(resSec['DEPTH'] >= intf[7]) &  (resSec['DEPTH'] <= intf[7+1])].copy()

In [ ]:
#chk1 = pd.read_csv(path_las+'p-wave_chk1.txt', sep='                                ',skiprows=1,names=('KB_depth','P_chk'))
#chk1['P_chk'] = chk1['P_chk']**-1 *10**6
##plt.plot(chk1['P_chk'],chk1['KB_depth'])
#plt.ylim(chk1['KB_depth'].max(),chk1['KB_depth'].min())
#plt.plot(bo2['DT']-chk1['P_chk'],bo2['DEPTH'])

#print(np.round(bo2['DEPTH'],2)[:],chk1['KB_depth'][1:-1])
#plt.plot(np.round(bo2['DEPTH'],2)-chk1['KB_depth'][:-1])

In [ ]:
#depth_twt = pd.read_csv(path_las+'twt_depth_hrs.txt', sep=' ',skiprows=1,names=('KB','twt'))
#depth_twt


In [ ]:
resSec_1 = resSec[(bo2['DEPTH'] > bo2_fms.loc[bo2_fms['fm name'] == 'Sola']['depth'][0]) &
             (bo2['DEPTH'] < bo2_fms.loc[bo2_fms['fm name'] == 'BCU']['depth'][0])].copy()

resSec_1 = resSec_1.loc[resSec_1['PHIE'] != 0]


resSec_1['PHIE'].loc[resSec_1['F_label'] == 'Valhall']

In [ ]:
# Physical properties of constituents
K_o=0.62     # oil
Rho_o=0.8

K_w=2.3
G_w=0         # water
Rho_w=1.03

K_ca=71  
G_ca=30       # calcite
Rho_ca=2.71

K_q=37.3  
G_q=44.8      # quartz
Rho_q=2.65

K_k=15  
G_k=7       # shale
Rho_k=2.7

# Volume fraction of large calcite and large silicates 
c_l = 0.1 # large calcite
q_l = 0.05 # large quartz
k_l = 0.1 # large kaolinite

# Volume fraction of small calcite and suspended silicates
c_s = 0.5 # small calcite
q_s = 0.15 # small quartz
k_s = 0.1 # small kaolinite
print('Total volume fraction of solids equals: ', np.round(c_l+q_l+k_l+c_s+q_s+k_s,0))

# Reservoir properties
Por_c = 0.66 # Critical porosity
Por=np.linspace(0,Por_c,100)
S_w = 1 # water saturation


# Modeling

K_sus = np.nan*np.ones((Por.size,11))
K_IF = np.nan*np.ones((Por.size,11))
G_IF = np.nan*np.ones((Por.size,11))
M_IF = np.nan*np.ones((Por.size,11))
Vp_IF = np.nan*np.ones((Por.size,11))
Vs_IF = np.nan*np.ones((Por.size,11))
AI_IF = np.nan*np.ones((Por.size,11))
PS_IF = np.nan*np.ones((Por.size,11))
Rho = np.nan*np.ones((Por.size,11))

n=0;
for IF in np.arange(0,1.1,0.1):
    

    K_f = S_w/K_w + (1-S_w)/K_o # Reuss
    #K_f = S_w*K_w + (1-S_w)*K_o; # Voigt
    
    K_sus[:,n] = ( Por/Por_c*K_f + (1-Por/Por_c)*((c_s*(1-IF)/K_ca) + (q_s/K_q) + (k_s/K_k) ) )**(-1) * (Por/Por_c + (1-Por/Por_c)*(c_s*(1-IF) + q_s + k_s) )

        
    Rho[:,n] = (Por/Por_c)*((1-IF)*c_s*Rho_ca + (Rho_w*S_w) + (Rho_o*(1-S_w)) + Rho_k*k_s + Rho_q*q_s) + (1-(Por/Por_c))*((IF*c_s+c_l)*Rho_ca + q_l*Rho_q + k_l*Rho_k);

    K_IF[:,n] = ( ((Por/Por_c)/(K_sus[:,n] + (4/3)*G_ca)) + (1-(Por/Por_c))* ( (((1-IF)*c_s + q_s + k_s)/(K_sus[:,n] + (4/3)*G_ca)) + ((IF*c_s+c_l)/(K_ca + (4/3)*G_ca)) + (q_l / (K_q + (4/3)*G_ca)) + (k_l / (K_k + (4/3)*G_ca)) ))**(-1) - (4/3)*G_ca

    zeta = (G_ca/6) * ((9*K_ca + 8*G_ca)/(K_ca + 2*G_ca));

    G_IF[:,n] = ( ((Por/Por_c)/zeta) +  (1-(Por/Por_c))*( (((1-IF)*c_s + q_s + k_s)/(zeta)) + ((IF*c_s+c_l)/(G_ca + zeta)) + (q_l/(G_q + zeta)) + (k_l / (G_k + zeta))) )**(-1) - zeta;
        
    M_IF[:,n] = K_IF[:,n] + (4/3)*G_IF[:,n]
    
    Vp_IF[:,n] = np.sqrt(K_IF[:,n] + (4/3)*G_IF[:,n] / Rho[:,n])
    Vs_IF[:,n] = np.sqrt(G_IF[:,n] / Rho[:,n])
    
    AI_IF[:,n] = Vp_IF[:,n] * Rho[:,n]
    PS_IF[:,n] = Vp_IF[:,n] / Vs_IF[:,n]
    
    n=n+1
    
    


resSec_1['G'] = (resSec_1['Vs']/1000.)**2 * resSec_1['RHOZ_edt']/1000.
resSec_1['K'] = (resSec_1['Vp']/1000.)**2 * resSec_1['RHOZ_edt']/1000. - (4/3)*resSec_1['G']
resSec_1['M'] = resSec_1['K'] + (4/3)*resSec_1['G']

def vrh(volumes,k,mu):
    '''
    Calculates Voigt-Reuss-Hill bounds, (C) aadm 2015

    INPUT
    volumes: array with volumetric fractions
    k: array with bulk modulus
    mu: array with shear modulus

    OUTPUT
    k_u, k_l: upper (Voigt) and lower (Reuss) average of k
    mu_u, mu_l: upper (Voigt) and lower (Reuss) average of mu
    k0, mu0: Hill average of k and mu
    '''
    f=np.array(volumes).T
    k=np.resize(np.array(k),np.shape(f))
    mu=np.resize(np.array(mu),np.shape(f))
    ax=0 if f.ndim==1 else 1
    k_u = np.sum(f*k,axis=ax)
    k_l = 1./np.sum(f/k,axis=ax)
    mu_u = np.sum(f*mu,axis=ax)
    mu_l = 1./np.sum(f/mu,axis=ax)
    k0 = (k_u+k_l)/2.
    mu0 = (mu_u+mu_l)/2.
    return k_u, k_l, mu_u, mu_l, k0, mu0


def geqHSW(k1,g1,k2,g2,vol1):
    # Hashin-Shtrikman-Walpole bounds (Walpole 1966). More general version of the
    # Hashin-Shtrikman bounds where the constituent with the higher bulk
    # modulus does not neccessarily have to have the highest shear modulus.
    
    vol2 = 1-vol1
    Kmin = np.min([k1,k2])
    Kmax = np.max([k1,k2])
    Gmin = np.min([g1,g2])
    Gmax = np.max([g1,g2])
    
    ku = k1 + vol2/((k2-k1)**(-1) + vol1*(k1+4/3*Gmax)**(-1))
    kl = k1 + vol2/((k2-k1)**(-1) + vol1*(k1+4/3*Gmin)**(-1))
    gu = g1 + vol2/((g2-g1)**(-1) + \
    vol1*(g1 + Gmax/6 * ((9*Kmax+8*Gmax)/(Kmax+2*Gmax)))**(-1))
    gl = g1 + vol2/((g2-g1)**(-1) + \
    vol1*(g1 + Gmin/6 * ((9*Kmin+8*Gmin)/(Kmin+2*Gmin)))**(-1))
    
    return ku,kl,gu,gl

def geqGassmannDry(Ksat, Ks, Kf, Phi):
    #   Calculates the dry rock bulk modulus using Gassmann. 
    # 
    #   DryBulkModulus  - [array]; dry rock bulk modulus [GPa]
    #   Ksat            - [array]; effective (saturated) bulk modulus [GPa]
    #   Ks              - [array]; effective solid bulk modulus [GPa]
    #   Kf              - [array]; effective fluid bulk modulus [GPa]
    #   Phi             - [array]; porosity (in fraction of 1)
    DryBulkModulus   = (Ksat * ( Phi * Ks / Kf + 1 - Phi) - Ks) / \
                    (Phi * Ks / Kf + Ksat / Ks - 1 - Phi)
        
    return DryBulkModulus 

def geqGassmannFluidSubst(Ksat1,Ks,Kf1,Kf2,Phi):
    #   Calculate effective bulk modulus doing a Gassmann fluid substitution.
    # 
    #   NewEffectiveBulkModulus - [array]; effective bulk modulus after 
    #                               substituting with fluid 2 [GPa]
    #   Ksat1       - [array]; effective bulk modulus with (original) fluid 1 [GPa]
    #   Ks          - [array]; effective solid bulk modulus [GPa]
    #   Kf1         - [array]; effective bulk modulus of fluid 1 [GPa]
    #   Kf2         - [array]; effective bulk modulus of fluid 2 [GPa]
    #   Phi         - [array]; porosity  (in fraction of 1)
    
    Tmp = Ksat1/(Ks-Ksat1) - Kf1 / (Phi*(Ks-Kf1)) + \
    Kf2/(Phi*(Ks-Kf2))
    
    NewEffectiveBulkModulus = Ks*Tmp / (1+Tmp)
    
    return NewEffectiveBulkModulus




# Calculate the mineral/solid constituents using a Hashin-Shtrikman bound
_,resSec_1['k_s'],_,_ = geqHSW(65,35,K_k,G_k,1-(resSec_1['VSHALE']))

# Calculate the fluid bulk moduli using a Hill average
_,_,_,_,resSec_1['k_f'],_ = vrh([resSec_1['SW_CRETSYS'], 1-resSec_1['SW_CRETSYS']], [K_w,K_o],[_,_])

# Calculate the dry bulk modulus using Gassmann
resSec_1['k_d'] = geqGassmannDry(resSec_1['K'],resSec_1['k_s'],resSec_1['k_f'],resSec_1['PHIE'])

# Calculate water substituted bulk modulus using Gassmann
resSec_1['k_w'] = geqGassmannFluidSubst(resSec_1['K'],resSec_1['k_s'],resSec_1['k_f'],K_w,resSec_1['PHIE'])

In [ ]:
# Plot model vs data

#fig, ax = plt.subplots(nrows = 2, ncols = 1,sharex=True,figsize=(12,10))
fig = plt.figure(figsize=set_size(width, subplots=(4,4)))
fig.tight_layout
gs1 = gridspec.GridSpec(nrows=6, ncols=5, figure=fig,hspace =1 ,wspace=3)
ax0 = fig.add_subplot(gs1[0:3, 0:4])
ax1 = fig.add_subplot(gs1[3:6, 0:4])


groups = resSec_1['F_label'].unique()

if S_w == 1:
    tmp = 'k_w'
else:
    tmp = 'K'

c_idx = 0    
colors = np.array(['red','m']) 
  
for i in range(len(groups)):
    x, y = (resSec_1[resSec_1['F_label']==groups[i]]['PHIE'],resSec_1[resSec_1['F_label']==groups[i]][tmp])
    if (groups[i]=='Sola'):
        ax0.plot(resSec_1[resSec_1['F_label']==groups[i]]['PHIE'],resSec_1[resSec_1['F_label']==groups[i]][tmp],'kx',ms=6,label=groups[i])
    elif (groups[i]=='Valhall'):
        ax0.plot(resSec_1[resSec_1['F_label']==groups[i]]['PHIE'],resSec_1[resSec_1['F_label']==groups[i]][tmp],'k^',ms=6,label=groups[i],color='k')
    else:
        ax0.scatter(x, y, alpha=0.5, s=20, label=groups[i],color=colors[c_idx])
        c_idx = c_idx+1
ax0.plot(Por,K_IF[:,::2],lw=1,alpha=0.75)        
    
ax0.set_ylabel('Bulk modulus (GPa)')
ax0.set_title('BO-2X')
ax0.set_xticklabels([''])
ax0.grid()
ax0.set_axisbelow(True)
lg = ax0.legend(bbox_to_anchor=(1.01,.39),title='Facies',fancybox=True,shadow=True)
#plt.setp(lg.get_title())

ax0.set_xlim([0.1,0.45])
ax0.set_ylim([5,20])
ax0.text(x =ax0.get_xlim()[0]-0.01, y = ax0.get_ylim()[1]+1, s = 'a)',weight='normal')



    
c_idx = 0    
for i in range(len(groups)):
    x, y = (resSec_1[resSec_1['F_label']==groups[i]]['PHIE'],resSec_1[resSec_1['F_label']==groups[i]]['G'])
    if (groups[i]=='Sola'):
        ax1.plot(resSec_1[resSec_1['F_label']==groups[i]]['PHIE'],resSec_1[resSec_1['F_label']==groups[i]]['G'],'kx',ms=6 )
    elif (groups[i]=='Valhall'):
        ax1.plot(resSec_1[resSec_1['F_label']==groups[i]]['PHIE'],resSec_1[resSec_1['F_label']==groups[i]]['G'],'k^',ms=6,color='k')
    else:
        ax1.scatter(x, y, alpha=0.5, s=20  ,color=colors[c_idx])
        c_idx = c_idx+1
        

if_val = np.linspace(0,1,len(G_IF[0,::2]))
for n in range(len(if_val)):    
    ax1.plot(Por,G_IF[:,n*2],lw=1,alpha=0.75,label=np.round(if_val[n],1))       


ax1.set_xlabel('Porosity' + r'$(v/v)$')
ax1.set_ylabel('Shear modulus (GPa)')
ax1.grid()
ax1.set_axisbelow(True)
ax1.set_xlim([0.1,0.45])
ax1.set_ylim([2,6])

ax1.text(x = ax1.get_xlim()[0]-0.01, y = ax1.get_ylim()[1]+.25, s = 'b)', weight='normal')

lg = ax1.legend(bbox_to_anchor=(1.01,.20),title='IF value',fancybox=True,shadow=True)

if savefig == True:
    plt.savefig(fig_path+'IF_BO2X.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')

<a id='RPmodelling_interactive'></a>
# Interactive Rock Physics AVO Modelling

In [ ]:
def f(r_Sw, r_por, r_shv):
    
    Tuxen['sw_mod'] = r_Sw
    Tuxen['phi_mod'] = r_por
    Tuxen['lit_mod'] = r_shv
    
    # Physical properties of constituents
    K_o=0.62     # oil
    Rho_o=0.8

    K_w=2.3
    G_w=0         # water
    Rho_w=1.03

    K_ca=71  
    G_ca=30       # calcite
    Rho_ca=2.71

    K_q=37.3  
    G_q=44.8      # quartz
    Rho_q=2.65

    K_k=15  
    G_k=7       # shale
    Rho_k=2.7


    # Volume fraction of large calcite and large silicates 
    Vc_l = 0.1 # large calcite
    Vq_l = 0 # large quartz
    Vk_l = 0.05 # large kaolinite

    # Volume fraction of small calcite and suspended silicates
    Vc_s = 0.7 # small calcite
    Vq_s = 0.1 # small quartz
    Vk_s = 0.05 # small kaolinite

    Vtot_sh = Vk_s + Vk_l
    Vtot_r = Vc_l+Vq_l+Vc_s+Vq_s

    c_l = np.zeros(len(Tuxen['phi_mod'])) 
    c_l[:] = np.nan
    c_s = np.zeros(len(Tuxen['phi_mod']))
    c_s[:] = np.nan
    c_l = np.zeros(len(Tuxen['phi_mod']))
    c_l[:] = np.nan
    q_l = np.zeros(len(Tuxen['phi_mod']))
    q_l[:] = np.nan
    q_s = np.zeros(len(Tuxen['phi_mod']))
    q_s[:] = np.nan
    k_s = np.zeros(len(Tuxen['phi_mod']))
    k_s[:] = np.nan
    k_l = np.zeros(len(Tuxen['phi_mod']))
    k_l[:] = np.nan

    Por = np.zeros(len(Tuxen['phi_mod']))
    Por[:] = np.nan
    S_w = np.zeros(len(Tuxen['phi_mod']))
    S_w[:] = np.nan

    # Dynamic reservoir properties from log data
    for n in range(len(Tuxen['phi_mod'])):

        # Normalizing the solid volume fractions according to the shale log
        c_l[n] = (1-Tuxen['lit_mod'].iloc[n]) * Vc_l/Vtot_r
        c_s[n] = (1-Tuxen['lit_mod'].iloc[n]) * Vc_s/Vtot_r
        q_l[n] = (1-Tuxen['lit_mod'].iloc[n]) * Vq_l/Vtot_r
        q_s[n] = (1-Tuxen['lit_mod'].iloc[n]) * Vq_s/Vtot_r
        k_s[n] = Tuxen['lit_mod'].iloc[n] * Vk_s/Vtot_sh
        k_l[n] = Tuxen['lit_mod'].iloc[n] * Vk_l/Vtot_sh

        Por[n] = Tuxen['phi_mod'].iloc[n]
        S_w[n] = Tuxen['sw_mod'].iloc[n]

    # Reservoir properties
    Por_c = 0.66 # Critical porosity
    fluidMix = 'reuss'
    
    
    # Modeling

    IFc = np.linspace(0,1,6) # Iso-frame value for the suspended chalk
    IFs = 0.2 # Iso-frame value for the suspended shale
    IFq = 0.2 # Iso-frame value for the suspended quartz

    K_sus = np.nan*np.ones((Por.size,IFc.size))
    K_IF = np.nan*np.ones((Por.size,IFc.size))
    G_IF = np.nan*np.ones((Por.size,IFc.size))
    M_IF = np.nan*np.ones((Por.size,IFc.size))
    Vp_IF = np.nan*np.ones((Por.size,IFc.size))
    Vs_IF = np.nan*np.ones((Por.size,IFc.size))
    AI_IF = np.nan*np.ones((Por.size,IFc.size))
    PS_IF = np.nan*np.ones((Por.size,IFc.size))
    Rho = np.nan*np.ones((Por.size,IFc.size))

    K_f = S_w/K_w + (1-S_w)/K_o
    Rho =  Por*((Rho_w*S_w) + (Rho_o*(1-S_w))) + ((1-Por)*((c_s+c_l)*Rho_ca + Rho_k*(k_s+k_l) + Rho_q*(q_s+q_l))) 

    #for n in range(len(IFc)):
    n=2
    K_sus[:,n] = ( Por/Por_c*K_f + (1-Por/Por_c)*( (c_s*(1-IFc[n])/K_ca) + (q_s*(1-IFq)/K_q) + (k_s*(1-IFs)/K_k) ) )**(-1) * (Por/Por_c + (1-Por/Por_c)*( c_s*(1-IFc[n]) + q_s*(1-IFq) + k_s*(1-IFs)) )

    K_IF[:,n] = ( ((Por/Por_c)/(K_sus[:,n] + (4/3)*G_ca)) + (1-(Por/Por_c))* ( (((1-IFc[n])*c_s + (1-IFq)*q_s + (1-IFs)*k_s)/(K_sus[:,n] + (4/3)*G_ca)) + ((IFc[n]*c_s+c_l) / (K_ca + (4/3)*G_ca)) + ( (IFq*q_s+q_l) / (K_q + (4/3)*G_ca)) + ((IFs*k_s+k_l) / (K_k + (4/3)*G_ca)) ))**(-1) - (4/3)*G_ca

    zeta = (G_ca/6) * ((9*K_ca + 8*G_ca)/(K_ca + 2*G_ca));

    G_IF[:,n] = ( ((Por/Por_c)/zeta) +  (1-(Por/Por_c))*( (((1-IFc[n])*c_s + (1-IFq)*q_s + (1-IFs)*k_s)/(zeta)) + ((IFc[n]*c_s+c_l)/(G_ca + zeta)) + ((IFq*q_s+q_l)/(G_q + zeta)) + ( (IFs*k_s+k_l) / (G_k + zeta))) )**(-1) - zeta

    M_IF[:,n] = K_IF[:,n] + (4/3)*G_IF[:,n]

    Vp_IF[:,n] = np.sqrt((K_IF[:,n] + (4/3)*G_IF[:,n]) / Rho)
    Vs_IF[:,n] = np.sqrt(G_IF[:,n] / Rho)

    AI_IF[:,n] = Vp_IF[:,n] * Rho
    PS_IF[:,n] = Vp_IF[:,n] / Vs_IF[:,n]


        
        
        #   Calculate reflectivities from model parameters
    
        #   Calculate reflectivities from model parameters
    Tuxen['vp_mod'] = Vp_IF[0,2]
    Tuxen['vs_mod'] = Vs_IF[0,2]

    rc_zoep_pp = []
    theta1 = []
    dummy_rc = np.zeros([9,4,4])
    
    #phi_mod[7] = Por[0]
    rho_mod[7] = Rho[0]*1000.
    vp_mod[7] = Vp_IF[0,2]*1000.
    vs_mod[7] = Vs_IF[0,2]*1000.
    #lit_mod[7] = 
    
    
    for i in range(5, nangles,10):
        theta1_buf = i*theta1_step + theta1_min
        for i in range (vp_mod.size-1):
            dummy_rc[i,:,:] = mlo.rc_zoep(vp_mod[i], vs_mod[i], rho_mod[i], vp_mod[i+1], vs_mod[i+1], rho_mod[i+1], theta1_buf)

        theta1.append(theta1_buf)
        rc_zoep_pp.append((dummy_rc[0,0,0],dummy_rc[1,0,0],dummy_rc[2,0,0],
                          dummy_rc[3,0,0],dummy_rc[4,0,0],dummy_rc[5,0,0],
                          dummy_rc[6,0,0],dummy_rc[7,0,0],dummy_rc[8,0,0]))
    
    syn_zoep_pp_well = [] 
    syn_zoep_pp_seis = []
    angle_dummy = np.array([5,15,25])
    j=0
    for angle in range(5, nangles,10):
            nlayers = len(rc_zoep_pp[j])
            nint = nlayers
            nsamp = len(t_plot_twt)
            rc = list(np.zeros(nsamp,dtype='float'))
            lyr = 0

            for i in range(0, nsamp):
                for k in range(0,tops_twt.size-1):
                    if t_plot_twt[i] == tops_twt[k]:
                        rc[i] = rc_zoep_pp[j][k-1]
                    if lyr > nint:
                        break
            #k=0
            #   Convolve wavelet with reflectivities

            syn_buf = np.convolve(rc, wavelet_well[:,j], mode='same')
            syn_buf = list(syn_buf)
            syn_zoep_pp_well.append(syn_buf)

            syn_buf_seis = np.convolve(rc, wavelet_seismic[:,j], mode='same')
            syn_buf_seis = list(syn_buf_seis)
            syn_zoep_pp_seis.append(syn_buf_seis)
            j=j+1
            #print("Calculated angle %i" % (angle))

    #    Convert data arrays from lists/tuples to numpy arrays    
    syn_zoep_pp_well = np.array(syn_zoep_pp_well)
    syn_zoep_pp_seis = np.array(syn_zoep_pp_seis)
    rc_zoep_pp = np.array(rc_zoep_pp)

    #   Copy convolved top/base reflectivity values to Lists for easier plotting
    [ntrc, nsamp] = syn_zoep_pp_well.shape
    line1 = []
    line2 = []
    line3 = []
    line4 = []
    line5 = []
    line6 = []
    line7 = []
    line8 = []
    line9 = []
    line10 = []

    Yzoep = np.array(rc_zoep_pp[:,0])
    Yzoep = Yzoep.reshape((ntrc, 1))


    sintheta2 = np.sin(np.radians(np.arange(0, ntrc)))**2
    sintheta2 = sintheta2.reshape((ntrc, 1))


    tops_lines = np.zeros([tops_idx.size , syn_zoep_pp_well.shape[0]])    


    for i in range(0, ntrc):
        line1.append(syn_zoep_pp_well[i,np.int(tops_idx[0])])
        line2.append(syn_zoep_pp_well[i,np.int(tops_idx[1])])
        line3.append(syn_zoep_pp_well[i,np.int(tops_idx[2])])
        line4.append(syn_zoep_pp_well[i,np.int(tops_idx[3])])
        line5.append(syn_zoep_pp_well[i,np.int(tops_idx[4])])
        line6.append(syn_zoep_pp_well[i,np.int(tops_idx[5])])
        line7.append(syn_zoep_pp_well[i,np.int(tops_idx[6])])
        line8.append(syn_zoep_pp_well[i,np.int(tops_idx[7])])
        line9.append(syn_zoep_pp_well[i,np.int(tops_idx[8])])     
        line10.append(syn_zoep_pp_well[i,np.int(tops_idx[9])])

    line1=np.squeeze(np.array([line1]))
    line2=np.squeeze(np.array([line2]))
    line3=np.squeeze(np.array([line3]))
    line4=np.squeeze(np.array([line4]))
    line5=np.squeeze(np.array([line5]))
    line6=np.squeeze(np.array([line6]))
    line7=np.squeeze(np.array([line7]))
    line8=np.squeeze(np.array([line8]))
    line9=np.squeeze(np.array([line9]))
    line10=np.squeeze(np.array([line10]))


    

    tops_lines = np.zeros([tops_idx.size , syn_zoep_pp_well.shape[0]])    

    fig = plt.figure(figsize=(20, 15))
    fig.set_facecolor('white')
    fig.tight_layout
    
    ax0 = fig.add_subplot(1,12,1)
    ax1 = fig.add_subplot(1,12,2)
    ax2 = fig.add_subplot(1,12,3)
    ax3 = fig.add_subplot(1,12,4)
    ax4 = fig.add_subplot(1,12,5)
        
    ax0.plot(resSec['lit_mod'],t_plot_twt,'k',alpha=0.5,label='Vsh',lw=3)
    ax0.plot(resSec['sw_mod'],t_plot_twt, 'b',alpha=0.75,label='Sw',lw=3)
    ax0.plot(Tuxen['sw_mod'],Tuxen['TWT'], 'k.',alpha=0.75,label='Sw',lw=3)
    ax0.plot(Tuxen['lit_mod'],Tuxen['TWT'], 'k.',alpha=0.75,label='Vsh',lw=3)
    ax0.fill_betweenx(Tuxen['TWT'],np.ones(Tuxen.shape[0])*sw_mod[7],Tuxen['sw_mod'],color='skyblue',alpha=0.4)
    ax0.fill_betweenx(Tuxen['TWT'],np.ones(Tuxen.shape[0])*lit_mod[7],Tuxen['lit_mod'],color='k',alpha=0.4)
    ax0.invert_yaxis()
    ax0.set_ylabel('TWT (ms)', fontsize = 14)
    ax0.set_xlabel('(v/v)', fontsize = 14)
    ax0.axis('tight')
 #   ax0.grid()
    ax0.set_axisbelow(True)
    ax0.set_xlim(0,1)
    ax0.legend(loc=1,fontsize=12,fancybox=True,shadow=True)

    
     
    ax1.plot(resSec['phi_mod'],t_plot_twt,'g',alpha=0.75,lw=3)
    ax1.plot(Tuxen['phi_mod'],Tuxen['TWT'], 'k.',alpha=0.75,label='Sw',lw=3)
    ax1.fill_betweenx(Tuxen['TWT'],np.ones(Tuxen.shape[0])*r_por,resSec['phi_mod'].loc[resSec['F_label']== 'U. Tuxen'],color='green',alpha=0.4)
    ax1.invert_yaxis()
    ax1.set_yticklabels('')
    ax1.set_title('POR')
    ax1.set_xlabel('(v/v)', fontsize = 14)
    ax1.xaxis.set_label_position('bottom')
    ax1.axis('tight')
  #  ax1.grid()
    ax1.set_axisbelow(True)
    ax1.set_xlim(0,0.5)

     
    ax2.plot(resSec['rho_mod']/1000.,t_plot_twt,'k',alpha=1,lw=3)
    ax2.plot(Rho,Tuxen['TWT'],'r.',alpha=0.5)
    yerr = Rho.std() * 2 # ~ 95% confidence interval
    ax2.fill_betweenx(Tuxen['TWT'],Rho-yerr,Rho+yerr,color='skyblue',alpha=0.4)
    ax2.invert_yaxis()
    ax2.set_yticklabels('')
    ax2.set_title('RHO')
    ax2.set_xlabel(r'$(g/cm^3)$', fontsize = 14)
    ax2.xaxis.set_label_position('bottom')
    ax2.axis('tight')
   # ax2.grid()
    ax2.set_axisbelow(True)


    
    ax3.plot(resSec['vp_mod']/1000.,t_plot_twt,'k',alpha=1,lw=3)
    ax3.plot(Tuxen['vp_mod'],Tuxen['TWT'],'r.',alpha=0.5)
    yerr = Tuxen['vp_mod'].std() * 2 # ~ 95% confidence interval
    ax3.fill_betweenx(Tuxen['TWT'],Tuxen['vp_mod']-yerr,Tuxen['vp_mod']+yerr,color='skyblue',alpha=0.4)
    
    
     
    ax4.plot(resSec['vs_mod']/1000.,t_plot_twt,'k',alpha=1,lw=3,label='Data')

    

    ax4.plot(Tuxen['vs_mod'],Tuxen['TWT'],'r.',alpha=0.5,label='IF Model')
    yerr = Tuxen['vs_mod'].std() * 2 # ~ 95% confidence interval
    #ax4.fill_betweenx(Tuxen['TWT'],Tuxen['vs_mod']-yerr,Tuxen['vs_mod']+yerr,color='skyblue',alpha=0.4,label='95% CI')


    ax3.invert_yaxis()
    ax3.set_yticklabels('')
    ax3.set_title('Vp')
    ax3.set_xlabel('(km/s)', fontsize = 14)
    ax3.xaxis.set_label_position('bottom')
    ax3.axis('tight')
   # ax3.grid()
    ax3.set_axisbelow(True)


    ax4.invert_yaxis()
    ax4.set_yticklabels('')
    ax4.set_title('Vs')
    ax4.set_xlabel('(km/s)', fontsize = 14)
    ax4.xaxis.set_label_position('bottom')
    ax4.axis('tight')
   # ax4.grid()
    ax4.set_axisbelow(True)
    ax4.legend(loc=1,fontsize=12,fancybox=True,shadow=True)    

    syn_zoep_cdp_well = syn_zoep_pp_well
    

    #Pre-stack trace
    ax5 = fig.add_subplot(1,5,3)
    mlo.plot_vawig(ax5, syn_zoep_cdp_well , t_plot_twt, 1)
    ax5.set_ylim(t_plot_twt.min(),t_plot_twt.max())
    ax5.axhline(tops_twt[6], color = 'green', lw=3, label = 'Top Sola')
    ax5.axhline(tops_twt[7], color = 'red', lw=3 , label = 'Upper Tuxen')
    ax5.axhline(tops_twt[8], color = 'm', lw=3, label = 'Lower Tuxen')
    ax5.axhline(tops_twt[9], color = 'blue', lw=3, label = 'Top Valhall')
    ax5.invert_yaxis()
    ax5.set_title('Synthetic CDP', fontsize = 20,weight = 'bold')
    ax5.xaxis.tick_bottom()
    ax5.xaxis.set_label_position('bottom')
    ax5.set_xlim(-1,3)
    ax5.yaxis.set_label_position("right")
    ax5.set_xticks(np.array([0,1,2]));
    ax5.set_xticklabels(['10','20','30']);
    ax5.set_xlabel('Offset (deg)', fontsize = 14)
    ax5.set_yticklabels('')
    
    
    ax6 = fig.add_subplot(1,5,4)
    mlo.plot_vawig(ax6, gather_sm_bo2.T/1e3 , time_sm, 1)
    ax6.set_ylim(time_sm.min(),time_sm.max())
    ax6.set_ylim(t_plot_twt.min(),t_plot_twt.max())
    ax6.axhline(tops_twt[6], color = 'green', lw=3, label = 'Top Sola')
    ax6.axhline(tops_twt[7], color = 'red', lw=3 , label = 'Upper Tuxen')
    ax6.axhline(tops_twt[8], color = 'm', lw=3, label = 'Lower Tuxen')
    ax6.axhline(tops_twt[9], color = 'blue', lw=3, label = 'Top Valhall')
    ax6.invert_yaxis()
    ax6.set_xlabel('Offset (deg)', fontsize = 14)
    ax6.set_title('CMP @ BO-2X', fontsize = 20,weight = 'bold')
    #ax6.set_ylabel('Time (s)', fontsize = 14)
    ax6.xaxis.tick_bottom()
    ax6.xaxis.set_label_position('bottom')
    ax6.set_xlim(-1,3)
    ax6.set_xticks(np.array([0,1,2]));
    ax6.set_xticklabels(['10','20','30']);
    ax6.set_yticklabels('')
    ax6.text(x = ax6.get_xlim()[1] +.5 , y = tops_twt[6]+0.002,  s = 'Top Sola', weight='normal', color = 'green')
    ax6.text(x = ax6.get_xlim()[1] +.5 , y = tops_twt[7]+0.002,  s = 'Top U. Tuxen', weight='normal', color = 'red')
    ax6.text(x = ax6.get_xlim()[1] +.5 , y = tops_twt[8]+0.002,  s = 'Top L. Tuxen (MM)', weight='normal', color = 'm')
    ax6.text(x = ax6.get_xlim()[1] +.5 , y = tops_twt[8]+0.002,  s = 'Top Valhall', weight='normal', color = 'blue')   
    
    
    for i in range (tops_twt.size-4):
        ax0.axhline(tops_twt[i],   lw=1, alpha = 0.3)
        ax1.axhline(tops_twt[i],   lw=1, alpha = 0.3)
        ax2.axhline(tops_twt[i],   lw=1, alpha = 0.3)
        ax3.axhline(tops_twt[i],   lw=1, alpha = 0.3)
        ax4.axhline(tops_twt[i],   lw=1, alpha = 0.3)
        ax5.axhline(tops_twt[i],   lw=1, alpha = 0.3)
        ax6.axhline(tops_twt[i],   lw=1, alpha = 0.3)

    
    ax0.set_ylim(t_plot_twt.max(),t_plot_twt.min())
    ax1.set_ylim(t_plot_twt.max(),t_plot_twt.min())
    ax2.set_ylim(t_plot_twt.max(),t_plot_twt.min())
    ax3.set_ylim(t_plot_twt.max(),t_plot_twt.min())
    ax4.set_ylim(t_plot_twt.max(),t_plot_twt.min())
    
    ax0.axhline(tops_twt[6], color = 'green', lw=3, alpha=0.3)
    ax0.axhline(tops_twt[7], color = 'red', lw=3,   alpha=0.3)
    ax0.axhline(tops_twt[8], color = 'm', lw=3,     alpha=0.3)
    ax0.axhline(tops_twt[9], color = 'blue', lw=3,  alpha=0.3)    
    ax1.axhline(tops_twt[6], color = 'green', lw=3, alpha=0.3)
    ax1.axhline(tops_twt[7], color = 'red', lw=3,   alpha=0.3)
    ax1.axhline(tops_twt[8], color = 'm', lw=3,     alpha=0.3)
    ax1.axhline(tops_twt[9], color = 'blue', lw=3,  alpha=0.3)
    ax2.axhline(tops_twt[6], color = 'green', lw=3, alpha=0.3)
    ax2.axhline(tops_twt[7], color = 'red', lw=3,   alpha=0.3)
    ax2.axhline(tops_twt[8], color = 'm', lw=3,     alpha=0.3)
    ax2.axhline(tops_twt[9], color = 'blue', lw=3,  alpha=0.3)
    ax3.axhline(tops_twt[6], color = 'green', lw=3, alpha=0.3)
    ax3.axhline(tops_twt[7], color = 'red', lw=3,   alpha=0.3)
    ax3.axhline(tops_twt[8], color = 'm', lw=3,     alpha=0.3)
    ax3.axhline(tops_twt[9], color = 'blue', lw=3,  alpha=0.3)
    ax4.axhline(tops_twt[6], color = 'green', lw=3, alpha=0.3)
    ax4.axhline(tops_twt[7], color = 'red', lw=3,   alpha=0.3)
    ax4.axhline(tops_twt[8], color = 'm', lw=3,     alpha=0.3)
    ax4.axhline(tops_twt[9], color = 'blue', lw=3,  alpha=0.3)

    
interactive_plot = interactive(f, r_Sw=(0, 1., 0.1),r_por=(0,Por_c,.1), r_shv = (0,1.,.05),continuous_update=False,)
output = interactive_plot.children[-1]
output.layout.height = '500x'
interactive_plot



<a id='wedge'></a>
# Wedge Model

In [ ]:
# Wavelets average:
wavelet_seismic = np.sum(wavelet_seismic, 1)/wavelet_seismic.shape[1]
wavelet_well = np.sum(wavelet_well, 1)/wavelet_well.shape[1]

In [ ]:
plt.plot(wavelet_well)

In [ ]:
##   3-Layer Model Parameters [Layer1, Layer2, Layer 3]

#mlo.plot_vawig(ax5, syn_zoep_cdp_well , t_plot_twt, 1)


vp_mod = np.array([resSec['Vp'].loc[resSec['F_label']=='Hidra'].mean() , resSec['Vp'].loc[resSec['F_label']=='Sola'].mean(), (resSec['Vp'].loc[resSec['F_label']=='U. Tuxen'].mean() + resSec['Vp'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])  # P-wave velocity (m/s)
vs_mod = np.array([resSec['Vs'].loc[resSec['F_label']=='Hidra'].mean() , resSec['Vs'].loc[resSec['F_label']=='Sola'].mean(), (resSec['Vs'].loc[resSec['F_label']=='U. Tuxen'].mean()+ resSec['Vs'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])  # S-wave velocity (m/s)
rho_mod= np.array([resSec['RHOZ_edt'].loc[resSec['F_label']=='Hidra'].mean() , resSec['RHOZ_edt'].loc[resSec['F_label']=='Sola'].mean(), (resSec['RHOZ_edt'].loc[resSec['F_label']=='U. Tuxen'].mean()+resSec['RHOZ_edt'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])         # Density (g/cc)
#
dz_min = 0.0   # Minimum thickness of Layer 2 (m)
dz_max = 60.0  # Maximum thickness of Layer 2 (m)
dz_step= 1.0   # Thickness step from trace-to-trace (normally 1.0 m)
#   Trace Parameters
tmin = 0.0
tmax = 0.3

##   Plot Parameters
min_plot_time = 0.12
max_plot_time = 0.22
excursion = 2

###########################################################
#
#       FUNCTIONS DEFINITIONS
#

def plot_vawig(axhdl, data, t, excursion, highlight=None):
        import numpy as np
        import matplotlib.pyplot as plt

        [ntrc, nsamp] = data.shape
        t = np.hstack([0, t, t.max()])

        for i in range(0, ntrc):
            tbuf = excursion * data[i] / np.max(np.abs(data)) + i

            tbuf = np.hstack([i, tbuf, i])

            if i==highlight:
                lw = 1
                clr = 'black'
            else:
                lw = 1
                clr = 'black'

            axhdl.plot(tbuf, t, color=clr, linewidth=lw)
            plt.fill_betweenx(t, tbuf, i, where=tbuf>i, facecolor='k', linewidth=0)
            #plt.fill_betweenx(t, tbuf, i, where=tbuf>i, facecolor=[0.6,0.6,1.0], linewidth=0)
            #plt.fill_betweenx(t, tbuf, i, where=tbuf<i, facecolor=[1.0,0.7,0.7], linewidth=0)


        axhdl.set_xlim((-excursion, ntrc+excursion))
        axhdl.xaxis.tick_top()
        axhdl.xaxis.set_label_position('top')
        axhdl.invert_yaxis()
        return
  
def calc_rc(vp_mod, rho_mod):
        '''
        rc_int = calc_rc(vp_mod, rho_mod)
        '''

        nlayers = len(vp_mod)
        nint = nlayers - 1

        rc_int = []
        for i in range(0, nint):
            buf1 = vp_mod[i+1]*rho_mod[i+1]-vp_mod[i]*rho_mod[i]
            buf2 = vp_mod[i+1]*rho_mod[i+1]+vp_mod[i]*rho_mod[i]
            buf3 = buf1/buf2
            rc_int.append(buf3)

        return rc_int

def calc_times(z_int, vp_mod):
        '''
        t_int = calc_times(z_int, vp_mod)
        '''

        nlayers = len(vp_mod)
        nint = nlayers - 1

        t_int = []
        for i in range(0, nint):
            if i == 0:
                tbuf = z_int[i]/vp_mod[i]
                t_int.append(tbuf)
            else:
                zdiff = z_int[i]-z_int[i-1]
                tbuf = 2*zdiff/vp_mod[i] + t_int[i-1]
                t_int.append(tbuf)

        return t_int

def digitize_model(rc_int, t_int, t):
        '''
        rc = digitize_model(rc, t_int, t)

        rc = reflection coefficients corresponding to interface times
        t_int = interface times
        t = regularly sampled time series defining model sampling
        '''

        import numpy as np

        nlayers = len(rc_int)
        nint = nlayers - 1
        nsamp = len(t)

        rc = list(np.zeros(nsamp,dtype='float'))
        lyr = 0

        for i in range(0, nsamp):

            if t[i] >= t_int[lyr]:
                rc[i] = rc_int[lyr]
                lyr = lyr + 1    

            if lyr > nint:
                break

        return rc
    
    
    ##########################################################
#
#       COMPUTATIONS BELOW HERE...
#
#   Some handy constants
nlayers = len(vp_mod)
nint = nlayers - 1
nmodel = int((dz_max-dz_min)/dz_step+1)


#   Calculate reflectivities from model parameters
rc_int = calc_rc(vp_mod, rho_mod)

syn_zo = []
rc_zo = []
lyr_times = []
print(nmodel)
for model in range(0, nmodel):
    
        #   Calculate interface depths
        z_int = [600.0]
        z_int.append(z_int[0]+dz_min+dz_step*model)

        #   Calculate interface times
        t_int = calc_times(z_int, vp_mod)
        lyr_times.append(t_int)

        #   Digitize 3-layer model
        nsamp = int((tmax-tmin)/dt) + 1
        t = []
        for i in range(0,nsamp):
            t.append(i*dt)

        rc = digitize_model(rc_int, t_int, t)
        rc_zo.append(rc)

        #   Convolve wavelet with reflectivities
        syn_buf = np.convolve(rc, wavelet_seismic, mode='same')     ###### WAVELET CONVOLUTION HAPPENS HERE
        syn_buf = list(syn_buf)
        syn_zo.append(syn_buf)
        #print("finished step %i" % (model))

syn_zo = np.array(syn_zo)
t = np.array(t)
lyr_times = np.array(lyr_times)
lyr_indx = np.array(np.round(lyr_times/dt), dtype='int16')

# Use the transpose because rows are traces;
# columns are time samples.
max_amp = np.zeros(syn_zo.shape[0])
thickness = np.zeros(syn_zo.shape[0])
for i in range(syn_zo.shape[0]):
        max_amp[i] = np.min(syn_zo[i,:])
#max_amp.argmax()

#tuning_trace = max_amp.argmax()#np.argmin((syn_zo.T)) % syn_zo.T.shape[1]
#tuning_thickness = max_amp.argmax()
tuning_trace = np.argmax(np.abs(syn_zo.T)) % syn_zo.T.shape[1]
#tuning_thickness = tuning_trace * dz_step

#   Plotting Code
[ntrc, nsamp] = syn_zo.shape

fig = plt.figure(figsize=set_size(width, subplots=(5,2)))
fig.tight_layout

gs = gridspec.GridSpec(3, 1, height_ratios=[1, 1, 1],figure=fig)

ax0 = fig.add_subplot(gs[0])
ax0.plot(lyr_times[:,0], color='blue', lw=3)
ax0.plot(lyr_times[:,1], color='red', lw=3)
ax0.set_ylim((min_plot_time,max_plot_time))
ax0.invert_yaxis()
ax0.set_xlabel('Thickness (m)')
ax0.set_ylabel('Time (s)')
plt.text(2,
        min_plot_time + (lyr_times[0,0] - min_plot_time)/2.+0.01,
        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Hidra' ,np.str(np.int(vp_mod[0])), np.str(np.int(vs_mod[0])),np.str(np.int(rho_mod[0]))),
        fontsize=12)
plt.text(dz_max/dz_step - 2,
        lyr_times[-1,0] + (lyr_times[-1,1] - lyr_times[-1,0])/2.+0.01,
        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Sola' ,np.str(np.int(vp_mod[1])), np.str(np.int(vs_mod[1])),np.str(np.int(rho_mod[1]))),
        fontsize=12,
        horizontalalignment='right')
plt.text(2,
        lyr_times[0,0] + (max_plot_time - lyr_times[0,0])/2.+0.02,
        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} kg/m$^3$'.format('Tuxen' ,np.str(np.int(vp_mod[2])), np.str(np.int(vs_mod[2])),np.str(np.int(rho_mod[2]))),
        fontsize=12)
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')
ax0.set_xlim((-excursion, ntrc+excursion))

tuning_trace = 20+np.argmin(syn_zo[:,lyr_indx[:,0]][20:40,0])

ax1 = fig.add_subplot(gs[1])
plot_vawig(ax1, syn_zo, t, excursion, highlight=tuning_trace)
ax1.plot(lyr_times[:,0], color='blue', lw=3)
ax1.plot(lyr_times[:,1], color='red', lw=3)
ax1.set_ylim((min_plot_time,max_plot_time))
ax1.invert_yaxis()
ax1.set_xlabel('Thickness (m)')
ax1.set_ylabel('Time (s)')

ax2 = fig.add_subplot(gs[2])
ax2.plot(syn_zo[:,lyr_indx[:,0]], color='blue')
ax2.set_xlim((-excursion, ntrc+excursion))
#ax2.axvline(tuning_trace, color='k', lw=1)
ax2.grid()
ax2.set_title('Upper interface amplitude')
ax2.set_xlabel('Thickness (m)')
ax2.set_ylabel('Amplitude')
#plt.text(tuning_trace -15,
#        plt.ylim()[0] * 1.1-0.02,
#        'tuning thickness = {0} m'.format(str(tuning_trace),
#        fontsize=16))

#if savefig == True:
#    plt.savefig(fig_path+'wedge_tuning_hidrasolatuxen.pdf',dpi=400)
#    print('FIGURE SAVED')
plt.show()

In [ ]:
plt.figure()

def f(x_deg):
    #plt.plot(wavelet_well[:,0])

    rot = bg.filters.rotate_phase(wavelet_seismic,x_deg,degrees=True)

    nlayers = len(vp_mod)
    nint = nlayers - 1
    nmodel = int((dz_max-dz_min)/dz_step+1)


    #   Calculate reflectivities from model parameters
    rc_int = calc_rc(vp_mod, rho_mod)

    syn_zo = []
    rc_zo = []
    lyr_times = []
    for model in range(0, nmodel):

            #   Calculate interface depths
            z_int = [500.0]
            z_int.append(z_int[0]+dz_min+dz_step*model)

            #   Calculate interface times
            t_int = calc_times(z_int, vp_mod)
            lyr_times.append(t_int)

            #   Digitize 3-layer model
            nsamp = int((tmax-tmin)/dt) + 1
            t = []
            for i in range(0,nsamp):
                t.append(i*dt)

            rc = digitize_model(rc_int, t_int, t)
            rc_zo.append(rc)

            #   Convolve wavelet with reflectivities
            syn_buf = np.convolve(rc, rot, mode='same')     ###### WAVELET CONVOLUTION HAPPENS HERE
            syn_buf = list(syn_buf)
            syn_zo.append(syn_buf)
            #print("finished step %i" % (model))

    syn_zo = np.array(syn_zo)
    t = np.array(t)
    lyr_times = np.array(lyr_times)
    lyr_indx = np.array(np.round(lyr_times/dt), dtype='int16')

    # Use the transpose because rows are traces;
    # columns are time samples.
    max_amp = np.zeros(syn_zo.shape[0])
    thickness = np.zeros(syn_zo.shape[0])
    for i in range(syn_zo.shape[0]):
            max_amp[i] = np.min(syn_zo[i,:])
    #max_amp.argmax()

    #tuning_trace = max_amp.argmax()#np.argmin((syn_zo.T)) % syn_zo.T.shape[1]
    #tuning_thickness = max_amp.argmax()
    tuning_trace = np.argmax(np.abs(syn_zo.T)) % syn_zo.T.shape[1]
    #tuning_thickness = tuning_trace * dz_step

    #   Plotting Code
    [ntrc, nsamp] = syn_zo.shape

    fig = plt.figure(figsize=set_size(width, subplots=(5,2)))
    fig.tight_layout
    gs1 = gridspec.GridSpec(nrows=12, ncols=6, figure=fig,hspace =1 ,wspace=0.25)#,width_ratios=np.array([.5,.5,.5,.5,.5,.5,.5]))


    ax0 = fig.add_subplot(gs1[0:4, 0:4])
    ax0.plot(lyr_times[:,0], color='blue', lw=3)
    ax0.plot(lyr_times[:,1], color='red', lw=3)
    ax0.set_ylim((min_plot_time,max_plot_time))
    ax0.invert_yaxis()
    ax0.set_xlabel('Thickness (m)')
    ax0.set_ylabel('Time (s)')
    plt.text(2,
            min_plot_time + (lyr_times[0,0] - min_plot_time)/2.+0.01,
            '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Hidra' ,np.str(np.int(vp_mod[0])), np.str(np.int(vs_mod[0])),np.str(np.int(rho_mod[0]))),
            fontsize=12)
    plt.text(dz_max/dz_step - 2,
            lyr_times[-1,0] + (lyr_times[-1,1] - lyr_times[-1,0])/2.+0.01,
            '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Sola' ,np.str(np.int(vp_mod[1])), np.str(np.int(vs_mod[1])),np.str(np.int(rho_mod[1]))),
            fontsize=12,
            horizontalalignment='right')
    plt.text(2,
            lyr_times[0,0] + (max_plot_time - lyr_times[0,0])/2.+0.02,
            '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} kg/m$^3$'.format('Tuxen' ,np.str(np.int(vp_mod[2])), np.str(np.int(vs_mod[2])),np.str(np.int(rho_mod[2]))),
            fontsize=12)
    plt.gca().xaxis.tick_top()
    plt.gca().xaxis.set_label_position('top')
    ax0.set_xlim((-excursion, ntrc+excursion))

    tuning_trace = 20+np.argmin(syn_zo[:,lyr_indx[:,0]][20:40,0])

    ax1 = fig.add_subplot(gs1[4:8, 0:4])
    plot_vawig(ax1, syn_zo, t, excursion, highlight=tuning_trace)
    ax1.plot(lyr_times[:,0], color='blue', lw=3)
    ax1.plot(lyr_times[:,1], color='red', lw=3)
    ax1.set_ylim((min_plot_time,max_plot_time))
    ax1.invert_yaxis()
    ax1.set_xlabel('Thickness (m)')
    ax1.set_ylabel('Time (s)')

    ax2 = fig.add_subplot(gs1[8:12, 0:4])
    ax2.plot(syn_zo[:,lyr_indx[:,0]], color='blue')
    ax2.set_xlim((-excursion, ntrc+excursion))
    #ax2.axvline(tuning_trace, color='k', lw=1)
    ax2.grid()
    ax2.set_title('Upper interface amplitude')
    ax2.set_xlabel('Thickness (m)')
    ax2.set_ylabel('Amplitude')
    
    
    ax3 = fig.add_subplot(gs1[4:8, 4:6])
    
    ax3.plot(wavelet_well)

    rot = bg.filters.rotate_phase(wavelet_well,x_deg,degrees=True)

    ax3.plot(rot)
    
    
interactive_plot = interactive(f, x_deg=(0, 360, 1),continuous_update=False,)
output = interactive_plot.children[-1]
output.layout.height = '500x'
interactive_plot

In [ ]:
##   3-Layer Model Parameters [Layer1, Layer2, Layer 3]

#mlo.plot_vawig(ax5, syn_zoep_cdp_well , t_plot_twt, 1)


vp_mod = np.array([resSec['Vp'].loc[resSec['F_label']=='Sola'].mean(), (resSec['Vp'].loc[resSec['F_label']=='U. Tuxen'].mean() + resSec['Vp'].loc[resSec['F_label']=='L. Tuxen'].mean())/2., resSec['Vp'].loc[resSec['F_label']=='Valhall'].mean()])  # P-wave velocity (m/s)
vs_mod = np.array([resSec['Vs'].loc[resSec['F_label']=='Sola'].mean(), (resSec['Vs'].loc[resSec['F_label']=='U. Tuxen'].mean()+ resSec['Vs'].loc[resSec['F_label']=='L. Tuxen'].mean())/2. , resSec['Vs'].loc[resSec['F_label']=='Valhall'].mean()])  # S-wave velocity (m/s)
rho_mod= np.array([resSec['RHOZ_edt'].loc[resSec['F_label']=='Sola'].mean(), (resSec['RHOZ_edt'].loc[resSec['F_label']=='U. Tuxen'].mean()+resSec['RHOZ_edt'].loc[resSec['F_label']=='L. Tuxen'].mean())/2., resSec['RHOZ_edt'].loc[resSec['F_label']=='Valhall'].mean()])         # Density (g/cc)
#
dz_min = 0.0   # Minimum thickness of Layer 2 (m)
dz_max = 100.0  # Maximum thickness of Layer 2 (m)
dz_step= 1.0   # Thickness step from trace-to-trace (normally 1.0 m)
#   Trace Parameters
tmin = 0.0
tmax = 0.3

##   Plot Parameters
min_plot_time = 0.12
max_plot_time = 0.22
excursion = 2

###########################################################
#
#       FUNCTIONS DEFINITIONS
#

def plot_vawig(axhdl, data, t, excursion, highlight=None):
        import numpy as np
        import matplotlib.pyplot as plt

        [ntrc, nsamp] = data.shape
        t = np.hstack([0, t, t.max()])

        for i in range(0, ntrc):
            tbuf = excursion * data[i] / np.max(np.abs(data)) + i

            tbuf = np.hstack([i, tbuf, i])

            if i==highlight:
                lw = 2
                clr = 'magenta'
            else:
                lw = 1
                clr = 'black'

            axhdl.plot(tbuf, t, color=clr, linewidth=lw)
            plt.fill_betweenx(t, tbuf, i, where=tbuf>i, facecolor='k', linewidth=0)
            #plt.fill_betweenx(t, tbuf, i, where=tbuf>i, facecolor=[0.6,0.6,1.0], linewidth=0)
            #plt.fill_betweenx(t, tbuf, i, where=tbuf<i, facecolor=[1.0,0.7,0.7], linewidth=0)


        axhdl.set_xlim((-excursion, ntrc+excursion))
        axhdl.xaxis.tick_top()
        axhdl.xaxis.set_label_position('top')
        axhdl.invert_yaxis()
        return
  
def calc_rc(vp_mod, rho_mod):
        '''
        rc_int = calc_rc(vp_mod, rho_mod)
        '''

        nlayers = len(vp_mod)
        nint = nlayers - 1

        rc_int = []
        for i in range(0, nint):
            buf1 = vp_mod[i+1]*rho_mod[i+1]-vp_mod[i]*rho_mod[i]
            buf2 = vp_mod[i+1]*rho_mod[i+1]+vp_mod[i]*rho_mod[i]
            buf3 = buf1/buf2
            rc_int.append(buf3)

        return rc_int

def calc_times(z_int, vp_mod):
        '''
        t_int = calc_times(z_int, vp_mod)
        '''

        nlayers = len(vp_mod)
        nint = nlayers - 1

        t_int = []
        for i in range(0, nint):
            if i == 0:
                tbuf = z_int[i]/vp_mod[i]
                t_int.append(tbuf)
            else:
                zdiff = z_int[i]-z_int[i-1]
                tbuf = 2*zdiff/vp_mod[i] + t_int[i-1]
                t_int.append(tbuf)

        return t_int

def digitize_model(rc_int, t_int, t):
        '''
        rc = digitize_model(rc, t_int, t)

        rc = reflection coefficients corresponding to interface times
        t_int = interface times
        t = regularly sampled time series defining model sampling
        '''

        import numpy as np

        nlayers = len(rc_int)
        nint = nlayers - 1
        nsamp = len(t)

        rc = list(np.zeros(nsamp,dtype='float'))
        lyr = 0

        for i in range(0, nsamp):

            if t[i] >= t_int[lyr]:
                rc[i] = rc_int[lyr]
                lyr = lyr + 1    

            if lyr > nint:
                break

        return rc
    
    
    ##########################################################
#
#       COMPUTATIONS BELOW HERE...
#
#   Some handy constants
nlayers = len(vp_mod)
nint = nlayers - 1
nmodel = int((dz_max-dz_min)/dz_step+1)


#   Calculate reflectivities from model parameters
rc_int = calc_rc(vp_mod, rho_mod)

syn_zo = []
rc_zo = []
lyr_times = []
for model in range(0, nmodel):
    
        #   Calculate interface depths
        z_int = [500.0]
        z_int.append(z_int[0]+dz_min+dz_step*model)

        #   Calculate interface times
        t_int = calc_times(z_int, vp_mod)
        lyr_times.append(t_int)

        #   Digitize 3-layer model
        nsamp = int((tmax-tmin)/dt) + 1
        t = []
        for i in range(0,nsamp):
            t.append(i*dt)

        rc = digitize_model(rc_int, t_int, t)
        rc_zo.append(rc)

        #   Convolve wavelet with reflectivities
        syn_buf = np.convolve(rc, wavelet_seismic, mode='same')     ###### WAVELET CONVOLUTION HAPPENS HERE
        syn_buf = list(syn_buf)
        syn_zo.append(syn_buf)
        #print("finished step %i" % (model))

syn_zo = np.array(syn_zo)
t = np.array(t)
lyr_times = np.array(lyr_times)
lyr_indx = np.array(np.round(lyr_times/dt), dtype='int16')

# Use the transpose because rows are traces;
# columns are time samples.
max_amp = np.zeros(syn_zo.shape[0])
thickness = np.zeros(syn_zo.shape[0])
for i in range(syn_zo.shape[0]):
        max_amp[i] = np.min(syn_zo[i,:])
#max_amp.argmax()

#tuning_trace = max_amp.argmax()#np.argmin((syn_zo.T)) % syn_zo.T.shape[1]
#tuning_thickness = max_amp.argmax()
tuning_trace = np.argmax(np.abs(syn_zo.T)) % syn_zo.T.shape[1]
#tuning_thickness = tuning_trace * dz_step

#   Plotting Code
[ntrc, nsamp] = syn_zo.shape

fig = plt.figure(figsize=(12, 14))
fig.set_facecolor('white')

gs = gridspec.GridSpec(3, 1, height_ratios=[1, 1, 1])

ax0 = fig.add_subplot(gs[0])
ax0.plot(lyr_times[:,0], color='blue', lw=3)
ax0.plot(lyr_times[:,1], color='red', lw=3)
ax0.set_ylim((min_plot_time,max_plot_time))
ax0.invert_yaxis()
ax0.set_xlabel('Thickness (m)')
ax0.set_ylabel('Time (s)')
plt.text(2,
        min_plot_time + (lyr_times[0,0] - min_plot_time)/2.+0.01,
        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Sola' ,np.str(np.int(vp_mod[0])), np.str(np.int(vs_mod[0])),np.str(np.int(rho_mod[0]))),
        fontsize=12)
plt.text(dz_max/dz_step - 2,
        lyr_times[-1,0] + (lyr_times[-1,1] - lyr_times[-1,0])/2.+0.01,
        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Tuxen' ,np.str(np.int(vp_mod[1])), np.str(np.int(vs_mod[1])),np.str(np.int(rho_mod[1]))),
        fontsize=12,
        horizontalalignment='right')
plt.text(2,
        lyr_times[0,0] + (max_plot_time - lyr_times[0,0])/2.+0.02,
        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} kg/m$^3$'.format('Valhall' ,np.str(np.int(vp_mod[2])), np.str(np.int(vs_mod[2])),np.str(np.int(rho_mod[2]))),
        fontsize=12)
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')
ax0.set_xlim((-excursion, ntrc+excursion))

tuning_trace = 20+np.argmax(syn_zo[:,lyr_indx[:,0]][20:40,0])

ax1 = fig.add_subplot(gs[1])
plot_vawig(ax1, syn_zo, t, excursion, highlight=tuning_trace)
ax1.plot(lyr_times[:,0], color='blue', lw=3)
ax1.plot(lyr_times[:,1], color='red', lw=3)
ax1.set_ylim((min_plot_time,max_plot_time))
ax1.invert_yaxis()
ax1.set_xlabel('Thickness (m)')
ax1.set_ylabel('Time (s)')

ax2 = fig.add_subplot(gs[2])
ax2.plot(syn_zo[:,lyr_indx[:,0]], color='blue')
ax2.set_xlim((-excursion, ntrc+excursion))
ax2.axvline(tuning_trace, color='k', lw=1)
ax2.grid()
ax2.set_title('Upper interface amplitude')
ax2.set_xlabel('Thickness (m)')
ax2.set_ylabel('Amplitude')
plt.text(tuning_trace -15,
        plt.ylim()[0] * 1.1-0.02,
        'tuning thickness = {0} m'.format(str(tuning_trace),
        fontsize=16))

#if savefig == True:
#    plt.savefig(fig_path+'wedge_tuning_solatuxenvalhall.pdf',dpi=400)
#    print('FIGURE SAVED')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(syn_zo.T, cmap="seismic", extent = (0,60,t.max(),t.min()) ,vmin=-.15,vmax=.15,  aspect='auto')
plt.colorbar()
plt.show()

# Tuning effect live demo <a id='wedge_interactive'></a>

In [ ]:
Sola = resSec.loc[(resSec['DEPTH'] >= intf[6]) &  (resSec['DEPTH'] <= intf[6+1])].copy()


def f(r_Sw, r_por, r_shv):
    
    Sola['sw_mod'] = r_Sw
    Sola['phi_mod'] = r_por
    Sola['lit_mod'] = r_shv
    
    
    # Physical properties of constituents
    K_o=0.62     # oil
    Rho_o=0.8

    K_w=2.3
    G_w=0         # water
    Rho_w=1.03

    K_ca=71  
    G_ca=30       # calcite
    Rho_ca=2.71

    K_q=37.3  
    G_q=44.8      # quartz
    Rho_q=2.65

    K_k=15  
    G_k=7       # shale
    Rho_k=2.7


    # Volume fraction of large calcite and large silicates 
    Vc_l = 0.1 # large calcite
    Vq_l = 0 # large quartz
    Vk_l = 0.05 # large kaolinite

    # Volume fraction of small calcite and suspended silicates
    Vc_s = 0.7 # small calcite
    Vq_s = 0.1 # small quartz
    Vk_s = 0.05 # small kaolinite

    Vtot_sh = Vk_s + Vk_l
    Vtot_r = Vc_l+Vq_l+Vc_s+Vq_s

    c_l = np.zeros(len(Sola['phi_mod'])) 
    c_l[:] = np.nan
    c_s = np.zeros(len(Sola['phi_mod']))
    c_s[:] = np.nan
    c_l = np.zeros(len(Sola['phi_mod']))
    c_l[:] = np.nan
    q_l = np.zeros(len(Sola['phi_mod']))
    q_l[:] = np.nan
    q_s = np.zeros(len(Sola['phi_mod']))
    q_s[:] = np.nan
    k_s = np.zeros(len(Sola['phi_mod']))
    k_s[:] = np.nan
    k_l = np.zeros(len(Sola['phi_mod']))
    k_l[:] = np.nan

    Por = np.zeros(len(Sola['phi_mod']))
    Por[:] = np.nan
    S_w = np.zeros(len(Sola['phi_mod']))
    S_w[:] = np.nan

    # Dynamic reservoir properties from log data
    for n in range(len(Sola['phi_mod'])):

        # Normalizing the solid volume fractions according to the shale log
        c_l[n] = (1-Sola['lit_mod'].iloc[n]) * Vc_l/Vtot_r
        c_s[n] = (1-Sola['lit_mod'].iloc[n]) * Vc_s/Vtot_r
        q_l[n] = (1-Sola['lit_mod'].iloc[n]) * Vq_l/Vtot_r
        q_s[n] = (1-Sola['lit_mod'].iloc[n]) * Vq_s/Vtot_r
        k_s[n] = Sola['lit_mod'].iloc[n] * Vk_s/Vtot_sh
        k_l[n] = Sola['lit_mod'].iloc[n] * Vk_l/Vtot_sh

        Por[n] = Sola['phi_mod'].iloc[n]
        S_w[n] = Sola['sw_mod'].iloc[n]

    # Reservoir properties
    Por_c = 0.66 # Critical porosity
    fluidMix = 'reuss'
    
    
    # Modeling

    IFc = np.linspace(0,1,6) # Iso-frame value for the suspended chalk
    IFs = 0.2 # Iso-frame value for the suspended shale
    IFq = 0.2 # Iso-frame value for the suspended quartz

    K_sus = np.nan*np.ones((Por.size,IFc.size))
    K_IF = np.nan*np.ones((Por.size,IFc.size))
    G_IF = np.nan*np.ones((Por.size,IFc.size))
    M_IF = np.nan*np.ones((Por.size,IFc.size))
    Vp_IF = np.nan*np.ones((Por.size,IFc.size))
    Vs_IF = np.nan*np.ones((Por.size,IFc.size))
    AI_IF = np.nan*np.ones((Por.size,IFc.size))
    PS_IF = np.nan*np.ones((Por.size,IFc.size))
    Rho = np.nan*np.ones((Por.size,IFc.size))

    K_f = S_w/K_w + (1-S_w)/K_o
    Rho =  Por*((Rho_w*S_w) + (Rho_o*(1-S_w))) + ((1-Por)*((c_s+c_l)*Rho_ca + Rho_k*(k_s+k_l) + Rho_q*(q_s+q_l))) 

    #for n in range(len(IFc)):
    n=2
    K_sus[:,n] = ( Por/Por_c*K_f + (1-Por/Por_c)*( (c_s*(1-IFc[n])/K_ca) + (q_s*(1-IFq)/K_q) + (k_s*(1-IFs)/K_k) ) )**(-1) * (Por/Por_c + (1-Por/Por_c)*( c_s*(1-IFc[n]) + q_s*(1-IFq) + k_s*(1-IFs)) )

    K_IF[:,n] = ( ((Por/Por_c)/(K_sus[:,n] + (4/3)*G_ca)) + (1-(Por/Por_c))* ( (((1-IFc[n])*c_s + (1-IFq)*q_s + (1-IFs)*k_s)/(K_sus[:,n] + (4/3)*G_ca)) + ((IFc[n]*c_s+c_l) / (K_ca + (4/3)*G_ca)) + ( (IFq*q_s+q_l) / (K_q + (4/3)*G_ca)) + ((IFs*k_s+k_l) / (K_k + (4/3)*G_ca)) ))**(-1) - (4/3)*G_ca

    zeta = (G_ca/6) * ((9*K_ca + 8*G_ca)/(K_ca + 2*G_ca));

    G_IF[:,n] = ( ((Por/Por_c)/zeta) +  (1-(Por/Por_c))*( (((1-IFc[n])*c_s + (1-IFq)*q_s + (1-IFs)*k_s)/(zeta)) + ((IFc[n]*c_s+c_l)/(G_ca + zeta)) + ((IFq*q_s+q_l)/(G_q + zeta)) + ( (IFs*k_s+k_l) / (G_k + zeta))) )**(-1) - zeta

    M_IF[:,n] = K_IF[:,n] + (4/3)*G_IF[:,n]

    Vp_IF[:,n] = np.sqrt((K_IF[:,n] + (4/3)*G_IF[:,n]) / Rho)
    Vs_IF[:,n] = np.sqrt(G_IF[:,n] / Rho)

    AI_IF[:,n] = Vp_IF[:,n] * Rho
    PS_IF[:,n] = Vp_IF[:,n] / Vs_IF[:,n]


        
        
    #   Calculate reflectivities from model parameters
    
    Sola['vp_mod'] = Vp_IF[0,2]
    Sola['vs_mod'] = Vs_IF[0,2]
    
    #phi_mod[6] = Por[0]
    #rho_mod[6] = Rho[0]*1000.
    #vp_mod[6] = Vp_IF[0,2]*1000.
    #vs_mod[6] = Vs_IF[0,2]*1000.
    
    
   
    vp_mod = np.array([resSec['Vp'].loc[resSec['F_label']=='Hidra'].mean() ,Vp_IF[0,2]*1000., (resSec['Vp'].loc[resSec['F_label']=='U. Tuxen'].mean() + resSec['Vp'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])  # P-wave velocity (m/s)
    vs_mod = np.array([resSec['Vs'].loc[resSec['F_label']=='Hidra'].mean() , Vs_IF[0,2]*1000., (resSec['Vs'].loc[resSec['F_label']=='U. Tuxen'].mean()+ resSec['Vs'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])  # S-wave velocity (m/s)
    rho_mod= np.array([resSec['RHOZ_edt'].loc[resSec['F_label']=='Hidra'].mean() , Rho[0]*1000., (resSec['RHOZ_edt'].loc[resSec['F_label']=='U. Tuxen'].mean()+resSec['RHOZ_edt'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])         # Density (g/cc)

    dz_min = 0.0   # Minimum thickness of Layer 2 (m)
    dz_max = 60.0  # Maximum thickness of Layer 2 (m)
    dz_step= 1.0   # Thickness step from trace-to-trace (normally 1.0 m)
    #   Trace Parameters
    tmin = 0.0
    tmax = 0.3

    ##   Plot Parameters
    min_plot_time = tmin
    max_plot_time = tmax
    excursion = 2
    
    
    
    nlayers = len(vp_mod)
    nint = nlayers - 1
    nmodel = int((dz_max-dz_min)/dz_step+1)


    #   Calculate reflectivities from model parameters
    rc_int = calc_rc(vp_mod, rho_mod)

    syn_zo = []
    rc_zo = []
    lyr_times = []
    for model in range(0, nmodel):

            #   Calculate interface depths
            z_int = [500.0]
            z_int.append(z_int[0]+dz_min+dz_step*model)

            #   Calculate interface times
            t_int = calc_times(z_int, vp_mod)
            lyr_times.append(t_int)

            #   Digitize 3-layer model
            nsamp = int((tmax-tmin)/dt) + 1
            t = []
            for i in range(0,nsamp):
                t.append(i*dt)

            rc = digitize_model(rc_int, t_int, t)
            rc_zo.append(rc)

            #   Convolve wavelet with reflectivities
            syn_buf = np.convolve(rc, wavelet_seismic, mode='same')     ###### WAVELET CONVOLUTION HAPPENS HERE
            syn_buf = list(syn_buf)
            syn_zo.append(syn_buf)
            #print("finished step %i" % (model))

    syn_zo = np.array(syn_zo)
    t = np.array(t)
    lyr_times = np.array(lyr_times)
    lyr_indx = np.array(np.round(lyr_times/dt), dtype='int16')

    # Use the transpose because rows are traces;
    # columns are time samples.
    max_amp = np.zeros(syn_zo.shape[0])
    thickness = np.zeros(syn_zo.shape[0])
    for i in range(syn_zo.shape[0]):
            max_amp[i] = np.min(syn_zo[i,:])
    #max_amp.argmax()

    #tuning_trace = max_amp.argmax()#np.argmin((syn_zo.T)) % syn_zo.T.shape[1]
    #tuning_thickness = max_amp.argmax()
    tuning_trace = np.argmax(np.abs(syn_zo.T)) % syn_zo.T.shape[1]
    #tuning_thickness = tuning_trace * dz_step

    #   Plotting Code
    [ntrc, nsamp] = syn_zo.shape

    fig = plt.figure(figsize=(20, 14))
    fig.set_facecolor('white')

    gs = gridspec.GridSpec(12,12)#, height_ratios=[1, 1, 1])

    ax0 = fig.add_subplot(gs[0:4,0:6])
    ax0.plot(lyr_times[:,0], color='blue', lw=3)
    ax0.plot(lyr_times[:,1], color='red', lw=3)
    ax0.set_ylim((min_plot_time,max_plot_time))
    ax0.invert_yaxis()
    ax0.set_xlabel('Sola Thickness (m)')
    ax0.set_ylabel('Time (s)')
    plt.text(2,
            min_plot_time + (lyr_times[0,0] - min_plot_time)/2.+0.01,
            '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Hidra' ,np.str(np.int(vp_mod[0])), np.str(np.int(vs_mod[0])),np.str(np.int(rho_mod[0]))),
            fontsize=12)
    plt.text(dz_max/dz_step - 2,
            lyr_times[-1,0] + (lyr_times[-1,1] - lyr_times[-1,0])/2.+0.01,
            '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Sola' ,np.str(np.int(vp_mod[1])), np.str(np.int(vs_mod[1])),np.str(np.int(rho_mod[1]))),
            fontsize=12,
            horizontalalignment='right')
    plt.text(2,
            lyr_times[0,0] + (max_plot_time - lyr_times[0,0])/2.+0.02,
            '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} kg/m$^3$'.format('Tuxen' ,np.str(np.int(vp_mod[2])), np.str(np.int(vs_mod[2])),np.str(np.int(rho_mod[2]))),
            fontsize=12)
    plt.gca().xaxis.tick_top()
    plt.gca().xaxis.set_label_position('top')
    ax0.set_xlim((-excursion, ntrc+excursion))

    #tuning_trace = 20+np.argmax(syn_zo[:,lyr_indx[:,0]][20:40,0])

    ax1 = fig.add_subplot(gs[4:8,0:6])
    im1 = ax1.imshow(syn_zo.T, cmap="seismic", extent = (0,60,t.max(),t.min()) ,vmin=-.35,vmax=.35,  aspect='auto')
    ax1.plot(lyr_times[:,0], color='k', lw=3,ls='--',alpha=0.5)
    ax1.plot(lyr_times[:,1], color='k', lw=3,ls='--',alpha=0.5)
    #ax1.colorbar()
#    plot_vawig(ax1, syn_zo, t, excursion, highlight=tuning_trace)
#    ax1.plot(lyr_times[:,0], color='blue', lw=3)
#    ax1.plot(lyr_times[:,1], color='red', lw=3)
#    ax1.set_ylim((min_plot_time,max_plot_time))
#    ax1.invert_yaxis()
#    ax1.set_xlabel('Thickness (m)')
#    ax1.set_ylabel('Time (s)')
    #fig.colorbar(im1, ax=ax1, orientation='horizontal')

    
    
    ax2 = fig.add_subplot(gs[8:,0:6])
    ax2.plot(syn_zo[:,lyr_indx[:,0]]/np.abs(syn_zo[:,lyr_indx[:,0]]).max(), lw=3,color='k')
    ax2.set_xlim((-excursion, ntrc+excursion))
    #ax2.axvline(tuning_trace, color='k', lw=1)
    ax2.set_ylim(0,1.1)
    ax2.grid()
    ax2.set_title('Upper interface amplitude')
    ax2.set_xlabel('Sola Thickness (m)')
    ax2.set_ylabel('Amplitude')
#    plt.text(tuning_trace -15,
#            plt.ylim()[0] * 1.1-0.02,
#            'tuning thickness = {0} m'.format(str(tuning_trace),
#            fontsize=16))
#
 

#gs2 = gridspec.GridSpec(nrows=12, ncols=12, figure=fig,hspace =1 ,wspace=0.25)
#
    ax8 = fig.add_subplot( gs[4:8, 6:7])
    ax9 = fig.add_subplot( gs[4:8, 7:8])
    ax10 = fig.add_subplot(gs[4:8, 8:9])
    ax11 = fig.add_subplot(gs[4:8, 9:10])



    size, top, base = 256, 0.4, 0.6
    layer='hard'
    polarity='reverse'
    freq='low'
    phase=180
    style='ramp'
    cmap=None
    cbar = np.linspace(-1, 1, size).reshape(-1, 1)
    freqs = {'vhi': 60, 'hi': 30, 'med': 15, 'lo': 7.5,
                 'vhigh': 60, 'high': 30, 'medium': 15, 'low': 7.5,
                 'mod': 15, 'mid': 15}
    backgr = 'soft' if layer == 'hard' else 'hard'
    value = 1 if layer == 'hard' else 0


    _, v, _, syn = make_synthetic(size, top, base, value, freq=freqs[freq], phase=phase)


    # Plot the earth model.
    ax = ax8
    cmap_ = 'Greys'
    ax.imshow(v.reshape(-1, 1), aspect='auto', cmap=cmap_, vmin=-1.5, vmax=1.5)
    ax.axhline(top*size, c='w', lw=4)
    ax.axhline(base*size, c='w', lw=4)
    ax.axvline(0.55, c='w', lw=1)  # Total hack to adjust RHS
    ax.text(0, size/4.75, backgr, ha='center', va='center', color=get_colour(cmap_, (1-value)*256))
    ax.text(0, size/2+0.75, layer, ha='center', va='center', color=get_colour(cmap_, (value)*256))

    # Plot the impedance diagram.
    ax = ax9
    cmap_ = 'Greys'
    ax.imshow(v.reshape(-1, 1), aspect='auto', cmap=cmap_, vmin=0, vmax=2)
    ax.axvline(-0.5, c=(0.58, 0.58, 0.58), lw=10)
    ax.text(0.45, 2*size/8, 'imp', ha='right', va='center', color='k')
    ax.annotate("", xy=(0.5, size/8), xytext=(-0.2, size/8), arrowprops=dict(arrowstyle="->", connectionstyle="arc3"))

    # Plot the waveform.
    ax = ax10
    y = np.arange(syn.size)
    ax.plot(syn, y, 'k', lw=0.5)
    ax.fill_betweenx(y, syn, 0, where=syn>0, color='k')
    ax.invert_yaxis()
    ax.text(0.65, size/8, '+', ha='center', va='center')
    ax.text(-0.9, size/8, '–', ha='center', va='center')

    # Plot the colourbar.
    ax = ax11
    cmap = 'seismic'
    frac = 1/8
    top_col = get_colour(cmap, frac)
    bot_col = get_colour(cmap, 7*frac)
    ax.imshow(cbar, cmap=cmap, aspect='auto')
    ax.text(0, frac*size, '+', ha='center', va='center', color=top_col)
    ax.text(0, 7*frac*size, '–', ha='center', va='center', color=bot_col)

    # Make final adjustments to subplots and figure.
    ax8.set_axis_off()
    ax9.set_axis_off()
    ax10.set_axis_off()
    ax11.set_axis_off()

    plt.tight_layout()


interactive_plot = interactive(f, r_Sw=(0, 1., .05),r_por=(0,Por_c,.05), r_shv = (0,1.,.05),continuous_update=False,)
output = interactive_plot.children[-1]
output.layout.height = '1000x'
interactive_plot




In [ ]:
print(t.max(),t.min(),dt)
t.shape

In [ ]:
Sola = resSec.loc[(resSec['DEPTH'] >= intf[6]) &  (resSec['DEPTH'] <= intf[6+1])].copy()

r_Sw = 0.3
r_por = 0.4
    
Sola['sw_mod'] = r_Sw
Sola['phi_mod'] = r_por

# Physical properties of constituents
K_o=0.62     # oil
Rho_o=0.8

K_w=2.3
G_w=0         # water
Rho_w=1.03

K_ca=71  
G_ca=30       # calcite
Rho_ca=2.71

K_q=37.3  
G_q=44.8      # quartz
Rho_q=2.65

K_k=15  
G_k=7       # shale
Rho_k=2.7


# Volume fraction of large calcite and large silicates 
Vc_l = 0.1 # large calcite
Vq_l = 0 # large quartz
Vk_l = 0.05 # large kaolinite

# Volume fraction of small calcite and suspended silicates
Vc_s = 0.7 # small calcite
Vq_s = 0.1 # small quartz
Vk_s = 0.05 # small kaolinite

Vtot_sh = Vk_s + Vk_l
Vtot_r = Vc_l+Vq_l+Vc_s+Vq_s

c_l = np.zeros(len(Sola['phi_mod'])) 
c_l[:] = np.nan
c_s = np.zeros(len(Sola['phi_mod']))
c_s[:] = np.nan
c_l = np.zeros(len(Sola['phi_mod']))
c_l[:] = np.nan
q_l = np.zeros(len(Sola['phi_mod']))
q_l[:] = np.nan
q_s = np.zeros(len(Sola['phi_mod']))
q_s[:] = np.nan
k_s = np.zeros(len(Sola['phi_mod']))
k_s[:] = np.nan
k_l = np.zeros(len(Sola['phi_mod']))
k_l[:] = np.nan

Por = np.zeros(len(Sola['phi_mod']))
Por[:] = np.nan
S_w = np.zeros(len(Sola['phi_mod']))
S_w[:] = np.nan

# Dynamic reservoir properties from log data
for n in range(len(Sola['phi_mod'])):

    # Normalizing the solid volume fractions according to the shale log
    c_l[n] = (1-Sola['lit_mod'].iloc[n]) * Vc_l/Vtot_r
    c_s[n] = (1-Sola['lit_mod'].iloc[n]) * Vc_s/Vtot_r
    q_l[n] = (1-Sola['lit_mod'].iloc[n]) * Vq_l/Vtot_r
    q_s[n] = (1-Sola['lit_mod'].iloc[n]) * Vq_s/Vtot_r
    k_s[n] = Sola['lit_mod'].iloc[n] * Vk_s/Vtot_sh
    k_l[n] = Sola['lit_mod'].iloc[n] * Vk_l/Vtot_sh

    Por[n] = Sola['phi_mod'].iloc[n]
    S_w[n] = Sola['sw_mod'].iloc[n]

# Reservoir properties
Por_c = 0.66 # Critical porosity
fluidMix = 'reuss'


# Modeling

IFc = np.linspace(0,1,6) # Iso-frame value for the suspended chalk
IFs = 0.2 # Iso-frame value for the suspended shale
IFq = 0.2 # Iso-frame value for the suspended quartz

K_sus = np.nan*np.ones((Por.size,IFc.size))
K_IF = np.nan*np.ones((Por.size,IFc.size))
G_IF = np.nan*np.ones((Por.size,IFc.size))
M_IF = np.nan*np.ones((Por.size,IFc.size))
Vp_IF = np.nan*np.ones((Por.size,IFc.size))
Vs_IF = np.nan*np.ones((Por.size,IFc.size))
AI_IF = np.nan*np.ones((Por.size,IFc.size))
PS_IF = np.nan*np.ones((Por.size,IFc.size))
Rho = np.nan*np.ones((Por.size,IFc.size))

K_f = S_w/K_w + (1-S_w)/K_o
Rho =  Por*((Rho_w*S_w) + (Rho_o*(1-S_w))) + ((1-Por)*((c_s+c_l)*Rho_ca + Rho_k*(k_s+k_l) + Rho_q*(q_s+q_l))) 

#for n in range(len(IFc)):
n=2
K_sus[:,n] = ( Por/Por_c*K_f + (1-Por/Por_c)*( (c_s*(1-IFc[n])/K_ca) + (q_s*(1-IFq)/K_q) + (k_s*(1-IFs)/K_k) ) )**(-1) * (Por/Por_c + (1-Por/Por_c)*( c_s*(1-IFc[n]) + q_s*(1-IFq) + k_s*(1-IFs)) )

K_IF[:,n] = ( ((Por/Por_c)/(K_sus[:,n] + (4/3)*G_ca)) + (1-(Por/Por_c))* ( (((1-IFc[n])*c_s + (1-IFq)*q_s + (1-IFs)*k_s)/(K_sus[:,n] + (4/3)*G_ca)) + ((IFc[n]*c_s+c_l) / (K_ca + (4/3)*G_ca)) + ( (IFq*q_s+q_l) / (K_q + (4/3)*G_ca)) + ((IFs*k_s+k_l) / (K_k + (4/3)*G_ca)) ))**(-1) - (4/3)*G_ca

zeta = (G_ca/6) * ((9*K_ca + 8*G_ca)/(K_ca + 2*G_ca));

G_IF[:,n] = ( ((Por/Por_c)/zeta) +  (1-(Por/Por_c))*( (((1-IFc[n])*c_s + (1-IFq)*q_s + (1-IFs)*k_s)/(zeta)) + ((IFc[n]*c_s+c_l)/(G_ca + zeta)) + ((IFq*q_s+q_l)/(G_q + zeta)) + ( (IFs*k_s+k_l) / (G_k + zeta))) )**(-1) - zeta

M_IF[:,n] = K_IF[:,n] + (4/3)*G_IF[:,n]

Vp_IF[:,n] = np.sqrt((K_IF[:,n] + (4/3)*G_IF[:,n]) / Rho)
Vs_IF[:,n] = np.sqrt(G_IF[:,n] / Rho)

AI_IF[:,n] = Vp_IF[:,n] * Rho
PS_IF[:,n] = Vp_IF[:,n] / Vs_IF[:,n]




    #   Calculate reflectivities from model parameters

Sola['vp_mod'] = Vp_IF[0,2]
Sola['vs_mod'] = Vs_IF[0,2]

#phi_mod[6] = Por[0]
#rho_mod[6] = Rho[0]*1000.
#vp_mod[6] = Vp_IF[0,2]*1000.
#vs_mod[6] = Vs_IF[0,2]*1000.



vp_mod = np.array([resSec['Vp'].loc[resSec['F_label']=='Hidra'].mean() ,Vp_IF[0,2]*1000., (resSec['Vp'].loc[resSec['F_label']=='U. Tuxen'].mean() + resSec['Vp'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])  # P-wave velocity (m/s)
vs_mod = np.array([resSec['Vs'].loc[resSec['F_label']=='Hidra'].mean() , Vs_IF[0,2]*1000., (resSec['Vs'].loc[resSec['F_label']=='U. Tuxen'].mean()+ resSec['Vs'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])  # S-wave velocity (m/s)
rho_mod= np.array([resSec['RHOZ_edt'].loc[resSec['F_label']=='Hidra'].mean() , Rho[0]*1000., (resSec['RHOZ_edt'].loc[resSec['F_label']=='U. Tuxen'].mean()+resSec['RHOZ_edt'].loc[resSec['F_label']=='L. Tuxen'].mean())/2.])         # Density (g/cc)

dz_min = 0.0   # Minimum thickness of Layer 2 (m)
dz_max = 60.0  # Maximum thickness of Layer 2 (m)
dz_step= 1.0   # Thickness step from trace-to-trace (normally 1.0 m)
#   Trace Parameters
tmin = 0.0
tmax = 0.3

##   Plot Parameters
min_plot_time = 0.12
max_plot_time = 0.22
excursion = 2



nlayers = len(vp_mod)
nint = nlayers - 1
nmodel = int((dz_max-dz_min)/dz_step+1)


#   Calculate reflectivities from model parameters
rc_int = calc_rc(vp_mod, rho_mod)

syn_zo = []
rc_zo = []
lyr_times = []
for model in range(0, nmodel):

        #   Calculate interface depths
        z_int = [500.0]
        z_int.append(z_int[0]+dz_min+dz_step*model)

        #   Calculate interface times
        t_int = calc_times(z_int, vp_mod)
        lyr_times.append(t_int)

        #   Digitize 3-layer model
        nsamp = int((tmax-tmin)/dt) + 1
        t = []
        for i in range(0,nsamp):
            t.append(i*dt)

        rc = digitize_model(rc_int, t_int, t)
        rc_zo.append(rc)
    
        #   Convolve wavelet with reflectivities
        syn_buf = np.convolve(rc, wavelet_seismic, mode='same')     ###### WAVELET CONVOLUTION HAPPENS HERE
        syn_buf = list(syn_buf)
        syn_zo.append(syn_buf)
        #print("finished step %i" % (model))

syn_zo = np.array(syn_zo)
t = np.array(t)
lyr_times = np.array(lyr_times)
lyr_indx = np.array(np.round(lyr_times/dt), dtype='int16')

# Use the transpose because rows are traces;
# columns are time samples.
max_amp = np.zeros(syn_zo.shape[0])
thickness = np.zeros(syn_zo.shape[0])
for i in range(syn_zo.shape[0]):
        max_amp[i] = np.min(syn_zo[i,:])
#max_amp.argmax()

#tuning_trace = max_amp.argmax()#np.argmin((syn_zo.T)) % syn_zo.T.shape[1]
#tuning_thickness = max_amp.argmax()
tuning_trace = np.argmax(np.abs(syn_zo.T)) % syn_zo.T.shape[1]
#tuning_thickness = tuning_trace * dz_step

#   Plotting Code
[ntrc, nsamp] = syn_zo.shape

fig = plt.figure(figsize=(12, 14))
fig.set_facecolor('white')

gs = gridspec.GridSpec(3, 1, height_ratios=[1, 1, 1])

ax0 = fig.add_subplot(gs[0])
ax0.plot(lyr_times[:,0], color='k', lw=3,ls='--')
ax0.plot(lyr_times[:,1], color='k', lw=3,ls='--')
ax0.set_ylim((min_plot_time,max_plot_time))
ax0.invert_yaxis()
ax0.set_xlabel('Thickness (m)')
ax0.set_ylabel('Time (s)')
#plt.text(2,
#        min_plot_time + (lyr_times[0,0] - min_plot_time)/2.+0.01,
#        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Sola' ,np.str(np.int(vp_mod[0])), np.str(np.int(vs_mod[0])),np.str(np.int(rho_mod[0]))),
#        fontsize=12)
#plt.text(dz_max/dz_step - 2,
#        lyr_times[-1,0] + (lyr_times[-1,1] - lyr_times[-1,0])/2.+0.01,
#        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} g/cm$^3$'.format('Tuxen' ,np.str(np.int(vp_mod[1])), np.str(np.int(vs_mod[1])),np.str(np.int(rho_mod[1]))),
#        fontsize=12,
#        horizontalalignment='right')
#plt.text(2,
#        lyr_times[0,0] + (max_plot_time - lyr_times[0,0])/2.+0.02,
#        '{}\n Vp:{} m/s \n Vs:{} m/s \n Rho:{} kg/m$^3$'.format('Valhall' ,np.str(np.int(vp_mod[2])), np.str(np.int(vs_mod[2])),np.str(np.int(rho_mod[2]))),
#        fontsize=12)
#plt.gca().xaxis.tick_top()
#plt.gca().xaxis.set_label_position('top')
#ax0.set_xlim((-excursion, ntrc+excursion))

#tuning_trace = 20+np.argmax(syn_zo[:,lyr_indx[:,0]][20:40,0])

ax1 = fig.add_subplot(gs[1])
ax1.imshow(syn_zo.T, cmap="seismic", extent = (0,60,max_plot_time,min_plot_time) ,vmin=-.15,vmax=.15,  aspect='auto')
ax1.plot(lyr_times[:,0], color='k', lw=3,ls='--')
ax1.plot(lyr_times[:,1], color='k', lw=3,ls='--')
#    plot_vawig(ax1, syn_zo, t, excursion, highlight=tuning_trace)
#    ax1.plot(lyr_times[:,0], color='blue', lw=3)
#    ax1.plot(lyr_times[:,1], color='red', lw=3)
#    ax1.set_ylim((min_plot_time,max_plot_time))
#    ax1.invert_yaxis()
#    ax1.set_xlabel('Thickness (m)')
#    ax1.set_ylabel('Time (s)')

#    ax2 = fig.add_subplot(gs[2])
#    ax2.plot(syn_zo[:,lyr_indx[:,0]], color='blue')
#    ax2.set_xlim((-excursion, ntrc+excursion))
#    ax2.axvline(tuning_trace, color='k', lw=1)
#    ax2.grid()
#    ax2.set_title('Upper interface amplitude')
#    ax2.set_xlabel('Thickness (m)')
#    ax2.set_ylabel('Amplitude')
#    plt.text(tuning_trace -15,
#            plt.ylim()[0] * 1.1-0.02,
#            'tuning thickness = {0} m'.format(str(tuning_trace),
#            fontsize=16))
#




In [ ]:
Sola['lit_mod']

In [ ]:
def f(x_deg):


    rot = bg.filters.rotate_phase(wavelet_well,x_deg,degrees=True)


    

        #   Some handy constants
    nlayers = len(vp_mod)
    #nint = nlayers - 1
    nangles = 35
    dt =  0.0001 #np.diff(t_plot_twt)[0]#


    #   Calculate reflectivities from model parameters

    rc_zoep_pp = []
    theta1 = []
    dummy_rc = np.zeros([9,4,4])

    for i in range(0,nangles):
        theta1_buf = i*theta1_step + theta1_min
        for i in range (vp_mod.size-1):
            dummy_rc[i,:,:] = mlo.rc_zoep(vp_mod[i], vs_mod[i], rho_mod[i], vp_mod[i+1], vs_mod[i+1], rho_mod[i+1], theta1_buf)

        theta1.append(theta1_buf)
        rc_zoep_pp.append((dummy_rc[0,0,0],dummy_rc[1,0,0],dummy_rc[2,0,0],
                          dummy_rc[3,0,0],dummy_rc[4,0,0],dummy_rc[5,0,0],
                          dummy_rc[6,0,0],dummy_rc[7,0,0],dummy_rc[8,0,0]))

    rc_zoep_pp = np.array([rc_zoep_pp])

    #rc_zoep_pp = np.array([rc_zoep_pp])
    stack1 = np.squeeze(  np.sum(rc_zoep_pp[:,5:15,:],axis=1) / rc_zoep_pp[:,5:15,:].shape[2])
    stack2 = np.squeeze(np.sum(rc_zoep_pp[:,15:25,:],axis=1) / rc_zoep_pp[:,15:25,:].shape[2])
    stack3 = np.squeeze(np.sum(rc_zoep_pp[:,25:35,:],axis=1) / rc_zoep_pp[:,25:35,:].shape[2])
    #stack4 = np.squeeze(np.sum(rc_zoep_pp[:,24:30,:],axis=1) / rc_zoep_pp[:,24:30,:].shape[2])



    rc_zoep_pp = np.array([stack1,stack2,stack3])



    syn_zoep_pp_well = [] 
    syn_zoep_pp_seis = []
    angle_dummy = np.array([5,15,25])
    j=0
    for angle in range(5, nangles,10):
            nlayers = len(rc_zoep_pp[j])
            nint = nlayers
            nsamp = len(t_plot_twt)
            rc = list(np.zeros(nsamp,dtype='float'))
            lyr = 0

            for i in range(0, nsamp):
                for k in range(0,tops_twt.size-1):
                    if t_plot_twt[i] == tops_twt[k]:
                        rc[i] = rc_zoep_pp[j][k-1]
                    if lyr > nint:
                        break
            #k=0
            #   Convolve wavelet with reflectivities

            syn_buf = np.convolve(rc, rot[:,j], mode='same')
            syn_buf = list(syn_buf)
            syn_zoep_pp_well.append(syn_buf)

            syn_buf_seis = np.convolve(rc, rot[:,j], mode='same')
            syn_buf_seis = list(syn_buf_seis)
            syn_zoep_pp_seis.append(syn_buf_seis)
            j=j+1
            #print("Calculated angle %i" % (angle))

    #    Convert data arrays from lists/tuples to numpy arrays    
    syn_zoep_pp_well = np.array(syn_zoep_pp_well)
    syn_zoep_pp_seis = np.array(syn_zoep_pp_seis)
    rc_zoep_pp = np.array(rc_zoep_pp)

    #   Copy convolved top/base reflectivity values to Lists for easier plotting
    [ntrc, nsamp] = syn_zoep_pp_well.shape
    line1 = []
    line2 = []
    line3 = []
    line4 = []
    line5 = []
    line6 = []
    line7 = []
    line8 = []
    line9 = []
    line10 = []

    Yzoep = np.array(rc_zoep_pp[:,0])
    Yzoep = Yzoep.reshape((ntrc, 1))

    #Yconv = np.array(line1)
    #Yconv = Yconv.reshape((ntrc, 1))
    #
    #ones = np.ones(ntrc)
    #ones = ones.reshape((ntrc,1))

    sintheta2 = np.sin(np.radians(np.arange(0, ntrc)))**2
    sintheta2 = sintheta2.reshape((ntrc, 1))


    tops_lines = np.zeros([tops_idx.size , syn_zoep_pp_well.shape[0]])    


    for i in range(0, ntrc):
        line1.append(syn_zoep_pp_well[i,np.int(tops_idx[0])])
        line2.append(syn_zoep_pp_well[i,np.int(tops_idx[1])])
        line3.append(syn_zoep_pp_well[i,np.int(tops_idx[2])])
        line4.append(syn_zoep_pp_well[i,np.int(tops_idx[3])])
        line5.append(syn_zoep_pp_well[i,np.int(tops_idx[4])])
        line6.append(syn_zoep_pp_well[i,np.int(tops_idx[5])])
        line7.append(syn_zoep_pp_well[i,np.int(tops_idx[6])])
        line8.append(syn_zoep_pp_well[i,np.int(tops_idx[7])])
        line9.append(syn_zoep_pp_well[i,np.int(tops_idx[8])])     
        line10.append(syn_zoep_pp_well[i,np.int(tops_idx[9])])

    line1=np.squeeze(np.array([line1]))
    line2=np.squeeze(np.array([line2]))
    line3=np.squeeze(np.array([line3]))
    line4=np.squeeze(np.array([line4]))
    line5=np.squeeze(np.array([line5]))
    line6=np.squeeze(np.array([line6]))
    line7=np.squeeze(np.array([line7]))
    line8=np.squeeze(np.array([line8]))
    line9=np.squeeze(np.array([line9]))
    line10=np.squeeze(np.array([line10]))

    syn_zoep_cdp_seis = syn_zoep_pp_seis
    syn_zoep_cdp_well = syn_zoep_pp_well
    


    fig = plt.figure(figsize=(15,20))#set_size(width, subplots=(5,2)))
    fig.tight_layout
    gs1 = gridspec.GridSpec(nrows=12, ncols=6, figure=fig,hspace =1 ,wspace=0.25)#,width_ratios=np.array([.5,.5,.5,.5,.5,.5,.5]))



    ax2 = fig.add_subplot(gs1[0:7, 0])


#mlo.plot_vawig(ax3, syn_zoep_cdp_well[:,lc_idx:] , t_plot_lc, 1)
#ax3.


    mlo.plot_vawig(ax2, gather_sm_bo2.T[:,lc_idx:]/1e3 , t_plot_lc, 1)
    ax2.set_ylim(t_plot_lc.min(),t_plot_lc.max())
    ax2.axhline(tops_twt[6], color = 'green', lw=1,alpha = 0.3, label = 'Top Sola')
    ax2.axhline(tops_twt[7], color = 'red',   lw=1,alpha = 0.3, label = 'Upper Tuxen')
    ax2.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
    ax2.axhline(tops_twt[9], color = 'blue',  lw=1,alpha = 0.3, label = 'Top Valhall')
    ax2.invert_yaxis()
    ax2.set_title('BO-2X CMP')
    ax2.xaxis.tick_bottom()
    #ax2.set_yticklabels('')
    ax2.set_xlim(-1,3)
    ax2.set_xticks(np.array([0,1,2]));
    #ax2.set_xticklabels(['']);


    ax3 = fig.add_subplot(gs1[0:7, 1])

    mlo.plot_vawig(ax3, syn_zoep_cdp_well[:,lc_idx:] , t_plot_lc, 1)
    ax3.set_ylim(t_plot_lc.min(),t_plot_lc.max())
    ax3.axhline(tops_twt[6], color = 'green', lw=1,alpha = 0.3, label = 'Top Sola')
    ax3.axhline(tops_twt[7], color = 'red',   lw=1,alpha = 0.3, label = 'Upper Tuxen')
    ax3.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
    ax3.axhline(tops_twt[9], color = 'blue',  lw=1,alpha = 0.3, label = 'Top Valhall')
    ax3.invert_yaxis()
    ax3.set_title('Synthetic Blocky AVO')
    ax3.xaxis.tick_bottom()
    ax3.xaxis.set_label_position('bottom')
    ax3.set_yticklabels('')
    ax3.set_xlim(-1,3)
    ax3.set_xticks(np.array([0,1,2]));
    
    #syn_zo = np.convolve(np.array(resSec['RCZO'])[:,lc_idx:],rot[:,0],'same')
    #ax4 = fig.add_subplot(gs1[0:7, 2])
    #ax4.plot(syn_zo,t_plot_twt,c='orange')
    #ax4.set_ylim(t_plot_lc.min(),t_plot_lc.max())
    #ax4.axhline(tops_twt[6], color = 'green', lw=1,alpha = 0.3, label = 'Top Sola')
    #ax4.axhline(tops_twt[7], color = 'red',   lw=1,alpha = 0.3, label = 'Upper Tuxen')
    #ax4.axhline(tops_twt[8], color = 'm',     lw=1,alpha = 0.3, label = 'Lower Tuxen')
    #ax4.axhline(tops_twt[9], color = 'blue',  lw=1,alpha = 0.3, label = 'Top Valhall')
    #ax4.invert_yaxis()
    #ax4.set_title('Synthetic ZO')
    #ax4.xaxis.tick_bottom()
    #ax4.xaxis.set_label_position('bottom')
    #ax4.set_yticklabels('')
    #ax4.set_xlim([-.25,.25])

    
    
    ax5 = fig.add_subplot(gs1[0:3, 3:5])
    ax5.plot(wavelet_well[:,0], label = 'Original')
    ax5.plot(rot[:,0], label='Phase Rotated')
    ax5.legend()
    
    


    for i in range (tops_twt.size-4):
        ax1.axhline(tops_twt[i],   lw=1, alpha = 0.3)
        ax2.axhline(tops_twt[i],   lw=1, alpha = 0.3)
        ax3.axhline(tops_twt[i],   lw=1, alpha = 0.3)

    cc1 = np.round(np.corrcoef(syn_zoep_cdp_well[0,lc_idx:], gather_sm_bo2.T[0,lc_idx:])[0][1],3)
   # cc2 = np.round(np.corrcoef(syn_zo[lc_idx:], gather_sm_bo2.T[0,lc_idx:],3))
    ax3.text(1.,t_plot_lc.max()+0.02,'Corr. coef = {}'.format(cc1))
 #   ax4.text(0,t_plot_twt.max()+0.02,'Corr. coef = {}'.format(cc2))
    
interactive_plot = interactive(f, x_deg=(0, 180, 1),continuous_update=False,)

output = interactive_plot.children[0]
output.layout.height = '500x'
interactive_plot

In [ ]:
np.corrcoef(syn_zoep_cdp_well[0,lc_idx:], gather_sm_bo2.T[0,lc_idx:])[0][1]

In [ ]:
twt_idx[0].shape

#twt_idx[0][9]
#syn_zoep_cdp_well.shape

In [ ]:
    w = widgets.Dropdown(
    options={'One': 1, 'Two': 2, 'Three': 3},
    value=1,
    description='Number:',
    )
    display(w)
    
    if w.value == 1:
        plt.plot(wavelet_well)
    else:
        plt.plot(wavelet_seismic)

In [ ]:
w.value

In [ ]:
from ipywidgets import VBox, SelectionSlider, Output, Dropdown
from IPython.display import display, clear_output
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget
plt.ioff()
plt.clf()
slider = SelectionSlider(options=np.arange(0.5, 10, 0.5), description="a", orientation='horizontal')
dropper = Dropdown(options=['linear', 'squared', 'cubed'], value='linear', description='Function:')
output = Output()
functions = {
    "linear": lambda x: x,
    "squared": lambda x: x*x,
    "cubed": lambda x: x*x*x
}
fig, axs = plt.subplots(1, figsize=(5,5), num=10)
fig.canvas.layout.height = "500px" # this stops matplotlib auto resizing the canvas on redraw
x = np.arange(-10, 10, 0.1)
line = axs.plot(x, x, color='red')
#def printer(change):
#    with output:
#        print(dropper.value)
def update_lineplot(change):
    func = functions[dropper.value]
    a = slider.value
    y = a*func(x)
    line[0].set_ydata(y)
    fig.canvas.draw()
    fig.canvas.flush_events()
dropper.observe(update_lineplot)
slider.observe(update_lineplot)
wid = VBox([dropper, slider, fig.canvas, output])
display(wid)

In [ ]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Testing matplotlib interactions with a simple plot
fig = plt.figure()
plt.plot(np.sin(np.linspace(0, 20, 100)));

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('kenneth_wavelets_5-35valdemar2016.txt',skiprows=0, header=None, sep='\t')
df = np.array(df)
df.shape

In [ ]:
plt.plot(df[:,0],df[:,3])

In [ ]:
resSec['RCZO']

### Polarity plot

In [ ]:

"""
Author: Matt Hall, Agile Scientific
Licence: Apache 2.0, please re-use this code!
To use the CLI type this on the command line:
    python polarity_cartoon.py --help
   
There is a web app running at agile.geosci.ai/polarity
"""
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

# Install click and bruges with pip.
import click
import bruges as bg


def get_colour(cmap, frac):
    """
    Decide whether to make white or black labels.
    """
    cmap = get_cmap(cmap)
    return 'k' if (np.mean(cmap(frac)[:3]) > 0.5) else 'w'


def make_synthetic(size=256, top=0.4, base=0.6, value=1, freq=25, phase=0):
    """Make a synthetic. Return the wavelet, the model, the RC, and the synthetic.
    """
    v = np.ones(size) - value

    v[int(top*size):int(base*size)] = value
    rc = np.diff(v)

    w = bg.filters.ricker(0.256, 0.001, freq)

    if phase != 0:
        w = bg.filters.rotate_phase(w, phase, degrees=True)

    syn = np.convolve(rc, w, mode='same')
    return w, v, rc, syn


#@click.command()
#@click.option('--layer', default='hard', help='Hard or soft layer.')
#@click.option('--polarity', default='normal', help='Normal or reverse polarity.')
#@click.option('--freq', default='med', help='Lo, med, hi or vhi frequency.')
#@click.option('--phase', default=0, help='Phase angle in degrees')
#@click.option('--style', default='synthetic', help='Ramp or synthetic')
#@click.option('--cmap', default='gray', help='Matplotlib cmap.')
def polarity_cartoon(layer='hard',
                     polarity='normal',
                     freq='low',
                     phase=120,
                     style='ramp',
                     cmap=None,
                     ):
    """
    Plot a polarity cartoon. See agile.geosci.ai/polarity for more info.
    """
    freqs = {'vhi': 60, 'hi': 30, 'med': 15, 'lo': 7.5,
             'vhigh': 60, 'high': 30, 'medium': 15, 'low': 7.5,
             'mod': 15, 'mid': 15}
    backgr = 'soft' if layer == 'hard' else 'hard'
    value = 1 if layer == 'hard' else 0
    size, top, base = 256, 0.4, 0.6

    _, v, _, syn = make_synthetic(size, top, base, value, freq=freqs[freq], phase=phase)

    if polarity.lower() not in ['normal', 'seg', 'usa', 'us', 'canada']:
        syn *= -1

    if style == 'ramp':
        # cbar is a ramp.
        cbar = np.linspace(-1, 1, size).reshape(-1, 1)
    else:
        # cbar is the synthetic.
        cbar = syn.reshape(-1, 1)

    gs = {'width_ratios':[2,2,2,1]}

    fig, axs = plt.subplots(ncols=4,
                            figsize=(6, 4),
                            gridspec_kw=gs,
                            facecolor='w', sharey=True,
                            )

    # Plot the earth model.
    ax = axs[0]
    cmap_ = 'Greys'
    ax.imshow(v.reshape(-1, 1), aspect='auto', cmap=cmap_, vmin=-1.5, vmax=1.5)
    ax.axhline(top*size, c='w', lw=4)
    ax.axhline(base*size, c='w', lw=4)
    ax.axvline(0.55, c='w', lw=6)  # Total hack to adjust RHS
    ax.text(0, size/4.75, backgr, ha='center', va='center', color=get_colour(cmap_, (1-value)*256), size=25)
    ax.text(0, size/2+0.75, layer, ha='center', va='center', color=get_colour(cmap_, (value)*256), size=25)

    # Plot the impedance diagram.
    ax = axs[1]
    cmap_ = 'Greys'
    ax.imshow(v.reshape(-1, 1), aspect='auto', cmap=cmap_, vmin=0, vmax=2)
    ax.axvline(-0.5, c=(0.58, 0.58, 0.58), lw=50)
    ax.text(0.45, 2*size/8, 'imp', ha='right', va='center', color='k', size=25)
    ax.annotate("", xy=(0.33, size/8), xytext=(0, size/8), arrowprops=dict(arrowstyle="->", connectionstyle="arc3"))

    # Plot the waveform.
    ax = axs[2]
    y = np.arange(syn.size)
    ax.plot(syn, y, 'k')
    ax.fill_betweenx(y, syn, 0, where=syn>0, color='k')
    ax.invert_yaxis()
    ax.text(0.65, size/8, '+', ha='center', va='center', size=30)
    ax.text(-0.9, size/8, '–', ha='center', va='center', size=40)

    # Plot the colourbar.
    ax = axs[3]
    cmap = cmap or 'gray'
    frac = 1/8
    top_col = get_colour(cmap, frac)
    bot_col = get_colour(cmap, 7*frac)
    ax.imshow(cbar, cmap=cmap, aspect='auto')
    if style == 'ramp':
        ax.text(0, frac*size, '+', ha='center', va='center', color=top_col, size=30)
        ax.text(0, 7*frac*size, '–', ha='center', va='center', color=bot_col, size=40)

    # Make final adjustments to subplots and figure.
    for ax in axs:
        ax.set_axis_off()

    plt.subplots_adjust(left=0.1)
    plt.show()

    return


if __name__ == '__main__':
    polarity_cartoon()

plt.show()
if savefig == True:
    plt.savefig(fig_path+'zerophase.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')

## Radon Transform Plot

In [ ]:
path_hrs

In [ ]:
il_radon = 7986
xl_radon = 7926

file_raw   = 'gathers_SE_2016_all_lines.sgy'
file_radon = 'valdemar2016_radon.sgy'
file_nosie = 'valdemar2016_radon_noise.sgy'

with segyio.open('//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/seismic/Valdemar_4D_pre-stack/SE_cubes_PSpro/' + file_raw, iline=189) as f: # iline=189 , xline=189
    offsets = f.offsets
    gather_raw = np.asarray([np.copy(x) for x in f.iline[il_radon,:]])

with segyio.open(path_hrs + file_radon, iline=189) as f: # iline=189 , xline=189
    offsets = f.offsets
    gather_radon = np.asarray([np.copy(x) for x in f.iline[il_radon,:]])
    
with segyio.open(path_hrs + file_nosie, iline=189) as f: # iline=189 , xline=189
    offsets = f.offsets
    gather_noise = np.asarray([np.copy(x) for x in f.iline[il_radon,:]])

    
gather_raw = gather_raw[:,np.argwhere(f.xlines==xl_radon)[0][0],:]
gather_radon = gather_radon[:,np.argwhere(f.xlines==xl_radon)[0][0],:]
gather_noise = gather_noise[:,np.argwhere(f.xlines==xl_radon)[0][0],:]




In [ ]:


# Trimming traces to reservoir zone:
time = np.arange(1.5,3.004,0.004)
t_start = 2.26
index_start = (np.abs(time-t_start)).argmin()
t_stop = 2.5
index_stop = (np.abs(time-t_stop)).argmin()

#print(gather_bo2.shape, time.shape)

gather_raw =     gather_raw[:,index_start:index_stop].T
gather_radon = gather_radon[:,index_start:index_stop].T
gather_noise = gather_noise[:,index_start:index_stop].T



In [ ]:
x =         np.linspace(0,1,gather_raw.shape[0])
xnew =      np.linspace(0,1,t_plot_twt.shape[0])

gather_sm_raw =   np.zeros([xnew.shape[0], gather_raw.shape[1]])
gather_sm_radon = np.zeros([xnew.shape[0], gather_raw.shape[1]])
gather_sm_noise = np.zeros([xnew.shape[0], gather_raw.shape[1]])

for i in range (gather_sm_raw.shape[1]):
    gather_sm_raw[:,i]   = interpolate.splev(xnew, interpolate.splrep(x, gather_raw[:,i],s=0) ,der=0) #use interpolation function returned by `interp1d`
    gather_sm_radon[:,i] = interpolate.splev(xnew, interpolate.splrep(x, gather_radon[:,i],s=0) ,der=0)
    gather_sm_noise[:,i] = interpolate.splev(xnew, interpolate.splrep(x, gather_noise[:,i],s=0) ,der=0)

In [ ]:
gather_sm_raw   = gather_sm_raw   / np.array([[gather_sm_raw],[gather_sm_radon],[gather_sm_noise]]).max()
gather_sm_radon = gather_sm_radon / np.array([[gather_sm_raw],[gather_sm_radon],[gather_sm_noise]]).max()
gather_sm_noise = gather_sm_noise / np.array([[gather_sm_raw],[gather_sm_radon],[gather_sm_noise]]).max()

#np.array([[gather_sm_raw],[gather_sm_radon],[gather_sm_noise]]).max()
#np.array([[gather_sm_raw],[gather_sm_radon],[gather_sm_noise]]).min()

fig = plt.figure(figsize=(7,8))#figsize=set_size(width, subplots=(5,2)))
fig.tight_layout
gs1 = gridspec.GridSpec(nrows=12, ncols=8, figure=fig,hspace =1 ,wspace=0.25)#,width_ratios=np.array([.5,.5,.5,.5,.5,.5,.5]))

ax0 = fig.add_subplot(gs1[0:7, 0:2])
ax0.imshow(gather_sm_raw,aspect='auto',interpolation = 'spline36',cmap = 'seismic_r', extent = (490,3090 ,t_stop*1000., t_start*1000.))
#ax0.invert_yaxis()

ax0.set_xticks(np.arange(490,3090,500));
ax0.set_title('Raw')
ax0.set_xlabel('Offset [m]')
ax0.set_ylabel('TWT [ms]')

ax1 = fig.add_subplot(gs1[0:7, 2:4])
ax1.imshow(gather_sm_radon,aspect='auto',interpolation = 'spline36',cmap = 'seismic_r', extent = (490,3090 ,t_stop*1000., t_start*1000.))
ax1.set_title('Radon')
ax1.set_yticklabels('')
ax1.set_xlabel('Offset [m]')

ax2 = fig.add_subplot(gs1[0:7, 4:6])
im2 = ax2.imshow(gather_sm_noise,aspect='auto',interpolation = 'spline36',cmap = 'seismic_r', extent = (490,3090 ,t_stop*1000., t_start*1000.))
ax2.set_title('Noise Model');
#fig.colorbar(im2, ax=ax2, orientation='vertical',label = 'Amplitude')
ax2.set_yticklabels('');
ax2.set_xlabel('Offset [m]')

#ax3 = fig.add_subplot(gs1[0:7, 6])
#fig.colorbar(im2, ax=ax3, orientation='vertical',label = 'Amplitude')


ax0.axhline(2330 , lw = 6, color = 'white')
ax0.axhline(2358 , lw = 6, color = 'white')
ax0.axhline(2400 , lw = 6, color = 'white')
ax0.axhline(2450 , lw = 6, color = 'white')
ax1.axhline(2330 , lw = 6, color = 'white')
ax1.axhline(2358 , lw = 6, color = 'white')
ax1.axhline(2400 , lw = 6, color = 'white')
ax1.axhline(2450 , lw = 6, color = 'white')
ax2.axhline(2330 , lw = 6, color = 'white')
ax2.axhline(2358 , lw = 6, color = 'white')
ax2.axhline(2400 , lw = 6, color = 'white')
ax2.axhline(2450 , lw = 6, color = 'white')



ax0.axhline(2330 , lw = 3, color='magenta')
ax0.axhline(2358 , lw = 3, color='green' ) 
ax0.axhline(2400 , lw = 3, color='orange') 
ax0.axhline(2450 , lw = 3, color='blue'  )



ax1.axhline(2330 , lw = 3, color='magenta')
ax1.axhline(2358 , lw = 3, color='green' ) 
ax1.axhline(2400 , lw = 3, color='orange') 
ax1.axhline(2450 , lw = 3, color='blue'  )
ax2.axhline(2330 , lw = 3, color='magenta', label = 'Base Chalk')
ax2.axhline(2358 , lw = 3, color='green',   label = 'Top Tuxen')
ax2.axhline(2400 , lw = 3, color='orange',  label = 'Top Valhall')
ax2.axhline(2450 , lw = 3, color='blue',    label = 'BCU')

ax2.legend()



if savefig == True:
    plt.savefig(fig_path+'radon.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')
plt.show()

In [ ]:

with segyio.open(path_data+'/seismic/'+'Tau-P_arb_line_7900_8448.sgy', ignore_geometry=True) as f: # iline=189 , xline=189
    arbitrary_line = np.asarray([np.copy(x) for x in f.trace])

In [ ]:
il_bo2 = 8366
xl_bo2 = 7926

#file = 'valdemar2016_5-35deg.sgy'
#trace range : 6077 , 6271
#4704, 4899
#107407 - 106231


with segyio.open(path_data+'/seismic/'+'Tau-P_arb_line_7900_8448.sgy', ignore_geometry=True) as f: # iline=189 , xline=189
    arbitrary_line_1 = np.asarray([np.copy(x) for x in f.trace])

time = np.arange(1.5,3.004,0.004)
t_start = 2.16
index_start = (np.abs(time-t_start)).argmin()
t_stop = 2.3
index_stop = (np.abs(time-t_stop)).argmin()

#trace_range = np.array([4725, 4800])
#trace_range = np.array([106650, 106700])

In [ ]:
trace_range = np.array([107250, 107285])
arbitrary_line = arbitrary_line_1[trace_range[0]:trace_range[1],index_start:index_stop]

In [ ]:

newcmp = matplotlib.colors.LinearSegmentedColormap.from_list("", ['red','white','white','blue'])#["blue","white","white","orange"])

plt.figure(figsize=set_size(.15*width,6))
           
plt.imshow(arbitrary_line.T, aspect='auto', extent = (-.1,.1 , t_stop*1000.  , t_start * 1000. ),cmap = newcmp, interpolation= 'spline36')
plt.hlines(tops_twt[5]*1000., -.1,.1,color = 'white', lw=6,   alpha=0.8)
plt.hlines(tops_twt[6]*1000., -.1,.1,color = 'white', lw=6,   alpha=0.8)
plt.hlines(tops_twt[7]*1000., -.1,.1,color = 'white', lw=6,   alpha=0.8)
plt.hlines(tops_twt[8]*1000., -.1,.1,color = 'white', lw=6,   alpha=0.8)



plt.hlines(tops_twt[5]*1000., -.1,.1,color = 'magenta', lw=3,   alpha=0.8, label='Base Chalk')
plt.hlines(tops_twt[6]*1000., -.1,.1,color = 'red',     lw=3,   alpha=0.8, label = 'Top Sola')
plt.hlines(tops_twt[7]*1000., -.1,.1,color = 'green',   lw=3,   alpha=0.8, label = 'Top Tuxen')
plt.hlines(tops_twt[8]*1000., -.1,.1,color = 'orange',  lw=3,   alpha=0.8, label = 'Top Valhall')   
plt.clim(-.12,.12)
plt.colorbar()
plt.ylabel('TWT [ms]')
plt.xlabel('Time [s]')
plt.legend()

if savefig == True:
    plt.savefig(fig_path+'taup_bo2x.pdf',format='pdf', bbox_inches='tight')
    print('FIGURE SAVED')

In [ ]:
arbitrary_line = arbitrary_line[6076:627]

In [ ]:
from segysak.segy import segy_loader, get_segy_texthead, segy_header_scan, segy_header_scrape, well_known_byte_locs
#
##ai_2010 = 'inverted2010_Zp.sgy'
##ai_2016 = 'inverted2016_Zp.sgy'
##seis_2016 = 'valdemar2016_poststack.sgy'
#
taup2016 = segy_loader(path_data+'/seismic/'+'Tau-P_arb_line_7900_8448.sgy',cdpx=181,cdpy=185, vert_domain='TWT', data_type='AMP')

In [ ]:
find_nearest(taup2016.cdp_x.data, utm_xy_bo[0,1])
#find_nearest(taup2016.cdp_y.data, utm_xy_bo[1,1])

In [ ]:
taup2016.cdp.data

In [ ]:
np.where(taup2016.cdp_x.data == 596898.100000)

In [ ]:
utm_xy_bo = np.array([[598533.0000,596898.100000,596935.1400], [  6185329.5000,  6184414.600000,6178863.0100] ]) # bo1, bo2, bo3


def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()    
    return idx


bo2_cdpx = find_nearest(taup2016.cdp.data, utm_xy_bo[0,1])
bo2_cdpy = find_nearest(taup2016.cdp.data, utm_xy_bo[1,1])


print(bo2_cdpx,bo2_cdpy)

In [ ]:
taup2016

In [ ]:
4704, 4899